In [1]:
import json
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def getAllCurrentEntityRevisions():
    results = query("""
        select
            max(entity_revision.id),
            max(entity.id),
            max(type.name),
            json_objectagg(entity_revision_field.field, entity_revision_field.value) as fields
        from entity
            join type on entity.type_id = type.id
            join entity_revision on entity_revision.id = entity.current_revision_id
            join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        group by entity_revision.id
    """)
    
    return [{
        "currentRevisionId": result[0],
        "entityId": result[1],
        "type": result[2],
        **json.loads(result[3])
    } for result in results]

getAllCurrentEntityRevisions()[0]

{'currentRevisionId': 1568,
 'entityId': 1567,
 'type': 'article',
 'title': 'Geschwindigkeit',
 'content': '[[{"col":24,"content":"Die Geschwindigkeit ist ein Ma\\u00df daf\\u00fcr, wie schnell sich ein K\\u00f6rper bewegt.\\n\\n\\u00a0\\n\\n\\n# Bekannte Ma\\u00dfeinheiten\\n\\n"}],[{"col":8,"content":"%%\\\\frac ms%%"},{"col":8,"content":"\\n\\n [Meter](/1681)  pro Sekunde"},{"col":8,"content":"\\n\\n%%1\\\\frac ms=1\\\\cdot3,6\\\\frac{\\\\mathrm{km}}h=3,6\\\\;\\\\frac{\\\\mathrm{km}}h%%"}],[{"col":8,"content":"\\n\\n%%\\\\frac{\\\\mathrm{km}}h%%"},{"col":8,"content":"\\n\\n [Kilometer](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n\\u00a0"}],[{"col":8,"content":"\\n\\nmph"},{"col":8,"content":"\\n\\n [Meilen](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n%%1\\\\frac{\\\\mathrm{km}}h=0,62\\\\;\\\\mathrm{mph}%%"}],[{"col":8,"content":"\\n\\nkn \\/ %%\\\\frac{\\\\mathrm{sm}}h%%"},{"col":8,"content":"\\n\\nKnoten \\/  [Seemeilen](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n%%1\\\\

In [2]:
def parseEdtrState(content):
    try:
        state = json.loads(content)
    except:
        return None
    
    if "plugin" not in state:
        return None
    
    return state

def stateHasPlugin(plugin, state):
    if isinstance(state, dict):
        if "plugin" in state and state["plugin"] == plugin:
            return True
        
        return any(stateHasPlugin(plugin, value) for value in state.values())
    elif isinstance(state, list):
        return any(stateHasPlugin(plugin, value) for value in state)
    else:
        return False

def contentHasPlugin(plugin, content):
    state = parseEdtrState(content)
    
    return stateHasPlugin(plugin, state) if state != None else False

content = getAllCurrentEntityRevisions()[0]["content"]

print(contentHasPlugin("text", content))
print(contentHasPlugin("important", content))

False
False


In [3]:
def printRevisions(revisions):
    def toMarkdown(revision):
        return "* [%s %s](https://serlo-staging.dev/%s) ([current revision %s](https://serlo-staging.dev/%s))" % (
            revision["type"], revision["entityId"], revision["entityId"],
            revision["currentRevisionId"], revision["currentRevisionId"]
        )
    
    code = "\n".join((toMarkdown(r) for r in revisions))
    display(Markdown(code))

def printRevisionsWithPlugin(plugin):
    display(Markdown(f"# Entites with plugin {plugin}"))
    
    revisions = getAllCurrentEntityRevisions()
    
    printRevisions((r for r in revisions if "content" in r and contentHasPlugin(plugin, r["content"])))

In [4]:
printRevisionsWithPlugin("important")
printRevisionsWithPlugin("blockquote")

# Entites with plugin important

* [course-page 143526](https://serlo-staging.dev/143526) ([current revision 144759](https://serlo-staging.dev/144759))
* [article 146841](https://serlo-staging.dev/146841) ([current revision 147378](https://serlo-staging.dev/147378))
* [event 145455](https://serlo-staging.dev/145455) ([current revision 147732](https://serlo-staging.dev/147732))
* [event 146675](https://serlo-staging.dev/146675) ([current revision 147742](https://serlo-staging.dev/147742))
* [text-exercise 60274](https://serlo-staging.dev/60274) ([current revision 147998](https://serlo-staging.dev/147998))
* [article 1781](https://serlo-staging.dev/1781) ([current revision 148027](https://serlo-staging.dev/148027))
* [article 148316](https://serlo-staging.dev/148316) ([current revision 148318](https://serlo-staging.dev/148318))
* [course-page 61686](https://serlo-staging.dev/61686) ([current revision 149204](https://serlo-staging.dev/149204))
* [course-page 61810](https://serlo-staging.dev/61810) ([current revision 149206](https://serlo-staging.dev/149206))
* [event 147532](https://serlo-staging.dev/147532) ([current revision 149743](https://serlo-staging.dev/149743))
* [event 147705](https://serlo-staging.dev/147705) ([current revision 149744](https://serlo-staging.dev/149744))
* [article 65164](https://serlo-staging.dev/65164) ([current revision 151250](https://serlo-staging.dev/151250))
* [course-page 128986](https://serlo-staging.dev/128986) ([current revision 152468](https://serlo-staging.dev/152468))
* [text-exercise 6705](https://serlo-staging.dev/6705) ([current revision 152977](https://serlo-staging.dev/152977))
* [event 149177](https://serlo-staging.dev/149177) ([current revision 153541](https://serlo-staging.dev/153541))
* [article 1949](https://serlo-staging.dev/1949) ([current revision 153839](https://serlo-staging.dev/153839))
* [article 154812](https://serlo-staging.dev/154812) ([current revision 154949](https://serlo-staging.dev/154949))
* [article 154665](https://serlo-staging.dev/154665) ([current revision 154991](https://serlo-staging.dev/154991))
* [course-page 127669](https://serlo-staging.dev/127669) ([current revision 155000](https://serlo-staging.dev/155000))
* [course-page 147269](https://serlo-staging.dev/147269) ([current revision 155019](https://serlo-staging.dev/155019))
* [course-page 73894](https://serlo-staging.dev/73894) ([current revision 156214](https://serlo-staging.dev/156214))
* [course-page 72987](https://serlo-staging.dev/72987) ([current revision 156302](https://serlo-staging.dev/156302))
* [course-page 156432](https://serlo-staging.dev/156432) ([current revision 156506](https://serlo-staging.dev/156506))
* [course-page 156410](https://serlo-staging.dev/156410) ([current revision 156519](https://serlo-staging.dev/156519))
* [text-exercise 60191](https://serlo-staging.dev/60191) ([current revision 156775](https://serlo-staging.dev/156775))
* [text-exercise 60216](https://serlo-staging.dev/60216) ([current revision 156782](https://serlo-staging.dev/156782))
* [course-page 156517](https://serlo-staging.dev/156517) ([current revision 156813](https://serlo-staging.dev/156813))
* [course-page 156507](https://serlo-staging.dev/156507) ([current revision 156855](https://serlo-staging.dev/156855))
* [course-page 156511](https://serlo-staging.dev/156511) ([current revision 156864](https://serlo-staging.dev/156864))
* [course-page 127666](https://serlo-staging.dev/127666) ([current revision 157162](https://serlo-staging.dev/157162))
* [article 152323](https://serlo-staging.dev/152323) ([current revision 157605](https://serlo-staging.dev/157605))
* [text-solution 101450](https://serlo-staging.dev/101450) ([current revision 158179](https://serlo-staging.dev/158179))
* [text-solution 70167](https://serlo-staging.dev/70167) ([current revision 158228](https://serlo-staging.dev/158228))
* [course-page 155806](https://serlo-staging.dev/155806) ([current revision 158376](https://serlo-staging.dev/158376))
* [text-solution 25961](https://serlo-staging.dev/25961) ([current revision 158442](https://serlo-staging.dev/158442))
* [article 154244](https://serlo-staging.dev/154244) ([current revision 159223](https://serlo-staging.dev/159223))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [article 152450](https://serlo-staging.dev/152450) ([current revision 159451](https://serlo-staging.dev/159451))
* [article 158031](https://serlo-staging.dev/158031) ([current revision 160200](https://serlo-staging.dev/160200))
* [text-solution 3083](https://serlo-staging.dev/3083) ([current revision 160430](https://serlo-staging.dev/160430))
* [text-solution 3263](https://serlo-staging.dev/3263) ([current revision 160780](https://serlo-staging.dev/160780))
* [article 153419](https://serlo-staging.dev/153419) ([current revision 160793](https://serlo-staging.dev/160793))
* [article 127338](https://serlo-staging.dev/127338) ([current revision 160838](https://serlo-staging.dev/160838))
* [article 154653](https://serlo-staging.dev/154653) ([current revision 160851](https://serlo-staging.dev/160851))
* [course-page 161361](https://serlo-staging.dev/161361) ([current revision 161373](https://serlo-staging.dev/161373))
* [text-exercise-group 5259](https://serlo-staging.dev/5259) ([current revision 161446](https://serlo-staging.dev/161446))
* [course-page 162051](https://serlo-staging.dev/162051) ([current revision 162071](https://serlo-staging.dev/162071))
* [course-page 162048](https://serlo-staging.dev/162048) ([current revision 162176](https://serlo-staging.dev/162176))
* [article 25812](https://serlo-staging.dev/25812) ([current revision 163795](https://serlo-staging.dev/163795))
* [article 54534](https://serlo-staging.dev/54534) ([current revision 164224](https://serlo-staging.dev/164224))
* [article 145927](https://serlo-staging.dev/145927) ([current revision 164607](https://serlo-staging.dev/164607))
* [article 165223](https://serlo-staging.dev/165223) ([current revision 165746](https://serlo-staging.dev/165746))
* [course-page 139538](https://serlo-staging.dev/139538) ([current revision 165776](https://serlo-staging.dev/165776))
* [article 163681](https://serlo-staging.dev/163681) ([current revision 166131](https://serlo-staging.dev/166131))
* [course-page 44099](https://serlo-staging.dev/44099) ([current revision 167115](https://serlo-staging.dev/167115))
* [article 167149](https://serlo-staging.dev/167149) ([current revision 167627](https://serlo-staging.dev/167627))
* [article 167635](https://serlo-staging.dev/167635) ([current revision 167790](https://serlo-staging.dev/167790))
* [course-page 51853](https://serlo-staging.dev/51853) ([current revision 168104](https://serlo-staging.dev/168104))
* [course-page 167929](https://serlo-staging.dev/167929) ([current revision 168111](https://serlo-staging.dev/168111))
* [course-page 168455](https://serlo-staging.dev/168455) ([current revision 168532](https://serlo-staging.dev/168532))
* [article 168274](https://serlo-staging.dev/168274) ([current revision 168855](https://serlo-staging.dev/168855))
* [article 168325](https://serlo-staging.dev/168325) ([current revision 169110](https://serlo-staging.dev/169110))
* [event 145464](https://serlo-staging.dev/145464) ([current revision 169335](https://serlo-staging.dev/169335))
* [event 145458](https://serlo-staging.dev/145458) ([current revision 169336](https://serlo-staging.dev/169336))
* [event 145459](https://serlo-staging.dev/145459) ([current revision 169337](https://serlo-staging.dev/169337))
* [event 147733](https://serlo-staging.dev/147733) ([current revision 169339](https://serlo-staging.dev/169339))
* [text-solution 94860](https://serlo-staging.dev/94860) ([current revision 169393](https://serlo-staging.dev/169393))
* [text-solution 158666](https://serlo-staging.dev/158666) ([current revision 169628](https://serlo-staging.dev/169628))
* [article 168166](https://serlo-staging.dev/168166) ([current revision 169647](https://serlo-staging.dev/169647))
* [course-page 169721](https://serlo-staging.dev/169721) ([current revision 169734](https://serlo-staging.dev/169734))
* [course-page 169722](https://serlo-staging.dev/169722) ([current revision 169735](https://serlo-staging.dev/169735))
* [course-page 169724](https://serlo-staging.dev/169724) ([current revision 169737](https://serlo-staging.dev/169737))
* [course-page 169725](https://serlo-staging.dev/169725) ([current revision 169738](https://serlo-staging.dev/169738))
* [course-page 169727](https://serlo-staging.dev/169727) ([current revision 169814](https://serlo-staging.dev/169814))
* [article 168248](https://serlo-staging.dev/168248) ([current revision 170414](https://serlo-staging.dev/170414))
* [article 168282](https://serlo-staging.dev/168282) ([current revision 170417](https://serlo-staging.dev/170417))
* [article 167755](https://serlo-staging.dev/167755) ([current revision 170467](https://serlo-staging.dev/170467))
* [article 167769](https://serlo-staging.dev/167769) ([current revision 170533](https://serlo-staging.dev/170533))
* [article 168363](https://serlo-staging.dev/168363) ([current revision 170623](https://serlo-staging.dev/170623))
* [article 168400](https://serlo-staging.dev/168400) ([current revision 170632](https://serlo-staging.dev/170632))
* [article 170734](https://serlo-staging.dev/170734) ([current revision 170805](https://serlo-staging.dev/170805))
* [article 167910](https://serlo-staging.dev/167910) ([current revision 170815](https://serlo-staging.dev/170815))
* [article 166154](https://serlo-staging.dev/166154) ([current revision 171150](https://serlo-staging.dev/171150))
* [article 25507](https://serlo-staging.dev/25507) ([current revision 171440](https://serlo-staging.dev/171440))
* [article 54317](https://serlo-staging.dev/54317) ([current revision 171699](https://serlo-staging.dev/171699))
* [text-solution 86758](https://serlo-staging.dev/86758) ([current revision 172534](https://serlo-staging.dev/172534))
* [text-solution 86761](https://serlo-staging.dev/86761) ([current revision 172536](https://serlo-staging.dev/172536))
* [text-solution 86763](https://serlo-staging.dev/86763) ([current revision 172538](https://serlo-staging.dev/172538))
* [article 169299](https://serlo-staging.dev/169299) ([current revision 173411](https://serlo-staging.dev/173411))
* [article 165740](https://serlo-staging.dev/165740) ([current revision 173425](https://serlo-staging.dev/173425))
* [article 168538](https://serlo-staging.dev/168538) ([current revision 173433](https://serlo-staging.dev/173433))
* [grouped-text-exercise 173786](https://serlo-staging.dev/173786) ([current revision 173796](https://serlo-staging.dev/173796))
* [article 32200](https://serlo-staging.dev/32200) ([current revision 173878](https://serlo-staging.dev/173878))
* [article 168256](https://serlo-staging.dev/168256) ([current revision 175193](https://serlo-staging.dev/175193))
* [text-solution 52244](https://serlo-staging.dev/52244) ([current revision 176295](https://serlo-staging.dev/176295))
* [course-page 176725](https://serlo-staging.dev/176725) ([current revision 176735](https://serlo-staging.dev/176735))
* [course-page 176727](https://serlo-staging.dev/176727) ([current revision 176737](https://serlo-staging.dev/176737))
* [course-page 176729](https://serlo-staging.dev/176729) ([current revision 176739](https://serlo-staging.dev/176739))
* [course-page 177860](https://serlo-staging.dev/177860) ([current revision 177879](https://serlo-staging.dev/177879))
* [course-page 177858](https://serlo-staging.dev/177858) ([current revision 177881](https://serlo-staging.dev/177881))
* [article 178239](https://serlo-staging.dev/178239) ([current revision 178299](https://serlo-staging.dev/178299))
* [article 178269](https://serlo-staging.dev/178269) ([current revision 178722](https://serlo-staging.dev/178722))
* [article 178847](https://serlo-staging.dev/178847) ([current revision 178903](https://serlo-staging.dev/178903))
* [course-page 179170](https://serlo-staging.dev/179170) ([current revision 179196](https://serlo-staging.dev/179196))
* [course-page 178850](https://serlo-staging.dev/178850) ([current revision 179246](https://serlo-staging.dev/179246))
* [course-page 178866](https://serlo-staging.dev/178866) ([current revision 179251](https://serlo-staging.dev/179251))
* [event 179657](https://serlo-staging.dev/179657) ([current revision 179698](https://serlo-staging.dev/179698))
* [course-page 179753](https://serlo-staging.dev/179753) ([current revision 179756](https://serlo-staging.dev/179756))
* [course-page 179759](https://serlo-staging.dev/179759) ([current revision 179802](https://serlo-staging.dev/179802))
* [article 167497](https://serlo-staging.dev/167497) ([current revision 179907](https://serlo-staging.dev/179907))
* [article 178254](https://serlo-staging.dev/178254) ([current revision 180002](https://serlo-staging.dev/180002))
* [event 145590](https://serlo-staging.dev/145590) ([current revision 180193](https://serlo-staging.dev/180193))
* [article 178258](https://serlo-staging.dev/178258) ([current revision 180707](https://serlo-staging.dev/180707))
* [article 178261](https://serlo-staging.dev/178261) ([current revision 180773](https://serlo-staging.dev/180773))
* [article 179744](https://serlo-staging.dev/179744) ([current revision 180839](https://serlo-staging.dev/180839))
* [article 179771](https://serlo-staging.dev/179771) ([current revision 180845](https://serlo-staging.dev/180845))
* [article 179780](https://serlo-staging.dev/179780) ([current revision 180850](https://serlo-staging.dev/180850))
* [course-page 179762](https://serlo-staging.dev/179762) ([current revision 180852](https://serlo-staging.dev/180852))
* [article 179789](https://serlo-staging.dev/179789) ([current revision 180853](https://serlo-staging.dev/180853))
* [course-page 19495](https://serlo-staging.dev/19495) ([current revision 181887](https://serlo-staging.dev/181887))
* [article 27823](https://serlo-staging.dev/27823) ([current revision 181889](https://serlo-staging.dev/181889))
* [article 1855](https://serlo-staging.dev/1855) ([current revision 183896](https://serlo-staging.dev/183896))
* [event 169327](https://serlo-staging.dev/169327) ([current revision 183938](https://serlo-staging.dev/183938))
* [course-page 167674](https://serlo-staging.dev/167674) ([current revision 184084](https://serlo-staging.dev/184084))
* [course-page 168343](https://serlo-staging.dev/168343) ([current revision 184086](https://serlo-staging.dev/184086))
* [course-page 168491](https://serlo-staging.dev/168491) ([current revision 184090](https://serlo-staging.dev/184090))
* [article 170363](https://serlo-staging.dev/170363) ([current revision 184688](https://serlo-staging.dev/184688))
* [article 98333](https://serlo-staging.dev/98333) ([current revision 184859](https://serlo-staging.dev/184859))
* [article 184028](https://serlo-staging.dev/184028) ([current revision 185563](https://serlo-staging.dev/185563))
* [article 181279](https://serlo-staging.dev/181279) ([current revision 185617](https://serlo-staging.dev/185617))
* [course-page 186587](https://serlo-staging.dev/186587) ([current revision 186596](https://serlo-staging.dev/186596))
* [course-page 186589](https://serlo-staging.dev/186589) ([current revision 186598](https://serlo-staging.dev/186598))
* [course-page 186591](https://serlo-staging.dev/186591) ([current revision 186600](https://serlo-staging.dev/186600))
* [article 186610](https://serlo-staging.dev/186610) ([current revision 186623](https://serlo-staging.dev/186623))
* [article 90785](https://serlo-staging.dev/90785) ([current revision 186661](https://serlo-staging.dev/186661))
* [article 107863](https://serlo-staging.dev/107863) ([current revision 186662](https://serlo-staging.dev/186662))
* [course-page 187203](https://serlo-staging.dev/187203) ([current revision 187216](https://serlo-staging.dev/187216))
* [course-page 188326](https://serlo-staging.dev/188326) ([current revision 188385](https://serlo-staging.dev/188385))
* [article 187487](https://serlo-staging.dev/187487) ([current revision 190508](https://serlo-staging.dev/190508))
* [article 181056](https://serlo-staging.dev/181056) ([current revision 191176](https://serlo-staging.dev/191176))
* [text-solution 190460](https://serlo-staging.dev/190460) ([current revision 191195](https://serlo-staging.dev/191195))
* [text-solution 190462](https://serlo-staging.dev/190462) ([current revision 191197](https://serlo-staging.dev/191197))
* [article 155311](https://serlo-staging.dev/155311) ([current revision 191244](https://serlo-staging.dev/191244))
* [article 185678](https://serlo-staging.dev/185678) ([current revision 191534](https://serlo-staging.dev/191534))
* [course-page 156422](https://serlo-staging.dev/156422) ([current revision 191596](https://serlo-staging.dev/191596))
* [text-solution 190464](https://serlo-staging.dev/190464) ([current revision 191849](https://serlo-staging.dev/191849))
* [article 1859](https://serlo-staging.dev/1859) ([current revision 192445](https://serlo-staging.dev/192445))
* [article 1803](https://serlo-staging.dev/1803) ([current revision 192807](https://serlo-staging.dev/192807))
* [article 1937](https://serlo-staging.dev/1937) ([current revision 192821](https://serlo-staging.dev/192821))
* [course-page 36766](https://serlo-staging.dev/36766) ([current revision 193043](https://serlo-staging.dev/193043))
* [course-page 36833](https://serlo-staging.dev/36833) ([current revision 193050](https://serlo-staging.dev/193050))
* [course-page 36825](https://serlo-staging.dev/36825) ([current revision 193053](https://serlo-staging.dev/193053))
* [article 62547](https://serlo-staging.dev/62547) ([current revision 193082](https://serlo-staging.dev/193082))
* [course-page 62226](https://serlo-staging.dev/62226) ([current revision 193084](https://serlo-staging.dev/193084))
* [course-page 99544](https://serlo-staging.dev/99544) ([current revision 193280](https://serlo-staging.dev/193280))
* [article 90405](https://serlo-staging.dev/90405) ([current revision 193490](https://serlo-staging.dev/193490))
* [article 51369](https://serlo-staging.dev/51369) ([current revision 193749](https://serlo-staging.dev/193749))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [course-page 193816](https://serlo-staging.dev/193816) ([current revision 193841](https://serlo-staging.dev/193841))
* [course-page 193817](https://serlo-staging.dev/193817) ([current revision 193842](https://serlo-staging.dev/193842))
* [article 181058](https://serlo-staging.dev/181058) ([current revision 194425](https://serlo-staging.dev/194425))
* [article 77792](https://serlo-staging.dev/77792) ([current revision 194907](https://serlo-staging.dev/194907))
* [article 194758](https://serlo-staging.dev/194758) ([current revision 194970](https://serlo-staging.dev/194970))
* [article 1625](https://serlo-staging.dev/1625) ([current revision 194972](https://serlo-staging.dev/194972))
* [article 195043](https://serlo-staging.dev/195043) ([current revision 195045](https://serlo-staging.dev/195045))
* [course-page 178835](https://serlo-staging.dev/178835) ([current revision 195226](https://serlo-staging.dev/195226))
* [course-page 195294](https://serlo-staging.dev/195294) ([current revision 195308](https://serlo-staging.dev/195308))
* [course-page 195306](https://serlo-staging.dev/195306) ([current revision 195311](https://serlo-staging.dev/195311))
* [course-page 195318](https://serlo-staging.dev/195318) ([current revision 195320](https://serlo-staging.dev/195320))
* [article 195041](https://serlo-staging.dev/195041) ([current revision 195469](https://serlo-staging.dev/195469))
* [course-page 196540](https://serlo-staging.dev/196540) ([current revision 196552](https://serlo-staging.dev/196552))
* [course-page 196155](https://serlo-staging.dev/196155) ([current revision 196592](https://serlo-staging.dev/196592))
* [course-page 196146](https://serlo-staging.dev/196146) ([current revision 196593](https://serlo-staging.dev/196593))
* [course-page 196542](https://serlo-staging.dev/196542) ([current revision 196600](https://serlo-staging.dev/196600))
* [course-page 196546](https://serlo-staging.dev/196546) ([current revision 196604](https://serlo-staging.dev/196604))
* [course-page 196567](https://serlo-staging.dev/196567) ([current revision 196607](https://serlo-staging.dev/196607))
* [course-page 196057](https://serlo-staging.dev/196057) ([current revision 196611](https://serlo-staging.dev/196611))
* [course-page 196543](https://serlo-staging.dev/196543) ([current revision 196656](https://serlo-staging.dev/196656))
* [course-page 196566](https://serlo-staging.dev/196566) ([current revision 196771](https://serlo-staging.dev/196771))
* [article 56795](https://serlo-staging.dev/56795) ([current revision 196827](https://serlo-staging.dev/196827))
* [course-page 52448](https://serlo-staging.dev/52448) ([current revision 196828](https://serlo-staging.dev/196828))
* [article 152028](https://serlo-staging.dev/152028) ([current revision 196948](https://serlo-staging.dev/196948))
* [course-page 195321](https://serlo-staging.dev/195321) ([current revision 196957](https://serlo-staging.dev/196957))
* [article 170218](https://serlo-staging.dev/170218) ([current revision 196962](https://serlo-staging.dev/196962))
* [article 1775](https://serlo-staging.dev/1775) ([current revision 197265](https://serlo-staging.dev/197265))
* [article 42091](https://serlo-staging.dev/42091) ([current revision 197307](https://serlo-staging.dev/197307))
* [article 43283](https://serlo-staging.dev/43283) ([current revision 197308](https://serlo-staging.dev/197308))
* [article 158205](https://serlo-staging.dev/158205) ([current revision 197316](https://serlo-staging.dev/197316))
* [article 53073](https://serlo-staging.dev/53073) ([current revision 197340](https://serlo-staging.dev/197340))
* [article 90410](https://serlo-staging.dev/90410) ([current revision 197881](https://serlo-staging.dev/197881))
* [text-solution 195378](https://serlo-staging.dev/195378) ([current revision 198061](https://serlo-staging.dev/198061))
* [article 196641](https://serlo-staging.dev/196641) ([current revision 198856](https://serlo-staging.dev/198856))
* [article 160297](https://serlo-staging.dev/160297) ([current revision 198881](https://serlo-staging.dev/198881))
* [article 1969](https://serlo-staging.dev/1969) ([current revision 198970](https://serlo-staging.dev/198970))
* [article 199872](https://serlo-staging.dev/199872) ([current revision 199879](https://serlo-staging.dev/199879))
* [course-page 199316](https://serlo-staging.dev/199316) ([current revision 200106](https://serlo-staging.dev/200106))
* [text-solution 60185](https://serlo-staging.dev/60185) ([current revision 201658](https://serlo-staging.dev/201658))
* [text-exercise 60195](https://serlo-staging.dev/60195) ([current revision 201674](https://serlo-staging.dev/201674))
* [text-solution 72972](https://serlo-staging.dev/72972) ([current revision 201708](https://serlo-staging.dev/201708))
* [article 186051](https://serlo-staging.dev/186051) ([current revision 202081](https://serlo-staging.dev/202081))
* [article 178250](https://serlo-staging.dev/178250) ([current revision 202862](https://serlo-staging.dev/202862))
* [course-page 72260](https://serlo-staging.dev/72260) ([current revision 202896](https://serlo-staging.dev/202896))
* [article 185660](https://serlo-staging.dev/185660) ([current revision 202910](https://serlo-staging.dev/202910))
* [article 203329](https://serlo-staging.dev/203329) ([current revision 203337](https://serlo-staging.dev/203337))
* [article 203664](https://serlo-staging.dev/203664) ([current revision 203669](https://serlo-staging.dev/203669))
* [article 150885](https://serlo-staging.dev/150885) ([current revision 204163](https://serlo-staging.dev/204163))
* [course-page 199305](https://serlo-staging.dev/199305) ([current revision 204288](https://serlo-staging.dev/204288))
* [course-page 206245](https://serlo-staging.dev/206245) ([current revision 206253](https://serlo-staging.dev/206253))
* [course-page 206248](https://serlo-staging.dev/206248) ([current revision 206256](https://serlo-staging.dev/206256))
* [course-page 206368](https://serlo-staging.dev/206368) ([current revision 206376](https://serlo-staging.dev/206376))
* [course-page 206369](https://serlo-staging.dev/206369) ([current revision 206377](https://serlo-staging.dev/206377))
* [course-page 203725](https://serlo-staging.dev/203725) ([current revision 207981](https://serlo-staging.dev/207981))
* [course-page 203731](https://serlo-staging.dev/203731) ([current revision 207994](https://serlo-staging.dev/207994))
* [course-page 203727](https://serlo-staging.dev/203727) ([current revision 207996](https://serlo-staging.dev/207996))
* [course-page 161358](https://serlo-staging.dev/161358) ([current revision 208316](https://serlo-staging.dev/208316))
* [course-page 161364](https://serlo-staging.dev/161364) ([current revision 208318](https://serlo-staging.dev/208318))
* [article 2005](https://serlo-staging.dev/2005) ([current revision 208323](https://serlo-staging.dev/208323))
* [article 150648](https://serlo-staging.dev/150648) ([current revision 209138](https://serlo-staging.dev/209138))
* [course-page 210270](https://serlo-staging.dev/210270) ([current revision 210280](https://serlo-staging.dev/210280))
* [course-page 210284](https://serlo-staging.dev/210284) ([current revision 210285](https://serlo-staging.dev/210285))
* [article 1679](https://serlo-staging.dev/1679) ([current revision 210897](https://serlo-staging.dev/210897))
* [article 211023](https://serlo-staging.dev/211023) ([current revision 211039](https://serlo-staging.dev/211039))
* [article 181051](https://serlo-staging.dev/181051) ([current revision 212149](https://serlo-staging.dev/212149))
* [article 149480](https://serlo-staging.dev/149480) ([current revision 212417](https://serlo-staging.dev/212417))
* [article 212604](https://serlo-staging.dev/212604) ([current revision 212636](https://serlo-staging.dev/212636))
* [article 210777](https://serlo-staging.dev/210777) ([current revision 213803](https://serlo-staging.dev/213803))
* [article 214104](https://serlo-staging.dev/214104) ([current revision 214125](https://serlo-staging.dev/214125))
* [course-page 212169](https://serlo-staging.dev/212169) ([current revision 214566](https://serlo-staging.dev/214566))
* [article 25369](https://serlo-staging.dev/25369) ([current revision 214983](https://serlo-staging.dev/214983))
* [article 215596](https://serlo-staging.dev/215596) ([current revision 215681](https://serlo-staging.dev/215681))
* [article 1769](https://serlo-staging.dev/1769) ([current revision 215689](https://serlo-staging.dev/215689))
* [text-solution 215752](https://serlo-staging.dev/215752) ([current revision 215754](https://serlo-staging.dev/215754))
* [event 147713](https://serlo-staging.dev/147713) ([current revision 215780](https://serlo-staging.dev/215780))
* [course-page 215763](https://serlo-staging.dev/215763) ([current revision 215806](https://serlo-staging.dev/215806))
* [course-page 215805](https://serlo-staging.dev/215805) ([current revision 215848](https://serlo-staging.dev/215848))
* [course-page 215847](https://serlo-staging.dev/215847) ([current revision 215852](https://serlo-staging.dev/215852))
* [course-page 215850](https://serlo-staging.dev/215850) ([current revision 215853](https://serlo-staging.dev/215853))
* [article 215910](https://serlo-staging.dev/215910) ([current revision 215924](https://serlo-staging.dev/215924))
* [article 215941](https://serlo-staging.dev/215941) ([current revision 215943](https://serlo-staging.dev/215943))
* [article 216917](https://serlo-staging.dev/216917) ([current revision 216921](https://serlo-staging.dev/216921))
* [course-page 209964](https://serlo-staging.dev/209964) ([current revision 217517](https://serlo-staging.dev/217517))
* [course-page 217772](https://serlo-staging.dev/217772) ([current revision 217799](https://serlo-staging.dev/217799))
* [course-page 217794](https://serlo-staging.dev/217794) ([current revision 218498](https://serlo-staging.dev/218498))
* [course-page 217795](https://serlo-staging.dev/217795) ([current revision 218518](https://serlo-staging.dev/218518))
* [course-page 217797](https://serlo-staging.dev/217797) ([current revision 219010](https://serlo-staging.dev/219010))
* [course-page 217796](https://serlo-staging.dev/217796) ([current revision 219011](https://serlo-staging.dev/219011))
* [article 219101](https://serlo-staging.dev/219101) ([current revision 220157](https://serlo-staging.dev/220157))
* [article 220154](https://serlo-staging.dev/220154) ([current revision 220213](https://serlo-staging.dev/220213))
* [article 205995](https://serlo-staging.dev/205995) ([current revision 220253](https://serlo-staging.dev/220253))
* [article 220146](https://serlo-staging.dev/220146) ([current revision 220571](https://serlo-staging.dev/220571))
* [article 2117](https://serlo-staging.dev/2117) ([current revision 220683](https://serlo-staging.dev/220683))
* [article 86467](https://serlo-staging.dev/86467) ([current revision 220723](https://serlo-staging.dev/220723))
* [article 148759](https://serlo-staging.dev/148759) ([current revision 220724](https://serlo-staging.dev/220724))
* [article 146609](https://serlo-staging.dev/146609) ([current revision 220725](https://serlo-staging.dev/220725))
* [article 146611](https://serlo-staging.dev/146611) ([current revision 220829](https://serlo-staging.dev/220829))
* [article 199237](https://serlo-staging.dev/199237) ([current revision 220831](https://serlo-staging.dev/220831))
* [article 205303](https://serlo-staging.dev/205303) ([current revision 220836](https://serlo-staging.dev/220836))
* [article 146610](https://serlo-staging.dev/146610) ([current revision 220955](https://serlo-staging.dev/220955))
* [article 204811](https://serlo-staging.dev/204811) ([current revision 221529](https://serlo-staging.dev/221529))
* [article 1821](https://serlo-staging.dev/1821) ([current revision 221534](https://serlo-staging.dev/221534))
* [article 179161](https://serlo-staging.dev/179161) ([current revision 221546](https://serlo-staging.dev/221546))
* [course-page 133596](https://serlo-staging.dev/133596) ([current revision 222011](https://serlo-staging.dev/222011))
* [article 51490](https://serlo-staging.dev/51490) ([current revision 222668](https://serlo-staging.dev/222668))
* [article 1835](https://serlo-staging.dev/1835) ([current revision 222710](https://serlo-staging.dev/222710))
* [article 222470](https://serlo-staging.dev/222470) ([current revision 222734](https://serlo-staging.dev/222734))
* [article 222735](https://serlo-staging.dev/222735) ([current revision 222743](https://serlo-staging.dev/222743))
* [article 181038](https://serlo-staging.dev/181038) ([current revision 222779](https://serlo-staging.dev/222779))
* [article 90412](https://serlo-staging.dev/90412) ([current revision 223073](https://serlo-staging.dev/223073))
* [course-page 221397](https://serlo-staging.dev/221397) ([current revision 223347](https://serlo-staging.dev/223347))
* [article 223492](https://serlo-staging.dev/223492) ([current revision 223499](https://serlo-staging.dev/223499))
* [article 114026](https://serlo-staging.dev/114026) ([current revision 223647](https://serlo-staging.dev/223647))
* [course-page 221142](https://serlo-staging.dev/221142) ([current revision 224222](https://serlo-staging.dev/224222))
* [course-page 221141](https://serlo-staging.dev/221141) ([current revision 224223](https://serlo-staging.dev/224223))
* [course-page 221145](https://serlo-staging.dev/221145) ([current revision 224224](https://serlo-staging.dev/224224))
* [course-page 221394](https://serlo-staging.dev/221394) ([current revision 224225](https://serlo-staging.dev/224225))
* [article 77998](https://serlo-staging.dev/77998) ([current revision 224237](https://serlo-staging.dev/224237))
* [article 134964](https://serlo-staging.dev/134964) ([current revision 224251](https://serlo-staging.dev/224251))
* [course-page 221400](https://serlo-staging.dev/221400) ([current revision 224261](https://serlo-staging.dev/224261))
* [article 1657](https://serlo-staging.dev/1657) ([current revision 224452](https://serlo-staging.dev/224452))
* [text-solution 223711](https://serlo-staging.dev/223711) ([current revision 224461](https://serlo-staging.dev/224461))
* [article 1527](https://serlo-staging.dev/1527) ([current revision 224591](https://serlo-staging.dev/224591))
* [course-page 222210](https://serlo-staging.dev/222210) ([current revision 224603](https://serlo-staging.dev/224603))
* [article 224615](https://serlo-staging.dev/224615) ([current revision 224617](https://serlo-staging.dev/224617))
* [course-page 222216](https://serlo-staging.dev/222216) ([current revision 224620](https://serlo-staging.dev/224620))
* [course-page 222214](https://serlo-staging.dev/222214) ([current revision 224637](https://serlo-staging.dev/224637))
* [course-page 222226](https://serlo-staging.dev/222226) ([current revision 224641](https://serlo-staging.dev/224641))
* [article 225024](https://serlo-staging.dev/225024) ([current revision 225051](https://serlo-staging.dev/225051))
* [article 216333](https://serlo-staging.dev/216333) ([current revision 225082](https://serlo-staging.dev/225082))
* [course-page 222218](https://serlo-staging.dev/222218) ([current revision 225090](https://serlo-staging.dev/225090))
* [article 224015](https://serlo-staging.dev/224015) ([current revision 225368](https://serlo-staging.dev/225368))
* [course-page 225447](https://serlo-staging.dev/225447) ([current revision 225460](https://serlo-staging.dev/225460))
* [article 1587](https://serlo-staging.dev/1587) ([current revision 225962](https://serlo-staging.dev/225962))
* [article 1603](https://serlo-staging.dev/1603) ([current revision 226180](https://serlo-staging.dev/226180))
* [course-page 222212](https://serlo-staging.dev/222212) ([current revision 226196](https://serlo-staging.dev/226196))
* [article 169303](https://serlo-staging.dev/169303) ([current revision 226802](https://serlo-staging.dev/226802))
* [article 167340](https://serlo-staging.dev/167340) ([current revision 226807](https://serlo-staging.dev/226807))
* [article 226897](https://serlo-staging.dev/226897) ([current revision 226914](https://serlo-staging.dev/226914))
* [course-page 134381](https://serlo-staging.dev/134381) ([current revision 227030](https://serlo-staging.dev/227030))
* [course-page 134384](https://serlo-staging.dev/134384) ([current revision 227036](https://serlo-staging.dev/227036))
* [course-page 134568](https://serlo-staging.dev/134568) ([current revision 227037](https://serlo-staging.dev/227037))
* [course-page 134561](https://serlo-staging.dev/134561) ([current revision 227039](https://serlo-staging.dev/227039))
* [course-page 72186](https://serlo-staging.dev/72186) ([current revision 227411](https://serlo-staging.dev/227411))
* [course-page 227433](https://serlo-staging.dev/227433) ([current revision 227442](https://serlo-staging.dev/227442))
* [course-page 227452](https://serlo-staging.dev/227452) ([current revision 227460](https://serlo-staging.dev/227460))
* [course-page 227453](https://serlo-staging.dev/227453) ([current revision 227461](https://serlo-staging.dev/227461))
* [article 143547](https://serlo-staging.dev/143547) ([current revision 227509](https://serlo-staging.dev/227509))
* [article 224867](https://serlo-staging.dev/224867) ([current revision 227634](https://serlo-staging.dev/227634))
* [article 209314](https://serlo-staging.dev/209314) ([current revision 227640](https://serlo-staging.dev/227640))
* [course-page 20326](https://serlo-staging.dev/20326) ([current revision 227835](https://serlo-staging.dev/227835))
* [course-page 21725](https://serlo-staging.dev/21725) ([current revision 227836](https://serlo-staging.dev/227836))
* [course-page 21729](https://serlo-staging.dev/21729) ([current revision 227837](https://serlo-staging.dev/227837))
* [course-page 20329](https://serlo-staging.dev/20329) ([current revision 227839](https://serlo-staging.dev/227839))
* [course-page 22074](https://serlo-staging.dev/22074) ([current revision 227840](https://serlo-staging.dev/227840))
* [article 226235](https://serlo-staging.dev/226235) ([current revision 227899](https://serlo-staging.dev/227899))
* [article 225324](https://serlo-staging.dev/225324) ([current revision 228161](https://serlo-staging.dev/228161))
* [article 226236](https://serlo-staging.dev/226236) ([current revision 228253](https://serlo-staging.dev/228253))
* [article 144838](https://serlo-staging.dev/144838) ([current revision 228465](https://serlo-staging.dev/228465))
* [article 210900](https://serlo-staging.dev/210900) ([current revision 228614](https://serlo-staging.dev/228614))
* [article 134821](https://serlo-staging.dev/134821) ([current revision 228618](https://serlo-staging.dev/228618))
* [article 1649](https://serlo-staging.dev/1649) ([current revision 228662](https://serlo-staging.dev/228662))
* [article 227544](https://serlo-staging.dev/227544) ([current revision 228792](https://serlo-staging.dev/228792))
* [event 142218](https://serlo-staging.dev/142218) ([current revision 228807](https://serlo-staging.dev/228807))
* [article 1647](https://serlo-staging.dev/1647) ([current revision 228830](https://serlo-staging.dev/228830))
* [article 1659](https://serlo-staging.dev/1659) ([current revision 229056](https://serlo-staging.dev/229056))
* [article 225632](https://serlo-staging.dev/225632) ([current revision 229278](https://serlo-staging.dev/229278))
* [article 2067](https://serlo-staging.dev/2067) ([current revision 229326](https://serlo-staging.dev/229326))
* [article 1675](https://serlo-staging.dev/1675) ([current revision 229560](https://serlo-staging.dev/229560))
* [article 229428](https://serlo-staging.dev/229428) ([current revision 229775](https://serlo-staging.dev/229775))
* [article 227518](https://serlo-staging.dev/227518) ([current revision 229919](https://serlo-staging.dev/229919))
* [course-page 145632](https://serlo-staging.dev/145632) ([current revision 230114](https://serlo-staging.dev/230114))
* [article 229982](https://serlo-staging.dev/229982) ([current revision 231011](https://serlo-staging.dev/231011))
* [article 1623](https://serlo-staging.dev/1623) ([current revision 231212](https://serlo-staging.dev/231212))
* [article 231195](https://serlo-staging.dev/231195) ([current revision 231224](https://serlo-staging.dev/231224))
* [text-solution 231324](https://serlo-staging.dev/231324) ([current revision 231326](https://serlo-staging.dev/231326))
* [article 227023](https://serlo-staging.dev/227023) ([current revision 231367](https://serlo-staging.dev/231367))
* [article 200617](https://serlo-staging.dev/200617) ([current revision 231388](https://serlo-staging.dev/231388))
* [article 1701](https://serlo-staging.dev/1701) ([current revision 231468](https://serlo-staging.dev/231468))
* [article 231763](https://serlo-staging.dev/231763) ([current revision 231787](https://serlo-staging.dev/231787))
* [course-page 232013](https://serlo-staging.dev/232013) ([current revision 232018](https://serlo-staging.dev/232018))
* [article 201287](https://serlo-staging.dev/201287) ([current revision 232108](https://serlo-staging.dev/232108))
* [article 168870](https://serlo-staging.dev/168870) ([current revision 232716](https://serlo-staging.dev/232716))
* [course-page 217723](https://serlo-staging.dev/217723) ([current revision 232900](https://serlo-staging.dev/232900))
* [article 213046](https://serlo-staging.dev/213046) ([current revision 233779](https://serlo-staging.dev/233779))
* [article 213105](https://serlo-staging.dev/213105) ([current revision 233781](https://serlo-staging.dev/233781))
* [article 213107](https://serlo-staging.dev/213107) ([current revision 233782](https://serlo-staging.dev/233782))
* [article 228138](https://serlo-staging.dev/228138) ([current revision 234112](https://serlo-staging.dev/234112))
* [article 230333](https://serlo-staging.dev/230333) ([current revision 234162](https://serlo-staging.dev/234162))
* [article 230337](https://serlo-staging.dev/230337) ([current revision 234167](https://serlo-staging.dev/234167))
* [course-page 233718](https://serlo-staging.dev/233718) ([current revision 234282](https://serlo-staging.dev/234282))
* [article 230345](https://serlo-staging.dev/230345) ([current revision 234622](https://serlo-staging.dev/234622))
* [article 234551](https://serlo-staging.dev/234551) ([current revision 234679](https://serlo-staging.dev/234679))
* [article 54352](https://serlo-staging.dev/54352) ([current revision 235196](https://serlo-staging.dev/235196))
* [article 234395](https://serlo-staging.dev/234395) ([current revision 235404](https://serlo-staging.dev/235404))
* [course-page 235390](https://serlo-staging.dev/235390) ([current revision 235525](https://serlo-staging.dev/235525))
* [course-page 235391](https://serlo-staging.dev/235391) ([current revision 235526](https://serlo-staging.dev/235526))
* [course-page 233009](https://serlo-staging.dev/233009) ([current revision 235628](https://serlo-staging.dev/235628))
* [course-page 235633](https://serlo-staging.dev/235633) ([current revision 235649](https://serlo-staging.dev/235649))
* [course-page 235532](https://serlo-staging.dev/235532) ([current revision 235716](https://serlo-staging.dev/235716))
* [article 230452](https://serlo-staging.dev/230452) ([current revision 236772](https://serlo-staging.dev/236772))
* [article 230454](https://serlo-staging.dev/230454) ([current revision 236780](https://serlo-staging.dev/236780))
* [article 1873](https://serlo-staging.dev/1873) ([current revision 236834](https://serlo-staging.dev/236834))
* [text-solution 236838](https://serlo-staging.dev/236838) ([current revision 236853](https://serlo-staging.dev/236853))
* [course-page 236212](https://serlo-staging.dev/236212) ([current revision 236888](https://serlo-staging.dev/236888))
* [course-page 236210](https://serlo-staging.dev/236210) ([current revision 236909](https://serlo-staging.dev/236909))
* [article 140481](https://serlo-staging.dev/140481) ([current revision 236957](https://serlo-staging.dev/236957))
* [article 231373](https://serlo-staging.dev/231373) ([current revision 237017](https://serlo-staging.dev/237017))
* [article 236942](https://serlo-staging.dev/236942) ([current revision 237267](https://serlo-staging.dev/237267))
* [course-page 62402](https://serlo-staging.dev/62402) ([current revision 237373](https://serlo-staging.dev/237373))
* [course-page 237339](https://serlo-staging.dev/237339) ([current revision 237453](https://serlo-staging.dev/237453))
* [course-page 237340](https://serlo-staging.dev/237340) ([current revision 237454](https://serlo-staging.dev/237454))
* [course-page 237469](https://serlo-staging.dev/237469) ([current revision 237473](https://serlo-staging.dev/237473))
* [article 201259](https://serlo-staging.dev/201259) ([current revision 237767](https://serlo-staging.dev/237767))
* [article 201374](https://serlo-staging.dev/201374) ([current revision 237769](https://serlo-staging.dev/237769))
* [course-page 236211](https://serlo-staging.dev/236211) ([current revision 237857](https://serlo-staging.dev/237857))
* [article 237673](https://serlo-staging.dev/237673) ([current revision 237885](https://serlo-staging.dev/237885))
* [article 1719](https://serlo-staging.dev/1719) ([current revision 237889](https://serlo-staging.dev/237889))
* [course-page 237985](https://serlo-staging.dev/237985) ([current revision 237991](https://serlo-staging.dev/237991))
* [article 201470](https://serlo-staging.dev/201470) ([current revision 238016](https://serlo-staging.dev/238016))
* [course-page 237487](https://serlo-staging.dev/237487) ([current revision 238022](https://serlo-staging.dev/238022))
* [article 1739](https://serlo-staging.dev/1739) ([current revision 238023](https://serlo-staging.dev/238023))
* [article 202484](https://serlo-staging.dev/202484) ([current revision 238032](https://serlo-staging.dev/238032))
* [course-page 238092](https://serlo-staging.dev/238092) ([current revision 238132](https://serlo-staging.dev/238132))
* [course-page 238093](https://serlo-staging.dev/238093) ([current revision 238133](https://serlo-staging.dev/238133))
* [course-page 238094](https://serlo-staging.dev/238094) ([current revision 238134](https://serlo-staging.dev/238134))
* [course-page 237816](https://serlo-staging.dev/237816) ([current revision 238517](https://serlo-staging.dev/238517))
* [course-page 237813](https://serlo-staging.dev/237813) ([current revision 238604](https://serlo-staging.dev/238604))
* [article 171893](https://serlo-staging.dev/171893) ([current revision 238616](https://serlo-staging.dev/238616))
* [article 169197](https://serlo-staging.dev/169197) ([current revision 238620](https://serlo-staging.dev/238620))
* [article 196626](https://serlo-staging.dev/196626) ([current revision 238627](https://serlo-staging.dev/238627))
* [article 213826](https://serlo-staging.dev/213826) ([current revision 238629](https://serlo-staging.dev/238629))
* [text-solution 208436](https://serlo-staging.dev/208436) ([current revision 238748](https://serlo-staging.dev/238748))
* [article 202957](https://serlo-staging.dev/202957) ([current revision 238887](https://serlo-staging.dev/238887))
* [article 1743](https://serlo-staging.dev/1743) ([current revision 238888](https://serlo-staging.dev/238888))
* [article 203204](https://serlo-staging.dev/203204) ([current revision 238890](https://serlo-staging.dev/238890))
* [article 203940](https://serlo-staging.dev/203940) ([current revision 238892](https://serlo-staging.dev/238892))
* [course-page 237927](https://serlo-staging.dev/237927) ([current revision 239058](https://serlo-staging.dev/239058))
* [article 117722](https://serlo-staging.dev/117722) ([current revision 239294](https://serlo-staging.dev/239294))
* [article 43367](https://serlo-staging.dev/43367) ([current revision 239302](https://serlo-staging.dev/239302))
* [course-page 239588](https://serlo-staging.dev/239588) ([current revision 239595](https://serlo-staging.dev/239595))
* [course-page 239571](https://serlo-staging.dev/239571) ([current revision 239608](https://serlo-staging.dev/239608))
* [course-page 239572](https://serlo-staging.dev/239572) ([current revision 239609](https://serlo-staging.dev/239609))
* [course-page 58150](https://serlo-staging.dev/58150) ([current revision 239640](https://serlo-staging.dev/239640))
* [course-page 58152](https://serlo-staging.dev/58152) ([current revision 239657](https://serlo-staging.dev/239657))
* [course-page 90385](https://serlo-staging.dev/90385) ([current revision 239685](https://serlo-staging.dev/239685))
* [article 136597](https://serlo-staging.dev/136597) ([current revision 239687](https://serlo-staging.dev/239687))
* [course-page 239508](https://serlo-staging.dev/239508) ([current revision 239726](https://serlo-staging.dev/239726))
* [course-page 223934](https://serlo-staging.dev/223934) ([current revision 239731](https://serlo-staging.dev/239731))
* [article 227314](https://serlo-staging.dev/227314) ([current revision 239788](https://serlo-staging.dev/239788))
* [course-page 223941](https://serlo-staging.dev/223941) ([current revision 239793](https://serlo-staging.dev/239793))
* [course-page 223938](https://serlo-staging.dev/223938) ([current revision 239820](https://serlo-staging.dev/239820))
* [course-page 223937](https://serlo-staging.dev/223937) ([current revision 239821](https://serlo-staging.dev/239821))
* [article 161890](https://serlo-staging.dev/161890) ([current revision 239838](https://serlo-staging.dev/239838))
* [article 127473](https://serlo-staging.dev/127473) ([current revision 239840](https://serlo-staging.dev/239840))
* [article 127479](https://serlo-staging.dev/127479) ([current revision 239868](https://serlo-staging.dev/239868))
* [course-page 223940](https://serlo-staging.dev/223940) ([current revision 239880](https://serlo-staging.dev/239880))
* [article 220139](https://serlo-staging.dev/220139) ([current revision 239895](https://serlo-staging.dev/239895))
* [course-page 239795](https://serlo-staging.dev/239795) ([current revision 239952](https://serlo-staging.dev/239952))
* [article 43256](https://serlo-staging.dev/43256) ([current revision 240114](https://serlo-staging.dev/240114))
* [article 1927](https://serlo-staging.dev/1927) ([current revision 240190](https://serlo-staging.dev/240190))
* [article 240300](https://serlo-staging.dev/240300) ([current revision 240349](https://serlo-staging.dev/240349))
* [article 1801](https://serlo-staging.dev/1801) ([current revision 240409](https://serlo-staging.dev/240409))
* [article 111179](https://serlo-staging.dev/111179) ([current revision 240418](https://serlo-staging.dev/240418))
* [article 240351](https://serlo-staging.dev/240351) ([current revision 240444](https://serlo-staging.dev/240444))
* [article 224320](https://serlo-staging.dev/224320) ([current revision 240858](https://serlo-staging.dev/240858))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [article 2213](https://serlo-staging.dev/2213) ([current revision 240899](https://serlo-staging.dev/240899))
* [article 234735](https://serlo-staging.dev/234735) ([current revision 240958](https://serlo-staging.dev/240958))
* [article 1963](https://serlo-staging.dev/1963) ([current revision 241026](https://serlo-staging.dev/241026))
* [article 1789](https://serlo-staging.dev/1789) ([current revision 241072](https://serlo-staging.dev/241072))
* [article 1935](https://serlo-staging.dev/1935) ([current revision 241129](https://serlo-staging.dev/241129))
* [article 241186](https://serlo-staging.dev/241186) ([current revision 241191](https://serlo-staging.dev/241191))
* [article 2015](https://serlo-staging.dev/2015) ([current revision 241245](https://serlo-staging.dev/241245))
* [article 140933](https://serlo-staging.dev/140933) ([current revision 241355](https://serlo-staging.dev/241355))
* [article 159135](https://serlo-staging.dev/159135) ([current revision 241356](https://serlo-staging.dev/241356))
* [article 1571](https://serlo-staging.dev/1571) ([current revision 241555](https://serlo-staging.dev/241555))
* [article 1929](https://serlo-staging.dev/1929) ([current revision 241616](https://serlo-staging.dev/241616))
* [text-solution 7799](https://serlo-staging.dev/7799) ([current revision 241878](https://serlo-staging.dev/241878))
* [article 25509](https://serlo-staging.dev/25509) ([current revision 242007](https://serlo-staging.dev/242007))
* [article 242060](https://serlo-staging.dev/242060) ([current revision 242064](https://serlo-staging.dev/242064))
* [article 26163](https://serlo-staging.dev/26163) ([current revision 242082](https://serlo-staging.dev/242082))
* [article 27801](https://serlo-staging.dev/27801) ([current revision 242132](https://serlo-staging.dev/242132))
* [article 239022](https://serlo-staging.dev/239022) ([current revision 242158](https://serlo-staging.dev/242158))
* [article 199231](https://serlo-staging.dev/199231) ([current revision 242222](https://serlo-staging.dev/242222))
* [article 29577](https://serlo-staging.dev/29577) ([current revision 242538](https://serlo-staging.dev/242538))
* [article 32567](https://serlo-staging.dev/32567) ([current revision 242562](https://serlo-staging.dev/242562))
* [article 34019](https://serlo-staging.dev/34019) ([current revision 242608](https://serlo-staging.dev/242608))
* [article 28557](https://serlo-staging.dev/28557) ([current revision 242806](https://serlo-staging.dev/242806))
* [article 36160](https://serlo-staging.dev/36160) ([current revision 242813](https://serlo-staging.dev/242813))
* [article 35135](https://serlo-staging.dev/35135) ([current revision 242867](https://serlo-staging.dev/242867))
* [article 1767](https://serlo-staging.dev/1767) ([current revision 242870](https://serlo-staging.dev/242870))
* [article 241104](https://serlo-staging.dev/241104) ([current revision 242899](https://serlo-staging.dev/242899))
* [article 240050](https://serlo-staging.dev/240050) ([current revision 242910](https://serlo-staging.dev/242910))
* [article 242841](https://serlo-staging.dev/242841) ([current revision 242981](https://serlo-staging.dev/242981))
* [article 186937](https://serlo-staging.dev/186937) ([current revision 243041](https://serlo-staging.dev/243041))
* [article 240630](https://serlo-staging.dev/240630) ([current revision 243057](https://serlo-staging.dev/243057))
* [article 230163](https://serlo-staging.dev/230163) ([current revision 243278](https://serlo-staging.dev/243278))
* [article 84541](https://serlo-staging.dev/84541) ([current revision 243335](https://serlo-staging.dev/243335))
* [article 1565](https://serlo-staging.dev/1565) ([current revision 243496](https://serlo-staging.dev/243496))
* [article 100130](https://serlo-staging.dev/100130) ([current revision 243587](https://serlo-staging.dev/243587))
* [article 233786](https://serlo-staging.dev/233786) ([current revision 243645](https://serlo-staging.dev/243645))
* [article 243577](https://serlo-staging.dev/243577) ([current revision 243977](https://serlo-staging.dev/243977))
* [article 36268](https://serlo-staging.dev/36268) ([current revision 243988](https://serlo-staging.dev/243988))
* [article 1945](https://serlo-staging.dev/1945) ([current revision 244060](https://serlo-staging.dev/244060))
* [article 241559](https://serlo-staging.dev/241559) ([current revision 244132](https://serlo-staging.dev/244132))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [article 27826](https://serlo-staging.dev/27826) ([current revision 244341](https://serlo-staging.dev/244341))
* [article 51384](https://serlo-staging.dev/51384) ([current revision 244423](https://serlo-staging.dev/244423))
* [article 232392](https://serlo-staging.dev/232392) ([current revision 244600](https://serlo-staging.dev/244600))
* [article 231956](https://serlo-staging.dev/231956) ([current revision 244601](https://serlo-staging.dev/244601))
* [article 55437](https://serlo-staging.dev/55437) ([current revision 244650](https://serlo-staging.dev/244650))
* [article 55476](https://serlo-staging.dev/55476) ([current revision 244654](https://serlo-staging.dev/244654))
* [article 243981](https://serlo-staging.dev/243981) ([current revision 244716](https://serlo-staging.dev/244716))
* [article 53475](https://serlo-staging.dev/53475) ([current revision 244796](https://serlo-staging.dev/244796))
* [article 244538](https://serlo-staging.dev/244538) ([current revision 244832](https://serlo-staging.dev/244832))
* [article 244568](https://serlo-staging.dev/244568) ([current revision 244834](https://serlo-staging.dev/244834))
* [article 43048](https://serlo-staging.dev/43048) ([current revision 245352](https://serlo-staging.dev/245352))
* [article 238693](https://serlo-staging.dev/238693) ([current revision 245555](https://serlo-staging.dev/245555))
* [article 245448](https://serlo-staging.dev/245448) ([current revision 245756](https://serlo-staging.dev/245756))
* [article 226222](https://serlo-staging.dev/226222) ([current revision 245912](https://serlo-staging.dev/245912))
* [article 244510](https://serlo-staging.dev/244510) ([current revision 245958](https://serlo-staging.dev/245958))
* [event 196426](https://serlo-staging.dev/196426) ([current revision 246085](https://serlo-staging.dev/246085))
* [article 224619](https://serlo-staging.dev/224619) ([current revision 246684](https://serlo-staging.dev/246684))
* [article 190844](https://serlo-staging.dev/190844) ([current revision 247171](https://serlo-staging.dev/247171))
* [event 146500](https://serlo-staging.dev/146500) ([current revision 247191](https://serlo-staging.dev/247191))
* [article 238459](https://serlo-staging.dev/238459) ([current revision 247279](https://serlo-staging.dev/247279))
* [article 224376](https://serlo-staging.dev/224376) ([current revision 247595](https://serlo-staging.dev/247595))
* [text-exercise 8997](https://serlo-staging.dev/8997) ([current revision 247636](https://serlo-staging.dev/247636))
* [article 244678](https://serlo-staging.dev/244678) ([current revision 248070](https://serlo-staging.dev/248070))
* [article 66440](https://serlo-staging.dev/66440) ([current revision 248098](https://serlo-staging.dev/248098))
* [article 1535](https://serlo-staging.dev/1535) ([current revision 248219](https://serlo-staging.dev/248219))
* [article 1595](https://serlo-staging.dev/1595) ([current revision 248552](https://serlo-staging.dev/248552))
* [event 146498](https://serlo-staging.dev/146498) ([current revision 248786](https://serlo-staging.dev/248786))
* [event 146512](https://serlo-staging.dev/146512) ([current revision 248787](https://serlo-staging.dev/248787))
* [event 150501](https://serlo-staging.dev/150501) ([current revision 248788](https://serlo-staging.dev/248788))
* [article 1559](https://serlo-staging.dev/1559) ([current revision 248816](https://serlo-staging.dev/248816))
* [article 1745](https://serlo-staging.dev/1745) ([current revision 248828](https://serlo-staging.dev/248828))
* [article 2033](https://serlo-staging.dev/2033) ([current revision 248853](https://serlo-staging.dev/248853))
* [article 46787](https://serlo-staging.dev/46787) ([current revision 248889](https://serlo-staging.dev/248889))
* [article 46629](https://serlo-staging.dev/46629) ([current revision 248891](https://serlo-staging.dev/248891))
* [article 36070](https://serlo-staging.dev/36070) ([current revision 248897](https://serlo-staging.dev/248897))
* [article 1553](https://serlo-staging.dev/1553) ([current revision 248981](https://serlo-staging.dev/248981))
* [article 214562](https://serlo-staging.dev/214562) ([current revision 248995](https://serlo-staging.dev/248995))
* [article 222167](https://serlo-staging.dev/222167) ([current revision 249064](https://serlo-staging.dev/249064))
* [article 213337](https://serlo-staging.dev/213337) ([current revision 249066](https://serlo-staging.dev/249066))
* [article 213334](https://serlo-staging.dev/213334) ([current revision 249067](https://serlo-staging.dev/249067))
* [article 212976](https://serlo-staging.dev/212976) ([current revision 249073](https://serlo-staging.dev/249073))
* [article 1577](https://serlo-staging.dev/1577) ([current revision 249079](https://serlo-staging.dev/249079))
* [article 210061](https://serlo-staging.dev/210061) ([current revision 249091](https://serlo-staging.dev/249091))
* [article 1579](https://serlo-staging.dev/1579) ([current revision 249098](https://serlo-staging.dev/249098))
* [article 209594](https://serlo-staging.dev/209594) ([current revision 249163](https://serlo-staging.dev/249163))
* [article 209254](https://serlo-staging.dev/209254) ([current revision 249164](https://serlo-staging.dev/249164))
* [article 209131](https://serlo-staging.dev/209131) ([current revision 249167](https://serlo-staging.dev/249167))
* [article 204851](https://serlo-staging.dev/204851) ([current revision 249170](https://serlo-staging.dev/249170))
* [article 203177](https://serlo-staging.dev/203177) ([current revision 249187](https://serlo-staging.dev/249187))
* [article 202125](https://serlo-staging.dev/202125) ([current revision 249200](https://serlo-staging.dev/249200))
* [article 201038](https://serlo-staging.dev/201038) ([current revision 249205](https://serlo-staging.dev/249205))
* [article 199695](https://serlo-staging.dev/199695) ([current revision 249221](https://serlo-staging.dev/249221))

# Entites with plugin blockquote

* [course-page 145842](https://serlo-staging.dev/145842) ([current revision 146879](https://serlo-staging.dev/146879))
* [text-exercise 149487](https://serlo-staging.dev/149487) ([current revision 150043](https://serlo-staging.dev/150043))
* [article 153253](https://serlo-staging.dev/153253) ([current revision 153259](https://serlo-staging.dev/153259))
* [article 153396](https://serlo-staging.dev/153396) ([current revision 153423](https://serlo-staging.dev/153423))
* [article 153437](https://serlo-staging.dev/153437) ([current revision 153438](https://serlo-staging.dev/153438))
* [article 153472](https://serlo-staging.dev/153472) ([current revision 153488](https://serlo-staging.dev/153488))
* [article 153915](https://serlo-staging.dev/153915) ([current revision 153927](https://serlo-staging.dev/153927))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [text-exercise 85711](https://serlo-staging.dev/85711) ([current revision 159826](https://serlo-staging.dev/159826))
* [course-page 160560](https://serlo-staging.dev/160560) ([current revision 160571](https://serlo-staging.dev/160571))
* [course-page 160581](https://serlo-staging.dev/160581) ([current revision 160591](https://serlo-staging.dev/160591))
* [text-exercise 161294](https://serlo-staging.dev/161294) ([current revision 164220](https://serlo-staging.dev/164220))
* [text-solution 164622](https://serlo-staging.dev/164622) ([current revision 166886](https://serlo-staging.dev/166886))
* [article 167847](https://serlo-staging.dev/167847) ([current revision 167922](https://serlo-staging.dev/167922))
* [text-exercise 168267](https://serlo-staging.dev/168267) ([current revision 169648](https://serlo-staging.dev/169648))
* [text-exercise 169709](https://serlo-staging.dev/169709) ([current revision 169711](https://serlo-staging.dev/169711))
* [text-solution 169716](https://serlo-staging.dev/169716) ([current revision 169718](https://serlo-staging.dev/169718))
* [course-page 169720](https://serlo-staging.dev/169720) ([current revision 169733](https://serlo-staging.dev/169733))
* [text-exercise 169713](https://serlo-staging.dev/169713) ([current revision 169809](https://serlo-staging.dev/169809))
* [text-solution 47202](https://serlo-staging.dev/47202) ([current revision 170464](https://serlo-staging.dev/170464))
* [article 167755](https://serlo-staging.dev/167755) ([current revision 170467](https://serlo-staging.dev/170467))
* [article 168400](https://serlo-staging.dev/168400) ([current revision 170632](https://serlo-staging.dev/170632))
* [article 170872](https://serlo-staging.dev/170872) ([current revision 171950](https://serlo-staging.dev/171950))
* [article 2105](https://serlo-staging.dev/2105) ([current revision 172953](https://serlo-staging.dev/172953))
* [article 168163](https://serlo-staging.dev/168163) ([current revision 173486](https://serlo-staging.dev/173486))
* [course-page 31241](https://serlo-staging.dev/31241) ([current revision 174401](https://serlo-staging.dev/174401))
* [course-page 177831](https://serlo-staging.dev/177831) ([current revision 177840](https://serlo-staging.dev/177840))
* [text-solution 106241](https://serlo-staging.dev/106241) ([current revision 179371](https://serlo-staging.dev/179371))
* [article 167497](https://serlo-staging.dev/167497) ([current revision 179907](https://serlo-staging.dev/179907))
* [text-exercise 182017](https://serlo-staging.dev/182017) ([current revision 183303](https://serlo-staging.dev/183303))
* [text-solution 183392](https://serlo-staging.dev/183392) ([current revision 184270](https://serlo-staging.dev/184270))
* [article 186621](https://serlo-staging.dev/186621) ([current revision 186626](https://serlo-staging.dev/186626))
* [course-page 188326](https://serlo-staging.dev/188326) ([current revision 188385](https://serlo-staging.dev/188385))
* [article 188985](https://serlo-staging.dev/188985) ([current revision 191254](https://serlo-staging.dev/191254))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [course-page 193828](https://serlo-staging.dev/193828) ([current revision 193829](https://serlo-staging.dev/193829))
* [course-page 193817](https://serlo-staging.dev/193817) ([current revision 193842](https://serlo-staging.dev/193842))
* [course-page 178835](https://serlo-staging.dev/178835) ([current revision 195226](https://serlo-staging.dev/195226))
* [article 195560](https://serlo-staging.dev/195560) ([current revision 195823](https://serlo-staging.dev/195823))
* [course-page 196057](https://serlo-staging.dev/196057) ([current revision 196611](https://serlo-staging.dev/196611))
* [course-page 195589](https://serlo-staging.dev/195589) ([current revision 196613](https://serlo-staging.dev/196613))
* [article 196646](https://serlo-staging.dev/196646) ([current revision 196666](https://serlo-staging.dev/196666))
* [article 167816](https://serlo-staging.dev/167816) ([current revision 197032](https://serlo-staging.dev/197032))
* [article 181857](https://serlo-staging.dev/181857) ([current revision 198744](https://serlo-staging.dev/198744))
* [article 201230](https://serlo-staging.dev/201230) ([current revision 201233](https://serlo-staging.dev/201233))
* [course-page 201507](https://serlo-staging.dev/201507) ([current revision 201586](https://serlo-staging.dev/201586))
* [article 185670](https://serlo-staging.dev/185670) ([current revision 202969](https://serlo-staging.dev/202969))
* [article 203263](https://serlo-staging.dev/203263) ([current revision 203265](https://serlo-staging.dev/203265))
* [article 184341](https://serlo-staging.dev/184341) ([current revision 204156](https://serlo-staging.dev/204156))
* [article 183223](https://serlo-staging.dev/183223) ([current revision 204647](https://serlo-staging.dev/204647))
* [article 204982](https://serlo-staging.dev/204982) ([current revision 204983](https://serlo-staging.dev/204983))
* [course-page 205022](https://serlo-staging.dev/205022) ([current revision 205026](https://serlo-staging.dev/205026))
* [course-page 205023](https://serlo-staging.dev/205023) ([current revision 205031](https://serlo-staging.dev/205031))
* [course-page 179604](https://serlo-staging.dev/179604) ([current revision 205308](https://serlo-staging.dev/205308))
* [article 188454](https://serlo-staging.dev/188454) ([current revision 209878](https://serlo-staging.dev/209878))
* [article 203882](https://serlo-staging.dev/203882) ([current revision 209883](https://serlo-staging.dev/209883))
* [article 204063](https://serlo-staging.dev/204063) ([current revision 209884](https://serlo-staging.dev/209884))
* [article 210007](https://serlo-staging.dev/210007) ([current revision 210013](https://serlo-staging.dev/210013))
* [course-page 210638](https://serlo-staging.dev/210638) ([current revision 210647](https://serlo-staging.dev/210647))
* [course-page 210640](https://serlo-staging.dev/210640) ([current revision 210649](https://serlo-staging.dev/210649))
* [course-page 210641](https://serlo-staging.dev/210641) ([current revision 210650](https://serlo-staging.dev/210650))
* [course-page 210655](https://serlo-staging.dev/210655) ([current revision 210663](https://serlo-staging.dev/210663))
* [course-page 210657](https://serlo-staging.dev/210657) ([current revision 210665](https://serlo-staging.dev/210665))
* [course-page 210658](https://serlo-staging.dev/210658) ([current revision 210666](https://serlo-staging.dev/210666))
* [course-page 210780](https://serlo-staging.dev/210780) ([current revision 210790](https://serlo-staging.dev/210790))
* [course-page 210783](https://serlo-staging.dev/210783) ([current revision 210793](https://serlo-staging.dev/210793))
* [course-page 210787](https://serlo-staging.dev/210787) ([current revision 210797](https://serlo-staging.dev/210797))
* [course-page 210798](https://serlo-staging.dev/210798) ([current revision 210806](https://serlo-staging.dev/210806))
* [course-page 210801](https://serlo-staging.dev/210801) ([current revision 210809](https://serlo-staging.dev/210809))
* [course-page 210805](https://serlo-staging.dev/210805) ([current revision 210813](https://serlo-staging.dev/210813))
* [course-page 196058](https://serlo-staging.dev/196058) ([current revision 212805](https://serlo-staging.dev/212805))
* [article 201375](https://serlo-staging.dev/201375) ([current revision 213797](https://serlo-staging.dev/213797))
* [article 205298](https://serlo-staging.dev/205298) ([current revision 213799](https://serlo-staging.dev/213799))
* [article 210777](https://serlo-staging.dev/210777) ([current revision 213803](https://serlo-staging.dev/213803))
* [article 201974](https://serlo-staging.dev/201974) ([current revision 213833](https://serlo-staging.dev/213833))
* [article 215903](https://serlo-staging.dev/215903) ([current revision 215938](https://serlo-staging.dev/215938))
* [article 215941](https://serlo-staging.dev/215941) ([current revision 215943](https://serlo-staging.dev/215943))
* [article 214242](https://serlo-staging.dev/214242) ([current revision 216186](https://serlo-staging.dev/216186))
* [article 216917](https://serlo-staging.dev/216917) ([current revision 216921](https://serlo-staging.dev/216921))
* [text-solution 217263](https://serlo-staging.dev/217263) ([current revision 217268](https://serlo-staging.dev/217268))
* [article 158556](https://serlo-staging.dev/158556) ([current revision 219189](https://serlo-staging.dev/219189))
* [article 199237](https://serlo-staging.dev/199237) ([current revision 220831](https://serlo-staging.dev/220831))
* [article 205303](https://serlo-staging.dev/205303) ([current revision 220836](https://serlo-staging.dev/220836))
* [article 220880](https://serlo-staging.dev/220880) ([current revision 220958](https://serlo-staging.dev/220958))
* [course-page 136399](https://serlo-staging.dev/136399) ([current revision 222176](https://serlo-staging.dev/222176))
* [course-page 132723](https://serlo-staging.dev/132723) ([current revision 222178](https://serlo-staging.dev/222178))
* [article 222611](https://serlo-staging.dev/222611) ([current revision 222626](https://serlo-staging.dev/222626))
* [article 1693](https://serlo-staging.dev/1693) ([current revision 223403](https://serlo-staging.dev/223403))
* [course-page 120589](https://serlo-staging.dev/120589) ([current revision 223987](https://serlo-staging.dev/223987))
* [course-page 145638](https://serlo-staging.dev/145638) ([current revision 224141](https://serlo-staging.dev/224141))
* [article 223914](https://serlo-staging.dev/223914) ([current revision 224306](https://serlo-staging.dev/224306))
* [course-page 222214](https://serlo-staging.dev/222214) ([current revision 224637](https://serlo-staging.dev/224637))
* [text-solution 224832](https://serlo-staging.dev/224832) ([current revision 224834](https://serlo-staging.dev/224834))
* [article 226501](https://serlo-staging.dev/226501) ([current revision 226553](https://serlo-staging.dev/226553))
* [article 193313](https://serlo-staging.dev/193313) ([current revision 226810](https://serlo-staging.dev/226810))
* [article 134867](https://serlo-staging.dev/134867) ([current revision 226927](https://serlo-staging.dev/226927))
* [text-solution 228168](https://serlo-staging.dev/228168) ([current revision 228442](https://serlo-staging.dev/228442))
* [article 210900](https://serlo-staging.dev/210900) ([current revision 228614](https://serlo-staging.dev/228614))
* [course-page 229603](https://serlo-staging.dev/229603) ([current revision 229947](https://serlo-staging.dev/229947))
* [course-page 229605](https://serlo-staging.dev/229605) ([current revision 229988](https://serlo-staging.dev/229988))
* [course-page 229609](https://serlo-staging.dev/229609) ([current revision 229999](https://serlo-staging.dev/229999))
* [course-page 119277](https://serlo-staging.dev/119277) ([current revision 230094](https://serlo-staging.dev/230094))
* [article 213046](https://serlo-staging.dev/213046) ([current revision 233779](https://serlo-staging.dev/233779))
* [article 211859](https://serlo-staging.dev/211859) ([current revision 233780](https://serlo-staging.dev/233780))
* [article 213105](https://serlo-staging.dev/213105) ([current revision 233781](https://serlo-staging.dev/233781))
* [course-page 222284](https://serlo-staging.dev/222284) ([current revision 234328](https://serlo-staging.dev/234328))
* [course-page 234472](https://serlo-staging.dev/234472) ([current revision 234480](https://serlo-staging.dev/234480))
* [course-page 235564](https://serlo-staging.dev/235564) ([current revision 236289](https://serlo-staging.dev/236289))
* [text-solution 227048](https://serlo-staging.dev/227048) ([current revision 237444](https://serlo-staging.dev/237444))
* [course-page 237924](https://serlo-staging.dev/237924) ([current revision 237930](https://serlo-staging.dev/237930))
* [course-page 237926](https://serlo-staging.dev/237926) ([current revision 237932](https://serlo-staging.dev/237932))
* [course-page 237982](https://serlo-staging.dev/237982) ([current revision 237988](https://serlo-staging.dev/237988))
* [course-page 237983](https://serlo-staging.dev/237983) ([current revision 237989](https://serlo-staging.dev/237989))
* [course-page 237984](https://serlo-staging.dev/237984) ([current revision 237990](https://serlo-staging.dev/237990))
* [article 202005](https://serlo-staging.dev/202005) ([current revision 238018](https://serlo-staging.dev/238018))
* [course-page 237816](https://serlo-staging.dev/237816) ([current revision 238517](https://serlo-staging.dev/238517))
* [article 231621](https://serlo-staging.dev/231621) ([current revision 239036](https://serlo-staging.dev/239036))
* [course-page 237925](https://serlo-staging.dev/237925) ([current revision 239057](https://serlo-staging.dev/239057))
* [course-page 237927](https://serlo-staging.dev/237927) ([current revision 239058](https://serlo-staging.dev/239058))
* [article 227314](https://serlo-staging.dev/227314) ([current revision 239788](https://serlo-staging.dev/239788))
* [article 240351](https://serlo-staging.dev/240351) ([current revision 240444](https://serlo-staging.dev/240444))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [article 27475](https://serlo-staging.dev/27475) ([current revision 242408](https://serlo-staging.dev/242408))
* [article 27403](https://serlo-staging.dev/27403) ([current revision 242418](https://serlo-staging.dev/242418))
* [article 214865](https://serlo-staging.dev/214865) ([current revision 242422](https://serlo-staging.dev/242422))
* [article 27566](https://serlo-staging.dev/27566) ([current revision 242516](https://serlo-staging.dev/242516))
* [article 33921](https://serlo-staging.dev/33921) ([current revision 242609](https://serlo-staging.dev/242609))
* [article 239733](https://serlo-staging.dev/239733) ([current revision 242869](https://serlo-staging.dev/242869))
* [article 202973](https://serlo-staging.dev/202973) ([current revision 242966](https://serlo-staging.dev/242966))
* [article 243790](https://serlo-staging.dev/243790) ([current revision 243844](https://serlo-staging.dev/243844))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [course-page 244309](https://serlo-staging.dev/244309) ([current revision 244474](https://serlo-staging.dev/244474))
* [article 243867](https://serlo-staging.dev/243867) ([current revision 244502](https://serlo-staging.dev/244502))
* [article 53415](https://serlo-staging.dev/53415) ([current revision 244576](https://serlo-staging.dev/244576))
* [text-solution 195446](https://serlo-staging.dev/195446) ([current revision 244870](https://serlo-staging.dev/244870))
* [text-solution 192951](https://serlo-staging.dev/192951) ([current revision 245225](https://serlo-staging.dev/245225))
* [article 241639](https://serlo-staging.dev/241639) ([current revision 245332](https://serlo-staging.dev/245332))
* [course-page 245715](https://serlo-staging.dev/245715) ([current revision 245722](https://serlo-staging.dev/245722))
* [article 226222](https://serlo-staging.dev/226222) ([current revision 245912](https://serlo-staging.dev/245912))
* [article 245690](https://serlo-staging.dev/245690) ([current revision 246004](https://serlo-staging.dev/246004))
* [article 214562](https://serlo-staging.dev/214562) ([current revision 248995](https://serlo-staging.dev/248995))
* [article 212976](https://serlo-staging.dev/212976) ([current revision 249073](https://serlo-staging.dev/249073))
* [article 212396](https://serlo-staging.dev/212396) ([current revision 249080](https://serlo-staging.dev/249080))
* [article 210119](https://serlo-staging.dev/210119) ([current revision 249087](https://serlo-staging.dev/249087))
* [article 200685](https://serlo-staging.dev/200685) ([current revision 249095](https://serlo-staging.dev/249095))
* [article 200819](https://serlo-staging.dev/200819) ([current revision 249096](https://serlo-staging.dev/249096))
* [article 209254](https://serlo-staging.dev/209254) ([current revision 249164](https://serlo-staging.dev/249164))
* [article 203177](https://serlo-staging.dev/203177) ([current revision 249187](https://serlo-staging.dev/249187))
* [article 201038](https://serlo-staging.dev/201038) ([current revision 249205](https://serlo-staging.dev/249205))

In [5]:
printRevisionsWithPlugin("table")

# Entites with plugin table

* [text-solution 128031](https://serlo-staging.dev/128031) ([current revision 151476](https://serlo-staging.dev/151476))
* [course-page 63751](https://serlo-staging.dev/63751) ([current revision 152187](https://serlo-staging.dev/152187))
* [text-exercise 9165](https://serlo-staging.dev/9165) ([current revision 152222](https://serlo-staging.dev/152222))
* [text-exercise 4185](https://serlo-staging.dev/4185) ([current revision 152664](https://serlo-staging.dev/152664))
* [text-exercise 5273](https://serlo-staging.dev/5273) ([current revision 152765](https://serlo-staging.dev/152765))
* [text-solution 5275](https://serlo-staging.dev/5275) ([current revision 152766](https://serlo-staging.dev/152766))
* [text-exercise 7217](https://serlo-staging.dev/7217) ([current revision 152842](https://serlo-staging.dev/152842))
* [text-exercise 9255](https://serlo-staging.dev/9255) ([current revision 152911](https://serlo-staging.dev/152911))
* [text-exercise 9259](https://serlo-staging.dev/9259) ([current revision 152925](https://serlo-staging.dev/152925))
* [text-exercise 9701](https://serlo-staging.dev/9701) ([current revision 152931](https://serlo-staging.dev/152931))
* [text-exercise 7077](https://serlo-staging.dev/7077) ([current revision 152984](https://serlo-staging.dev/152984))
* [text-solution 7079](https://serlo-staging.dev/7079) ([current revision 152985](https://serlo-staging.dev/152985))
* [text-solution 84806](https://serlo-staging.dev/84806) ([current revision 153102](https://serlo-staging.dev/153102))
* [text-solution 151792](https://serlo-staging.dev/151792) ([current revision 153232](https://serlo-staging.dev/153232))
* [text-exercise 7611](https://serlo-staging.dev/7611) ([current revision 153280](https://serlo-staging.dev/153280))
* [text-exercise 7797](https://serlo-staging.dev/7797) ([current revision 153312](https://serlo-staging.dev/153312))
* [text-exercise 63994](https://serlo-staging.dev/63994) ([current revision 153504](https://serlo-staging.dev/153504))
* [text-solution 8303](https://serlo-staging.dev/8303) ([current revision 153900](https://serlo-staging.dev/153900))
* [text-solution 107676](https://serlo-staging.dev/107676) ([current revision 154248](https://serlo-staging.dev/154248))
* [text-solution 107667](https://serlo-staging.dev/107667) ([current revision 154313](https://serlo-staging.dev/154313))
* [text-exercise 10613](https://serlo-staging.dev/10613) ([current revision 154534](https://serlo-staging.dev/154534))
* [text-solution 13419](https://serlo-staging.dev/13419) ([current revision 154698](https://serlo-staging.dev/154698))
* [text-solution 28153](https://serlo-staging.dev/28153) ([current revision 154703](https://serlo-staging.dev/154703))
* [text-solution 29973](https://serlo-staging.dev/29973) ([current revision 155064](https://serlo-staging.dev/155064))
* [text-solution 29987](https://serlo-staging.dev/29987) ([current revision 155083](https://serlo-staging.dev/155083))
* [text-solution 30001](https://serlo-staging.dev/30001) ([current revision 155103](https://serlo-staging.dev/155103))
* [text-exercise 11217](https://serlo-staging.dev/11217) ([current revision 155124](https://serlo-staging.dev/155124))
* [text-exercise 12105](https://serlo-staging.dev/12105) ([current revision 155395](https://serlo-staging.dev/155395))
* [text-solution 12107](https://serlo-staging.dev/12107) ([current revision 155396](https://serlo-staging.dev/155396))
* [text-exercise 37648](https://serlo-staging.dev/37648) ([current revision 155592](https://serlo-staging.dev/155592))
* [text-exercise 39690](https://serlo-staging.dev/39690) ([current revision 156144](https://serlo-staging.dev/156144))
* [article 70058](https://serlo-staging.dev/70058) ([current revision 156219](https://serlo-staging.dev/156219))
* [text-exercise 50507](https://serlo-staging.dev/50507) ([current revision 156746](https://serlo-staging.dev/156746))
* [text-solution 76409](https://serlo-staging.dev/76409) ([current revision 157441](https://serlo-staging.dev/157441))
* [text-exercise 6861](https://serlo-staging.dev/6861) ([current revision 157683](https://serlo-staging.dev/157683))
* [text-solution 9257](https://serlo-staging.dev/9257) ([current revision 157737](https://serlo-staging.dev/157737))
* [text-exercise 142928](https://serlo-staging.dev/142928) ([current revision 158103](https://serlo-staging.dev/158103))
* [text-solution 145346](https://serlo-staging.dev/145346) ([current revision 158104](https://serlo-staging.dev/158104))
* [text-solution 75763](https://serlo-staging.dev/75763) ([current revision 158218](https://serlo-staging.dev/158218))
* [text-solution 128366](https://serlo-staging.dev/128366) ([current revision 158551](https://serlo-staging.dev/158551))
* [text-solution 128372](https://serlo-staging.dev/128372) ([current revision 158552](https://serlo-staging.dev/158552))
* [text-solution 107656](https://serlo-staging.dev/107656) ([current revision 158893](https://serlo-staging.dev/158893))
* [text-solution 138566](https://serlo-staging.dev/138566) ([current revision 158992](https://serlo-staging.dev/158992))
* [text-solution 7219](https://serlo-staging.dev/7219) ([current revision 159023](https://serlo-staging.dev/159023))
* [text-solution 13923](https://serlo-staging.dev/13923) ([current revision 159042](https://serlo-staging.dev/159042))
* [text-solution 9735](https://serlo-staging.dev/9735) ([current revision 159066](https://serlo-staging.dev/159066))
* [text-solution 128358](https://serlo-staging.dev/128358) ([current revision 159118](https://serlo-staging.dev/159118))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [text-solution 9261](https://serlo-staging.dev/9261) ([current revision 159453](https://serlo-staging.dev/159453))
* [text-solution 77083](https://serlo-staging.dev/77083) ([current revision 159456](https://serlo-staging.dev/159456))
* [text-exercise 85854](https://serlo-staging.dev/85854) ([current revision 159943](https://serlo-staging.dev/159943))
* [text-solution 6875](https://serlo-staging.dev/6875) ([current revision 160184](https://serlo-staging.dev/160184))
* [text-exercise 5255](https://serlo-staging.dev/5255) ([current revision 160873](https://serlo-staging.dev/160873))
* [text-solution 5257](https://serlo-staging.dev/5257) ([current revision 160878](https://serlo-staging.dev/160878))
* [text-exercise 38736](https://serlo-staging.dev/38736) ([current revision 161350](https://serlo-staging.dev/161350))
* [grouped-text-exercise 6003](https://serlo-staging.dev/6003) ([current revision 161435](https://serlo-staging.dev/161435))
* [grouped-text-exercise 6007](https://serlo-staging.dev/6007) ([current revision 161437](https://serlo-staging.dev/161437))
* [text-solution 145569](https://serlo-staging.dev/145569) ([current revision 162402](https://serlo-staging.dev/162402))
* [article 150045](https://serlo-staging.dev/150045) ([current revision 163078](https://serlo-staging.dev/163078))
* [text-solution 163846](https://serlo-staging.dev/163846) ([current revision 163850](https://serlo-staging.dev/163850))
* [text-solution 10795](https://serlo-staging.dev/10795) ([current revision 164102](https://serlo-staging.dev/164102))
* [text-solution 10799](https://serlo-staging.dev/10799) ([current revision 164104](https://serlo-staging.dev/164104))
* [text-solution 10803](https://serlo-staging.dev/10803) ([current revision 164106](https://serlo-staging.dev/164106))
* [text-solution 10807](https://serlo-staging.dev/10807) ([current revision 164108](https://serlo-staging.dev/164108))
* [text-solution 10811](https://serlo-staging.dev/10811) ([current revision 164110](https://serlo-staging.dev/164110))
* [text-solution 10815](https://serlo-staging.dev/10815) ([current revision 164112](https://serlo-staging.dev/164112))
* [text-solution 10819](https://serlo-staging.dev/10819) ([current revision 164114](https://serlo-staging.dev/164114))
* [text-solution 10823](https://serlo-staging.dev/10823) ([current revision 164116](https://serlo-staging.dev/164116))
* [text-solution 10827](https://serlo-staging.dev/10827) ([current revision 164118](https://serlo-staging.dev/164118))
* [text-solution 10831](https://serlo-staging.dev/10831) ([current revision 164120](https://serlo-staging.dev/164120))
* [article 163790](https://serlo-staging.dev/163790) ([current revision 164538](https://serlo-staging.dev/164538))
* [text-solution 11607](https://serlo-staging.dev/11607) ([current revision 164719](https://serlo-staging.dev/164719))
* [text-solution 11615](https://serlo-staging.dev/11615) ([current revision 164721](https://serlo-staging.dev/164721))
* [text-solution 19653](https://serlo-staging.dev/19653) ([current revision 165399](https://serlo-staging.dev/165399))
* [text-solution 19659](https://serlo-staging.dev/19659) ([current revision 165401](https://serlo-staging.dev/165401))
* [text-solution 19684](https://serlo-staging.dev/19684) ([current revision 165403](https://serlo-staging.dev/165403))
* [text-solution 27290](https://serlo-staging.dev/27290) ([current revision 165700](https://serlo-staging.dev/165700))
* [text-solution 27294](https://serlo-staging.dev/27294) ([current revision 165702](https://serlo-staging.dev/165702))
* [text-solution 27298](https://serlo-staging.dev/27298) ([current revision 165704](https://serlo-staging.dev/165704))
* [text-solution 27302](https://serlo-staging.dev/27302) ([current revision 165706](https://serlo-staging.dev/165706))
* [article 166172](https://serlo-staging.dev/166172) ([current revision 167146](https://serlo-staging.dev/167146))
* [text-exercise 13921](https://serlo-staging.dev/13921) ([current revision 167448](https://serlo-staging.dev/167448))
* [article 167851](https://serlo-staging.dev/167851) ([current revision 167921](https://serlo-staging.dev/167921))
* [article 62559](https://serlo-staging.dev/62559) ([current revision 168430](https://serlo-staging.dev/168430))
* [course-page 109329](https://serlo-staging.dev/109329) ([current revision 168575](https://serlo-staging.dev/168575))
* [text-exercise-group 133463](https://serlo-staging.dev/133463) ([current revision 168808](https://serlo-staging.dev/168808))
* [text-exercise-group 111250](https://serlo-staging.dev/111250) ([current revision 168919](https://serlo-staging.dev/168919))
* [text-exercise 2991](https://serlo-staging.dev/2991) ([current revision 169412](https://serlo-staging.dev/169412))
* [text-solution 2993](https://serlo-staging.dev/2993) ([current revision 169413](https://serlo-staging.dev/169413))
* [text-solution 2997](https://serlo-staging.dev/2997) ([current revision 169440](https://serlo-staging.dev/169440))
* [text-solution 5183](https://serlo-staging.dev/5183) ([current revision 169871](https://serlo-staging.dev/169871))
* [article 170030](https://serlo-staging.dev/170030) ([current revision 170072](https://serlo-staging.dev/170072))
* [text-solution 3461](https://serlo-staging.dev/3461) ([current revision 170157](https://serlo-staging.dev/170157))
* [text-solution 5999](https://serlo-staging.dev/5999) ([current revision 170446](https://serlo-staging.dev/170446))
* [text-solution 123360](https://serlo-staging.dev/123360) ([current revision 170554](https://serlo-staging.dev/170554))
* [text-solution 124347](https://serlo-staging.dev/124347) ([current revision 170561](https://serlo-staging.dev/170561))
* [text-solution 6377](https://serlo-staging.dev/6377) ([current revision 170564](https://serlo-staging.dev/170564))
* [text-exercise 6411](https://serlo-staging.dev/6411) ([current revision 170571](https://serlo-staging.dev/170571))
* [text-solution 6413](https://serlo-staging.dev/6413) ([current revision 170572](https://serlo-staging.dev/170572))
* [text-solution 6451](https://serlo-staging.dev/6451) ([current revision 170591](https://serlo-staging.dev/170591))
* [text-solution 6467](https://serlo-staging.dev/6467) ([current revision 170690](https://serlo-staging.dev/170690))
* [article 169704](https://serlo-staging.dev/169704) ([current revision 170843](https://serlo-staging.dev/170843))
* [text-exercise 9613](https://serlo-staging.dev/9613) ([current revision 170857](https://serlo-staging.dev/170857))
* [text-solution 96333](https://serlo-staging.dev/96333) ([current revision 170901](https://serlo-staging.dev/170901))
* [text-solution 96344](https://serlo-staging.dev/96344) ([current revision 170904](https://serlo-staging.dev/170904))
* [text-exercise 97378](https://serlo-staging.dev/97378) ([current revision 170907](https://serlo-staging.dev/170907))
* [text-solution 7333](https://serlo-staging.dev/7333) ([current revision 170913](https://serlo-staging.dev/170913))
* [text-solution 137837](https://serlo-staging.dev/137837) ([current revision 171267](https://serlo-staging.dev/171267))
* [article 1989](https://serlo-staging.dev/1989) ([current revision 171348](https://serlo-staging.dev/171348))
* [text-exercise 9649](https://serlo-staging.dev/9649) ([current revision 171420](https://serlo-staging.dev/171420))
* [article 1993](https://serlo-staging.dev/1993) ([current revision 171444](https://serlo-staging.dev/171444))
* [article 171413](https://serlo-staging.dev/171413) ([current revision 171462](https://serlo-staging.dev/171462))
* [text-solution 22523](https://serlo-staging.dev/22523) ([current revision 171509](https://serlo-staging.dev/171509))
* [text-exercise 28591](https://serlo-staging.dev/28591) ([current revision 171524](https://serlo-staging.dev/171524))
* [text-solution 92152](https://serlo-staging.dev/92152) ([current revision 171554](https://serlo-staging.dev/171554))
* [text-solution 171634](https://serlo-staging.dev/171634) ([current revision 171640](https://serlo-staging.dev/171640))
* [text-solution 171336](https://serlo-staging.dev/171336) ([current revision 171677](https://serlo-staging.dev/171677))
* [text-exercise 11863](https://serlo-staging.dev/11863) ([current revision 171735](https://serlo-staging.dev/171735))
* [text-solution 171624](https://serlo-staging.dev/171624) ([current revision 171802](https://serlo-staging.dev/171802))
* [article 171898](https://serlo-staging.dev/171898) ([current revision 171903](https://serlo-staging.dev/171903))
* [text-exercise 38536](https://serlo-staging.dev/38536) ([current revision 172101](https://serlo-staging.dev/172101))
* [text-solution 86520](https://serlo-staging.dev/86520) ([current revision 172361](https://serlo-staging.dev/172361))
* [article 171955](https://serlo-staging.dev/171955) ([current revision 172626](https://serlo-staging.dev/172626))
* [article 170884](https://serlo-staging.dev/170884) ([current revision 172629](https://serlo-staging.dev/172629))
* [article 164956](https://serlo-staging.dev/164956) ([current revision 172794](https://serlo-staging.dev/172794))
* [article 172793](https://serlo-staging.dev/172793) ([current revision 172796](https://serlo-staging.dev/172796))
* [article 172797](https://serlo-staging.dev/172797) ([current revision 172799](https://serlo-staging.dev/172799))
* [article 172800](https://serlo-staging.dev/172800) ([current revision 172801](https://serlo-staging.dev/172801))
* [article 172802](https://serlo-staging.dev/172802) ([current revision 172803](https://serlo-staging.dev/172803))
* [article 172929](https://serlo-staging.dev/172929) ([current revision 172930](https://serlo-staging.dev/172930))
* [text-solution 5397](https://serlo-staging.dev/5397) ([current revision 173009](https://serlo-staging.dev/173009))
* [text-solution 173077](https://serlo-staging.dev/173077) ([current revision 173125](https://serlo-staging.dev/173125))
* [grouped-text-exercise 55565](https://serlo-staging.dev/55565) ([current revision 173297](https://serlo-staging.dev/173297))
* [grouped-text-exercise 55575](https://serlo-staging.dev/55575) ([current revision 173299](https://serlo-staging.dev/173299))
* [article 168791](https://serlo-staging.dev/168791) ([current revision 173430](https://serlo-staging.dev/173430))
* [article 167810](https://serlo-staging.dev/167810) ([current revision 173431](https://serlo-staging.dev/173431))
* [article 55508](https://serlo-staging.dev/55508) ([current revision 173870](https://serlo-staging.dev/173870))
* [text-exercise-group 6063](https://serlo-staging.dev/6063) ([current revision 174311](https://serlo-staging.dev/174311))
* [text-solution 6473](https://serlo-staging.dev/6473) ([current revision 174347](https://serlo-staging.dev/174347))
* [text-solution 174707](https://serlo-staging.dev/174707) ([current revision 174709](https://serlo-staging.dev/174709))
* [course-page 175035](https://serlo-staging.dev/175035) ([current revision 175037](https://serlo-staging.dev/175037))
* [course-page 174944](https://serlo-staging.dev/174944) ([current revision 175493](https://serlo-staging.dev/175493))
* [course-page 175029](https://serlo-staging.dev/175029) ([current revision 175501](https://serlo-staging.dev/175501))
* [text-solution 29569](https://serlo-staging.dev/29569) ([current revision 176983](https://serlo-staging.dev/176983))
* [text-solution 34293](https://serlo-staging.dev/34293) ([current revision 176985](https://serlo-staging.dev/176985))
* [text-solution 34297](https://serlo-staging.dev/34297) ([current revision 176987](https://serlo-staging.dev/176987))
* [text-exercise 176821](https://serlo-staging.dev/176821) ([current revision 177011](https://serlo-staging.dev/177011))
* [text-exercise 177115](https://serlo-staging.dev/177115) ([current revision 177167](https://serlo-staging.dev/177167))
* [text-exercise 177382](https://serlo-staging.dev/177382) ([current revision 177385](https://serlo-staging.dev/177385))
* [text-exercise 177451](https://serlo-staging.dev/177451) ([current revision 177469](https://serlo-staging.dev/177469))
* [article 175766](https://serlo-staging.dev/175766) ([current revision 177473](https://serlo-staging.dev/177473))
* [text-solution 13545](https://serlo-staging.dev/13545) ([current revision 177620](https://serlo-staging.dev/177620))
* [article 178103](https://serlo-staging.dev/178103) ([current revision 178105](https://serlo-staging.dev/178105))
* [article 178183](https://serlo-staging.dev/178183) ([current revision 178184](https://serlo-staging.dev/178184))
* [article 178239](https://serlo-staging.dev/178239) ([current revision 178299](https://serlo-staging.dev/178299))
* [text-solution 49413](https://serlo-staging.dev/49413) ([current revision 178488](https://serlo-staging.dev/178488))
* [article 178185](https://serlo-staging.dev/178185) ([current revision 178688](https://serlo-staging.dev/178688))
* [article 178269](https://serlo-staging.dev/178269) ([current revision 178722](https://serlo-staging.dev/178722))
* [article 178847](https://serlo-staging.dev/178847) ([current revision 178903](https://serlo-staging.dev/178903))
* [course-page 178850](https://serlo-staging.dev/178850) ([current revision 179246](https://serlo-staging.dev/179246))
* [course-page 178858](https://serlo-staging.dev/178858) ([current revision 179249](https://serlo-staging.dev/179249))
* [course-page 178864](https://serlo-staging.dev/178864) ([current revision 179250](https://serlo-staging.dev/179250))
* [text-exercise 178981](https://serlo-staging.dev/178981) ([current revision 179362](https://serlo-staging.dev/179362))
* [article 179453](https://serlo-staging.dev/179453) ([current revision 179460](https://serlo-staging.dev/179460))
* [article 179499](https://serlo-staging.dev/179499) ([current revision 179502](https://serlo-staging.dev/179502))
* [article 179507](https://serlo-staging.dev/179507) ([current revision 179509](https://serlo-staging.dev/179509))
* [article 179510](https://serlo-staging.dev/179510) ([current revision 179511](https://serlo-staging.dev/179511))
* [article 179512](https://serlo-staging.dev/179512) ([current revision 179513](https://serlo-staging.dev/179513))
* [article 179526](https://serlo-staging.dev/179526) ([current revision 179528](https://serlo-staging.dev/179528))
* [article 179533](https://serlo-staging.dev/179533) ([current revision 179535](https://serlo-staging.dev/179535))
* [article 179536](https://serlo-staging.dev/179536) ([current revision 179541](https://serlo-staging.dev/179541))
* [article 179503](https://serlo-staging.dev/179503) ([current revision 179553](https://serlo-staging.dev/179553))
* [article 179556](https://serlo-staging.dev/179556) ([current revision 179557](https://serlo-staging.dev/179557))
* [article 179559](https://serlo-staging.dev/179559) ([current revision 179562](https://serlo-staging.dev/179562))
* [article 179560](https://serlo-staging.dev/179560) ([current revision 179563](https://serlo-staging.dev/179563))
* [article 179494](https://serlo-staging.dev/179494) ([current revision 179677](https://serlo-staging.dev/179677))
* [article 179505](https://serlo-staging.dev/179505) ([current revision 179678](https://serlo-staging.dev/179678))
* [article 179496](https://serlo-staging.dev/179496) ([current revision 179679](https://serlo-staging.dev/179679))
* [article 179524](https://serlo-staging.dev/179524) ([current revision 179681](https://serlo-staging.dev/179681))
* [article 179529](https://serlo-staging.dev/179529) ([current revision 179682](https://serlo-staging.dev/179682))
* [article 179514](https://serlo-staging.dev/179514) ([current revision 179732](https://serlo-staging.dev/179732))
* [article 179517](https://serlo-staging.dev/179517) ([current revision 179733](https://serlo-staging.dev/179733))
* [event 145590](https://serlo-staging.dev/145590) ([current revision 180193](https://serlo-staging.dev/180193))
* [article 178261](https://serlo-staging.dev/178261) ([current revision 180773](https://serlo-staging.dev/180773))
* [text-exercise 177200](https://serlo-staging.dev/177200) ([current revision 180903](https://serlo-staging.dev/180903))
* [text-solution 5199](https://serlo-staging.dev/5199) ([current revision 180955](https://serlo-staging.dev/180955))
* [article 181141](https://serlo-staging.dev/181141) ([current revision 181142](https://serlo-staging.dev/181142))
* [article 181138](https://serlo-staging.dev/181138) ([current revision 181143](https://serlo-staging.dev/181143))
* [text-solution 108099](https://serlo-staging.dev/108099) ([current revision 181301](https://serlo-staging.dev/181301))
* [text-exercise 181311](https://serlo-staging.dev/181311) ([current revision 181314](https://serlo-staging.dev/181314))
* [text-solution 181320](https://serlo-staging.dev/181320) ([current revision 181327](https://serlo-staging.dev/181327))
* [text-solution 181322](https://serlo-staging.dev/181322) ([current revision 181329](https://serlo-staging.dev/181329))
* [text-solution 181324](https://serlo-staging.dev/181324) ([current revision 181331](https://serlo-staging.dev/181331))
* [text-solution 181847](https://serlo-staging.dev/181847) ([current revision 181849](https://serlo-staging.dev/181849))
* [text-solution 181861](https://serlo-staging.dev/181861) ([current revision 181863](https://serlo-staging.dev/181863))
* [course-page 19495](https://serlo-staging.dev/19495) ([current revision 181887](https://serlo-staging.dev/181887))
* [article 27823](https://serlo-staging.dev/27823) ([current revision 181889](https://serlo-staging.dev/181889))
* [text-solution 182181](https://serlo-staging.dev/182181) ([current revision 182943](https://serlo-staging.dev/182943))
* [text-solution 182169](https://serlo-staging.dev/182169) ([current revision 182945](https://serlo-staging.dev/182945))
* [text-solution 182171](https://serlo-staging.dev/182171) ([current revision 182949](https://serlo-staging.dev/182949))
* [text-solution 182173](https://serlo-staging.dev/182173) ([current revision 182952](https://serlo-staging.dev/182952))
* [text-solution 182177](https://serlo-staging.dev/182177) ([current revision 182955](https://serlo-staging.dev/182955))
* [text-solution 182175](https://serlo-staging.dev/182175) ([current revision 182958](https://serlo-staging.dev/182958))
* [text-solution 182179](https://serlo-staging.dev/182179) ([current revision 182961](https://serlo-staging.dev/182961))
* [text-solution 182183](https://serlo-staging.dev/182183) ([current revision 182962](https://serlo-staging.dev/182962))
* [article 182840](https://serlo-staging.dev/182840) ([current revision 183020](https://serlo-staging.dev/183020))
* [text-solution 93470](https://serlo-staging.dev/93470) ([current revision 183446](https://serlo-staging.dev/183446))
* [article 183431](https://serlo-staging.dev/183431) ([current revision 183467](https://serlo-staging.dev/183467))
* [text-solution 3031](https://serlo-staging.dev/3031) ([current revision 183512](https://serlo-staging.dev/183512))
* [text-solution 183014](https://serlo-staging.dev/183014) ([current revision 183621](https://serlo-staging.dev/183621))
* [text-solution 7945](https://serlo-staging.dev/7945) ([current revision 183654](https://serlo-staging.dev/183654))
* [text-solution 5153](https://serlo-staging.dev/5153) ([current revision 184055](https://serlo-staging.dev/184055))
* [course-page 168491](https://serlo-staging.dev/168491) ([current revision 184090](https://serlo-staging.dev/184090))
* [course-page 109351](https://serlo-staging.dev/109351) ([current revision 184350](https://serlo-staging.dev/184350))
* [course-page 109419](https://serlo-staging.dev/109419) ([current revision 184351](https://serlo-staging.dev/184351))
* [article 170325](https://serlo-staging.dev/170325) ([current revision 184622](https://serlo-staging.dev/184622))
* [article 169545](https://serlo-staging.dev/169545) ([current revision 184628](https://serlo-staging.dev/184628))
* [article 170363](https://serlo-staging.dev/170363) ([current revision 184688](https://serlo-staging.dev/184688))
* [text-exercise 183740](https://serlo-staging.dev/183740) ([current revision 184698](https://serlo-staging.dev/184698))
* [text-solution 183002](https://serlo-staging.dev/183002) ([current revision 184887](https://serlo-staging.dev/184887))
* [course-page 109308](https://serlo-staging.dev/109308) ([current revision 185592](https://serlo-staging.dev/185592))
* [article 181279](https://serlo-staging.dev/181279) ([current revision 185617](https://serlo-staging.dev/185617))
* [course-page 182848](https://serlo-staging.dev/182848) ([current revision 186326](https://serlo-staging.dev/186326))
* [course-page 182855](https://serlo-staging.dev/182855) ([current revision 186328](https://serlo-staging.dev/186328))
* [text-solution 76128](https://serlo-staging.dev/76128) ([current revision 186352](https://serlo-staging.dev/186352))
* [text-solution 8803](https://serlo-staging.dev/8803) ([current revision 186472](https://serlo-staging.dev/186472))
* [text-solution 69318](https://serlo-staging.dev/69318) ([current revision 186493](https://serlo-staging.dev/186493))
* [text-solution 100504](https://serlo-staging.dev/100504) ([current revision 186503](https://serlo-staging.dev/186503))
* [article 65162](https://serlo-staging.dev/65162) ([current revision 186699](https://serlo-staging.dev/186699))
* [course-page 187203](https://serlo-staging.dev/187203) ([current revision 187216](https://serlo-staging.dev/187216))
* [course-page 18826](https://serlo-staging.dev/18826) ([current revision 187420](https://serlo-staging.dev/187420))
* [course-page 19046](https://serlo-staging.dev/19046) ([current revision 187421](https://serlo-staging.dev/187421))
* [text-exercise 5963](https://serlo-staging.dev/5963) ([current revision 187548](https://serlo-staging.dev/187548))
* [text-solution 5965](https://serlo-staging.dev/5965) ([current revision 187549](https://serlo-staging.dev/187549))
* [text-solution 9233](https://serlo-staging.dev/9233) ([current revision 187622](https://serlo-staging.dev/187622))
* [course-page 186293](https://serlo-staging.dev/186293) ([current revision 188403](https://serlo-staging.dev/188403))
* [article 178256](https://serlo-staging.dev/178256) ([current revision 188868](https://serlo-staging.dev/188868))
* [text-solution 189891](https://serlo-staging.dev/189891) ([current revision 189898](https://serlo-staging.dev/189898))
* [article 187487](https://serlo-staging.dev/187487) ([current revision 190508](https://serlo-staging.dev/190508))
* [text-solution 191118](https://serlo-staging.dev/191118) ([current revision 191125](https://serlo-staging.dev/191125))
* [article 181056](https://serlo-staging.dev/181056) ([current revision 191176](https://serlo-staging.dev/191176))
* [article 185678](https://serlo-staging.dev/185678) ([current revision 191534](https://serlo-staging.dev/191534))
* [course-page 191987](https://serlo-staging.dev/191987) ([current revision 191998](https://serlo-staging.dev/191998))
* [course-page 192037](https://serlo-staging.dev/192037) ([current revision 192064](https://serlo-staging.dev/192064))
* [article 192432](https://serlo-staging.dev/192432) ([current revision 192439](https://serlo-staging.dev/192439))
* [text-solution 191116](https://serlo-staging.dev/191116) ([current revision 192442](https://serlo-staging.dev/192442))
* [text-solution 37657](https://serlo-staging.dev/37657) ([current revision 192796](https://serlo-staging.dev/192796))
* [article 1607](https://serlo-staging.dev/1607) ([current revision 192797](https://serlo-staging.dev/192797))
* [course-page 182860](https://serlo-staging.dev/182860) ([current revision 192833](https://serlo-staging.dev/192833))
* [article 62547](https://serlo-staging.dev/62547) ([current revision 193082](https://serlo-staging.dev/193082))
* [course-page 185999](https://serlo-staging.dev/185999) ([current revision 193162](https://serlo-staging.dev/193162))
* [article 85864](https://serlo-staging.dev/85864) ([current revision 193281](https://serlo-staging.dev/193281))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [article 181058](https://serlo-staging.dev/181058) ([current revision 194425](https://serlo-staging.dev/194425))
* [course-page 180035](https://serlo-staging.dev/180035) ([current revision 194651](https://serlo-staging.dev/194651))
* [course-page 178836](https://serlo-staging.dev/178836) ([current revision 194655](https://serlo-staging.dev/194655))
* [text-solution 81263](https://serlo-staging.dev/81263) ([current revision 194778](https://serlo-staging.dev/194778))
* [course-page 178835](https://serlo-staging.dev/178835) ([current revision 195226](https://serlo-staging.dev/195226))
* [text-solution 31550](https://serlo-staging.dev/31550) ([current revision 195789](https://serlo-staging.dev/195789))
* [text-solution 31561](https://serlo-staging.dev/31561) ([current revision 195791](https://serlo-staging.dev/195791))
* [course-page 185231](https://serlo-staging.dev/185231) ([current revision 196496](https://serlo-staging.dev/196496))
* [text-exercise 9311](https://serlo-staging.dev/9311) ([current revision 196589](https://serlo-staging.dev/196589))
* [course-page 196541](https://serlo-staging.dev/196541) ([current revision 196599](https://serlo-staging.dev/196599))
* [course-page 196542](https://serlo-staging.dev/196542) ([current revision 196600](https://serlo-staging.dev/196600))
* [course-page 196547](https://serlo-staging.dev/196547) ([current revision 196657](https://serlo-staging.dev/196657))
* [text-solution 108892](https://serlo-staging.dev/108892) ([current revision 196744](https://serlo-staging.dev/196744))
* [article 1967](https://serlo-staging.dev/1967) ([current revision 196774](https://serlo-staging.dev/196774))
* [course-page 197204](https://serlo-staging.dev/197204) ([current revision 197231](https://serlo-staging.dev/197231))
* [article 42017](https://serlo-staging.dev/42017) ([current revision 197304](https://serlo-staging.dev/197304))
* [article 42091](https://serlo-staging.dev/42091) ([current revision 197307](https://serlo-staging.dev/197307))
* [article 43283](https://serlo-staging.dev/43283) ([current revision 197308](https://serlo-staging.dev/197308))
* [article 48261](https://serlo-staging.dev/48261) ([current revision 197322](https://serlo-staging.dev/197322))
* [article 53494](https://serlo-staging.dev/53494) ([current revision 197333](https://serlo-staging.dev/197333))
* [course-page 197207](https://serlo-staging.dev/197207) ([current revision 197381](https://serlo-staging.dev/197381))
* [course-page 197203](https://serlo-staging.dev/197203) ([current revision 197384](https://serlo-staging.dev/197384))
* [text-solution 5993](https://serlo-staging.dev/5993) ([current revision 197587](https://serlo-staging.dev/197587))
* [article 1699](https://serlo-staging.dev/1699) ([current revision 197695](https://serlo-staging.dev/197695))
* [course-page 197206](https://serlo-staging.dev/197206) ([current revision 197703](https://serlo-staging.dev/197703))
* [article 90410](https://serlo-staging.dev/90410) ([current revision 197881](https://serlo-staging.dev/197881))
* [article 2107](https://serlo-staging.dev/2107) ([current revision 198108](https://serlo-staging.dev/198108))
* [text-exercise 198285](https://serlo-staging.dev/198285) ([current revision 198286](https://serlo-staging.dev/198286))
* [text-solution 87707](https://serlo-staging.dev/87707) ([current revision 198287](https://serlo-staging.dev/198287))
* [article 25253](https://serlo-staging.dev/25253) ([current revision 198862](https://serlo-staging.dev/198862))
* [article 160143](https://serlo-staging.dev/160143) ([current revision 198957](https://serlo-staging.dev/198957))
* [text-solution 51598](https://serlo-staging.dev/51598) ([current revision 199458](https://serlo-staging.dev/199458))
* [text-solution 183770](https://serlo-staging.dev/183770) ([current revision 199574](https://serlo-staging.dev/199574))
* [grouped-text-exercise 199118](https://serlo-staging.dev/199118) ([current revision 199707](https://serlo-staging.dev/199707))
* [text-solution 199119](https://serlo-staging.dev/199119) ([current revision 199708](https://serlo-staging.dev/199708))
* [grouped-text-exercise 198944](https://serlo-staging.dev/198944) ([current revision 199774](https://serlo-staging.dev/199774))
* [text-solution 4191](https://serlo-staging.dev/4191) ([current revision 199966](https://serlo-staging.dev/199966))
* [text-solution 198945](https://serlo-staging.dev/198945) ([current revision 200079](https://serlo-staging.dev/200079))
* [text-solution 7279](https://serlo-staging.dev/7279) ([current revision 200374](https://serlo-staging.dev/200374))
* [text-solution 8071](https://serlo-staging.dev/8071) ([current revision 200394](https://serlo-staging.dev/200394))
* [text-solution 9127](https://serlo-staging.dev/9127) ([current revision 200456](https://serlo-staging.dev/200456))
* [text-solution 29825](https://serlo-staging.dev/29825) ([current revision 200733](https://serlo-staging.dev/200733))
* [text-solution 29996](https://serlo-staging.dev/29996) ([current revision 200734](https://serlo-staging.dev/200734))
* [text-solution 39279](https://serlo-staging.dev/39279) ([current revision 200929](https://serlo-staging.dev/200929))
* [text-solution 47253](https://serlo-staging.dev/47253) ([current revision 201529](https://serlo-staging.dev/201529))
* [text-solution 47531](https://serlo-staging.dev/47531) ([current revision 201535](https://serlo-staging.dev/201535))
* [text-solution 66219](https://serlo-staging.dev/66219) ([current revision 201682](https://serlo-staging.dev/201682))
* [text-solution 66175](https://serlo-staging.dev/66175) ([current revision 201685](https://serlo-staging.dev/201685))
* [article 201713](https://serlo-staging.dev/201713) ([current revision 201889](https://serlo-staging.dev/201889))
* [article 186051](https://serlo-staging.dev/186051) ([current revision 202081](https://serlo-staging.dev/202081))
* [article 202629](https://serlo-staging.dev/202629) ([current revision 202632](https://serlo-staging.dev/202632))
* [article 202619](https://serlo-staging.dev/202619) ([current revision 202634](https://serlo-staging.dev/202634))
* [article 203495](https://serlo-staging.dev/203495) ([current revision 203496](https://serlo-staging.dev/203496))
* [course-page 203728](https://serlo-staging.dev/203728) ([current revision 203739](https://serlo-staging.dev/203739))
* [course-page 204723](https://serlo-staging.dev/204723) ([current revision 204733](https://serlo-staging.dev/204733))
* [text-solution 205174](https://serlo-staging.dev/205174) ([current revision 205492](https://serlo-staging.dev/205492))
* [text-solution 205717](https://serlo-staging.dev/205717) ([current revision 205726](https://serlo-staging.dev/205726))
* [text-exercise-group 205809](https://serlo-staging.dev/205809) ([current revision 205821](https://serlo-staging.dev/205821))
* [text-solution 28965](https://serlo-staging.dev/28965) ([current revision 207033](https://serlo-staging.dev/207033))
* [text-solution 204477](https://serlo-staging.dev/204477) ([current revision 207070](https://serlo-staging.dev/207070))
* [text-solution 206269](https://serlo-staging.dev/206269) ([current revision 207158](https://serlo-staging.dev/207158))
* [text-solution 207307](https://serlo-staging.dev/207307) ([current revision 207308](https://serlo-staging.dev/207308))
* [course-page 202939](https://serlo-staging.dev/202939) ([current revision 207330](https://serlo-staging.dev/207330))
* [text-exercise-group 207490](https://serlo-staging.dev/207490) ([current revision 207520](https://serlo-staging.dev/207520))
* [grouped-text-exercise 207687](https://serlo-staging.dev/207687) ([current revision 207694](https://serlo-staging.dev/207694))
* [grouped-text-exercise 207689](https://serlo-staging.dev/207689) ([current revision 207696](https://serlo-staging.dev/207696))
* [grouped-text-exercise 207691](https://serlo-staging.dev/207691) ([current revision 207698](https://serlo-staging.dev/207698))
* [text-exercise-group 207752](https://serlo-staging.dev/207752) ([current revision 207757](https://serlo-staging.dev/207757))
* [course-page 203732](https://serlo-staging.dev/203732) ([current revision 207992](https://serlo-staging.dev/207992))
* [course-page 203724](https://serlo-staging.dev/203724) ([current revision 207998](https://serlo-staging.dev/207998))
* [course-page 207315](https://serlo-staging.dev/207315) ([current revision 208239](https://serlo-staging.dev/208239))
* [text-solution 207492](https://serlo-staging.dev/207492) ([current revision 208303](https://serlo-staging.dev/208303))
* [grouped-text-exercise 13423](https://serlo-staging.dev/13423) ([current revision 208666](https://serlo-staging.dev/208666))
* [text-exercise 9371](https://serlo-staging.dev/9371) ([current revision 208880](https://serlo-staging.dev/208880))
* [text-solution 111460](https://serlo-staging.dev/111460) ([current revision 208899](https://serlo-staging.dev/208899))
* [text-solution 9615](https://serlo-staging.dev/9615) ([current revision 209009](https://serlo-staging.dev/209009))
* [article 208775](https://serlo-staging.dev/208775) ([current revision 209043](https://serlo-staging.dev/209043))
* [text-solution 209068](https://serlo-staging.dev/209068) ([current revision 209074](https://serlo-staging.dev/209074))
* [text-solution 2433](https://serlo-staging.dev/2433) ([current revision 209395](https://serlo-staging.dev/209395))
* [text-solution 209075](https://serlo-staging.dev/209075) ([current revision 209555](https://serlo-staging.dev/209555))
* [text-solution 204225](https://serlo-staging.dev/204225) ([current revision 209723](https://serlo-staging.dev/209723))
* [text-solution 209815](https://serlo-staging.dev/209815) ([current revision 209824](https://serlo-staging.dev/209824))
* [article 203873](https://serlo-staging.dev/203873) ([current revision 209881](https://serlo-staging.dev/209881))
* [article 203879](https://serlo-staging.dev/203879) ([current revision 209882](https://serlo-staging.dev/209882))
* [article 204063](https://serlo-staging.dev/204063) ([current revision 209884](https://serlo-staging.dev/209884))
* [course-page 210084](https://serlo-staging.dev/210084) ([current revision 210090](https://serlo-staging.dev/210090))
* [text-exercise 208256](https://serlo-staging.dev/208256) ([current revision 210305](https://serlo-staging.dev/210305))
* [grouped-text-exercise 210351](https://serlo-staging.dev/210351) ([current revision 210356](https://serlo-staging.dev/210356))
* [text-solution 210555](https://serlo-staging.dev/210555) ([current revision 210560](https://serlo-staging.dev/210560))
* [text-solution 210557](https://serlo-staging.dev/210557) ([current revision 210562](https://serlo-staging.dev/210562))
* [article 55999](https://serlo-staging.dev/55999) ([current revision 210692](https://serlo-staging.dev/210692))
* [article 1947](https://serlo-staging.dev/1947) ([current revision 210693](https://serlo-staging.dev/210693))
* [article 142576](https://serlo-staging.dev/142576) ([current revision 210820](https://serlo-staging.dev/210820))
* [text-solution 210842](https://serlo-staging.dev/210842) ([current revision 210847](https://serlo-staging.dev/210847))
* [article 1679](https://serlo-staging.dev/1679) ([current revision 210897](https://serlo-staging.dev/210897))
* [article 211023](https://serlo-staging.dev/211023) ([current revision 211039](https://serlo-staging.dev/211039))
* [grouped-text-exercise 211133](https://serlo-staging.dev/211133) ([current revision 211142](https://serlo-staging.dev/211142))
* [text-exercise-group 212038](https://serlo-staging.dev/212038) ([current revision 212045](https://serlo-staging.dev/212045))
* [grouped-text-exercise 113662](https://serlo-staging.dev/113662) ([current revision 212101](https://serlo-staging.dev/212101))
* [grouped-text-exercise 113679](https://serlo-staging.dev/113679) ([current revision 212102](https://serlo-staging.dev/212102))
* [article 181051](https://serlo-staging.dev/181051) ([current revision 212149](https://serlo-staging.dev/212149))
* [article 149480](https://serlo-staging.dev/149480) ([current revision 212417](https://serlo-staging.dev/212417))
* [article 212604](https://serlo-staging.dev/212604) ([current revision 212636](https://serlo-staging.dev/212636))
* [text-solution 14455](https://serlo-staging.dev/14455) ([current revision 213017](https://serlo-staging.dev/213017))
* [text-solution 14459](https://serlo-staging.dev/14459) ([current revision 213019](https://serlo-staging.dev/213019))
* [text-solution 213704](https://serlo-staging.dev/213704) ([current revision 213711](https://serlo-staging.dev/213711))
* [text-solution 206418](https://serlo-staging.dev/206418) ([current revision 213786](https://serlo-staging.dev/213786))
* [text-exercise-group 213922](https://serlo-staging.dev/213922) ([current revision 213934](https://serlo-staging.dev/213934))
* [text-solution 214007](https://serlo-staging.dev/214007) ([current revision 214014](https://serlo-staging.dev/214014))
* [text-solution 213460](https://serlo-staging.dev/213460) ([current revision 214064](https://serlo-staging.dev/214064))
* [text-solution 213462](https://serlo-staging.dev/213462) ([current revision 214065](https://serlo-staging.dev/214065))
* [text-solution 213466](https://serlo-staging.dev/213466) ([current revision 214067](https://serlo-staging.dev/214067))
* [grouped-text-exercise 214082](https://serlo-staging.dev/214082) ([current revision 214087](https://serlo-staging.dev/214087))
* [article 214036](https://serlo-staging.dev/214036) ([current revision 214095](https://serlo-staging.dev/214095))
* [article 214104](https://serlo-staging.dev/214104) ([current revision 214125](https://serlo-staging.dev/214125))
* [text-solution 214139](https://serlo-staging.dev/214139) ([current revision 214146](https://serlo-staging.dev/214146))
* [text-solution 214141](https://serlo-staging.dev/214141) ([current revision 214148](https://serlo-staging.dev/214148))
* [text-solution 213528](https://serlo-staging.dev/213528) ([current revision 214263](https://serlo-staging.dev/214263))
* [text-solution 213532](https://serlo-staging.dev/213532) ([current revision 214267](https://serlo-staging.dev/214267))
* [text-solution 206689](https://serlo-staging.dev/206689) ([current revision 214386](https://serlo-staging.dev/214386))
* [text-exercise-group 213883](https://serlo-staging.dev/213883) ([current revision 214397](https://serlo-staging.dev/214397))
* [text-solution 214771](https://serlo-staging.dev/214771) ([current revision 214877](https://serlo-staging.dev/214877))
* [text-exercise-group 206726](https://serlo-staging.dev/206726) ([current revision 214989](https://serlo-staging.dev/214989))
* [text-solution 207453](https://serlo-staging.dev/207453) ([current revision 215209](https://serlo-staging.dev/215209))
* [text-solution 207603](https://serlo-staging.dev/207603) ([current revision 215321](https://serlo-staging.dev/215321))
* [text-solution 215103](https://serlo-staging.dev/215103) ([current revision 215630](https://serlo-staging.dev/215630))
* [text-solution 212008](https://serlo-staging.dev/212008) ([current revision 215887](https://serlo-staging.dev/215887))
* [text-exercise 216307](https://serlo-staging.dev/216307) ([current revision 216310](https://serlo-staging.dev/216310))
* [text-solution 216309](https://serlo-staging.dev/216309) ([current revision 216311](https://serlo-staging.dev/216311))
* [text-solution 207754](https://serlo-staging.dev/207754) ([current revision 216328](https://serlo-staging.dev/216328))
* [text-solution 216623](https://serlo-staging.dev/216623) ([current revision 216630](https://serlo-staging.dev/216630))
* [text-exercise-group 217225](https://serlo-staging.dev/217225) ([current revision 217250](https://serlo-staging.dev/217250))
* [article 216891](https://serlo-staging.dev/216891) ([current revision 217721](https://serlo-staging.dev/217721))
* [text-solution 216774](https://serlo-staging.dev/216774) ([current revision 217834](https://serlo-staging.dev/217834))
* [text-solution 216822](https://serlo-staging.dev/216822) ([current revision 217860](https://serlo-staging.dev/217860))
* [grouped-text-exercise 217947](https://serlo-staging.dev/217947) ([current revision 217954](https://serlo-staging.dev/217954))
* [grouped-text-exercise 218350](https://serlo-staging.dev/218350) ([current revision 218354](https://serlo-staging.dev/218354))
* [text-solution 210136](https://serlo-staging.dev/210136) ([current revision 218521](https://serlo-staging.dev/218521))
* [text-solution 217546](https://serlo-staging.dev/217546) ([current revision 218640](https://serlo-staging.dev/218640))
* [text-solution 217626](https://serlo-staging.dev/217626) ([current revision 218642](https://serlo-staging.dev/218642))
* [text-solution 217628](https://serlo-staging.dev/217628) ([current revision 218643](https://serlo-staging.dev/218643))
* [text-solution 217636](https://serlo-staging.dev/217636) ([current revision 218644](https://serlo-staging.dev/218644))
* [text-solution 217638](https://serlo-staging.dev/217638) ([current revision 218645](https://serlo-staging.dev/218645))
* [text-solution 217700](https://serlo-staging.dev/217700) ([current revision 218687](https://serlo-staging.dev/218687))
* [text-solution 217845](https://serlo-staging.dev/217845) ([current revision 218691](https://serlo-staging.dev/218691))
* [text-solution 217849](https://serlo-staging.dev/217849) ([current revision 218693](https://serlo-staging.dev/218693))
* [text-solution 217866](https://serlo-staging.dev/217866) ([current revision 218694](https://serlo-staging.dev/218694))
* [text-solution 217907](https://serlo-staging.dev/217907) ([current revision 218706](https://serlo-staging.dev/218706))
* [text-solution 217952](https://serlo-staging.dev/217952) ([current revision 218724](https://serlo-staging.dev/218724))
* [text-solution 218081](https://serlo-staging.dev/218081) ([current revision 218766](https://serlo-staging.dev/218766))
* [text-solution 218083](https://serlo-staging.dev/218083) ([current revision 218767](https://serlo-staging.dev/218767))
* [text-solution 218085](https://serlo-staging.dev/218085) ([current revision 218769](https://serlo-staging.dev/218769))
* [text-solution 218087](https://serlo-staging.dev/218087) ([current revision 218770](https://serlo-staging.dev/218770))
* [text-solution 218456](https://serlo-staging.dev/218456) ([current revision 218948](https://serlo-staging.dev/218948))
* [text-solution 218458](https://serlo-staging.dev/218458) ([current revision 218949](https://serlo-staging.dev/218949))
* [text-solution 218533](https://serlo-staging.dev/218533) ([current revision 218951](https://serlo-staging.dev/218951))
* [text-solution 218584](https://serlo-staging.dev/218584) ([current revision 218965](https://serlo-staging.dev/218965))
* [text-solution 211227](https://serlo-staging.dev/211227) ([current revision 219057](https://serlo-staging.dev/219057))
* [text-solution 219241](https://serlo-staging.dev/219241) ([current revision 219250](https://serlo-staging.dev/219250))
* [article 219518](https://serlo-staging.dev/219518) ([current revision 219519](https://serlo-staging.dev/219519))
* [article 219587](https://serlo-staging.dev/219587) ([current revision 219597](https://serlo-staging.dev/219597))
* [text-solution 213692](https://serlo-staging.dev/213692) ([current revision 219628](https://serlo-staging.dev/219628))
* [text-solution 220099](https://serlo-staging.dev/220099) ([current revision 220114](https://serlo-staging.dev/220114))
* [text-exercise-group 207977](https://serlo-staging.dev/207977) ([current revision 220280](https://serlo-staging.dev/220280))
* [text-solution 207983](https://serlo-staging.dev/207983) ([current revision 220281](https://serlo-staging.dev/220281))
* [text-solution 220101](https://serlo-staging.dev/220101) ([current revision 220285](https://serlo-staging.dev/220285))
* [text-solution 220103](https://serlo-staging.dev/220103) ([current revision 220287](https://serlo-staging.dev/220287))
* [text-solution 220105](https://serlo-staging.dev/220105) ([current revision 220289](https://serlo-staging.dev/220289))
* [text-solution 220374](https://serlo-staging.dev/220374) ([current revision 220481](https://serlo-staging.dev/220481))
* [text-exercise 220435](https://serlo-staging.dev/220435) ([current revision 220501](https://serlo-staging.dev/220501))
* [text-solution 220107](https://serlo-staging.dev/220107) ([current revision 220706](https://serlo-staging.dev/220706))
* [text-solution 220109](https://serlo-staging.dev/220109) ([current revision 220708](https://serlo-staging.dev/220708))
* [text-exercise 29860](https://serlo-staging.dev/29860) ([current revision 220728](https://serlo-staging.dev/220728))
* [text-solution 220111](https://serlo-staging.dev/220111) ([current revision 220908](https://serlo-staging.dev/220908))
* [text-solution 213959](https://serlo-staging.dev/213959) ([current revision 221104](https://serlo-staging.dev/221104))
* [text-solution 213961](https://serlo-staging.dev/213961) ([current revision 221105](https://serlo-staging.dev/221105))
* [text-solution 221193](https://serlo-staging.dev/221193) ([current revision 221195](https://serlo-staging.dev/221195))
* [article 204811](https://serlo-staging.dev/204811) ([current revision 221529](https://serlo-staging.dev/221529))
* [text-solution 214740](https://serlo-staging.dev/214740) ([current revision 222040](https://serlo-staging.dev/222040))
* [article 166757](https://serlo-staging.dev/166757) ([current revision 222326](https://serlo-staging.dev/222326))
* [article 222052](https://serlo-staging.dev/222052) ([current revision 222720](https://serlo-staging.dev/222720))
* [article 222470](https://serlo-staging.dev/222470) ([current revision 222734](https://serlo-staging.dev/222734))
* [article 222049](https://serlo-staging.dev/222049) ([current revision 222739](https://serlo-staging.dev/222739))
* [article 90414](https://serlo-staging.dev/90414) ([current revision 222745](https://serlo-staging.dev/222745))
* [article 181038](https://serlo-staging.dev/181038) ([current revision 222779](https://serlo-staging.dev/222779))
* [article 28296](https://serlo-staging.dev/28296) ([current revision 222786](https://serlo-staging.dev/222786))
* [article 90412](https://serlo-staging.dev/90412) ([current revision 223073](https://serlo-staging.dev/223073))
* [article 1693](https://serlo-staging.dev/1693) ([current revision 223403](https://serlo-staging.dev/223403))
* [text-exercise-group 206727](https://serlo-staging.dev/206727) ([current revision 223549](https://serlo-staging.dev/223549))
* [text-solution 215858](https://serlo-staging.dev/215858) ([current revision 223704](https://serlo-staging.dev/223704))
* [text-solution 223754](https://serlo-staging.dev/223754) ([current revision 224018](https://serlo-staging.dev/224018))
* [article 77998](https://serlo-staging.dev/77998) ([current revision 224237](https://serlo-staging.dev/224237))
* [article 1563](https://serlo-staging.dev/1563) ([current revision 224875](https://serlo-staging.dev/224875))
* [article 1593](https://serlo-staging.dev/1593) ([current revision 225972](https://serlo-staging.dev/225972))
* [article 1603](https://serlo-staging.dev/1603) ([current revision 226180](https://serlo-staging.dev/226180))
* [article 1599](https://serlo-staging.dev/1599) ([current revision 226482](https://serlo-staging.dev/226482))
* [article 171393](https://serlo-staging.dev/171393) ([current revision 226806](https://serlo-staging.dev/226806))
* [article 215581](https://serlo-staging.dev/215581) ([current revision 226808](https://serlo-staging.dev/226808))
* [article 226897](https://serlo-staging.dev/226897) ([current revision 226914](https://serlo-staging.dev/226914))
* [article 2085](https://serlo-staging.dev/2085) ([current revision 227292](https://serlo-staging.dev/227292))
* [article 134087](https://serlo-staging.dev/134087) ([current revision 227295](https://serlo-staging.dev/227295))
* [text-solution 2791](https://serlo-staging.dev/2791) ([current revision 227355](https://serlo-staging.dev/227355))
* [article 227568](https://serlo-staging.dev/227568) ([current revision 227574](https://serlo-staging.dev/227574))
* [text-solution 2779](https://serlo-staging.dev/2779) ([current revision 227621](https://serlo-staging.dev/227621))
* [text-solution 2775](https://serlo-staging.dev/2775) ([current revision 227647](https://serlo-staging.dev/227647))
* [text-solution 2795](https://serlo-staging.dev/2795) ([current revision 227649](https://serlo-staging.dev/227649))
* [text-exercise 227668](https://serlo-staging.dev/227668) ([current revision 227970](https://serlo-staging.dev/227970))
* [article 1627](https://serlo-staging.dev/1627) ([current revision 228033](https://serlo-staging.dev/228033))
* [text-exercise 228056](https://serlo-staging.dev/228056) ([current revision 228057](https://serlo-staging.dev/228057))
* [text-solution 227715](https://serlo-staging.dev/227715) ([current revision 228116](https://serlo-staging.dev/228116))
* [article 226208](https://serlo-staging.dev/226208) ([current revision 228252](https://serlo-staging.dev/228252))
* [article 226236](https://serlo-staging.dev/226236) ([current revision 228253](https://serlo-staging.dev/228253))
* [article 228337](https://serlo-staging.dev/228337) ([current revision 228338](https://serlo-staging.dev/228338))
* [article 228339](https://serlo-staging.dev/228339) ([current revision 228341](https://serlo-staging.dev/228341))
* [article 210900](https://serlo-staging.dev/210900) ([current revision 228614](https://serlo-staging.dev/228614))
* [article 1659](https://serlo-staging.dev/1659) ([current revision 229056](https://serlo-staging.dev/229056))
* [article 131323](https://serlo-staging.dev/131323) ([current revision 229154](https://serlo-staging.dev/229154))
* [article 227342](https://serlo-staging.dev/227342) ([current revision 229427](https://serlo-staging.dev/229427))
* [article 1673](https://serlo-staging.dev/1673) ([current revision 229559](https://serlo-staging.dev/229559))
* [article 1675](https://serlo-staging.dev/1675) ([current revision 229560](https://serlo-staging.dev/229560))
* [article 225689](https://serlo-staging.dev/225689) ([current revision 229627](https://serlo-staging.dev/229627))
* [text-exercise 9173](https://serlo-staging.dev/9173) ([current revision 230144](https://serlo-staging.dev/230144))
* [article 206397](https://serlo-staging.dev/206397) ([current revision 230258](https://serlo-staging.dev/230258))
* [grouped-text-exercise 230499](https://serlo-staging.dev/230499) ([current revision 230504](https://serlo-staging.dev/230504))
* [grouped-text-exercise 230501](https://serlo-staging.dev/230501) ([current revision 230506](https://serlo-staging.dev/230506))
* [article 1691](https://serlo-staging.dev/1691) ([current revision 230874](https://serlo-staging.dev/230874))
* [article 231195](https://serlo-staging.dev/231195) ([current revision 231224](https://serlo-staging.dev/231224))
* [text-exercise 231322](https://serlo-staging.dev/231322) ([current revision 231325](https://serlo-staging.dev/231325))
* [text-solution 231324](https://serlo-staging.dev/231324) ([current revision 231326](https://serlo-staging.dev/231326))
* [article 129133](https://serlo-staging.dev/129133) ([current revision 231331](https://serlo-staging.dev/231331))
* [article 199259](https://serlo-staging.dev/199259) ([current revision 231337](https://serlo-staging.dev/231337))
* [article 199669](https://serlo-staging.dev/199669) ([current revision 231340](https://serlo-staging.dev/231340))
* [article 199564](https://serlo-staging.dev/199564) ([current revision 231341](https://serlo-staging.dev/231341))
* [article 199808](https://serlo-staging.dev/199808) ([current revision 231382](https://serlo-staging.dev/231382))
* [article 200341](https://serlo-staging.dev/200341) ([current revision 231385](https://serlo-staging.dev/231385))
* [article 200617](https://serlo-staging.dev/200617) ([current revision 231388](https://serlo-staging.dev/231388))
* [article 1701](https://serlo-staging.dev/1701) ([current revision 231468](https://serlo-staging.dev/231468))
* [article 200702](https://serlo-staging.dev/200702) ([current revision 231747](https://serlo-staging.dev/231747))
* [article 232092](https://serlo-staging.dev/232092) ([current revision 232093](https://serlo-staging.dev/232093))
* [article 201061](https://serlo-staging.dev/201061) ([current revision 232105](https://serlo-staging.dev/232105))
* [article 201255](https://serlo-staging.dev/201255) ([current revision 232106](https://serlo-staging.dev/232106))
* [article 228142](https://serlo-staging.dev/228142) ([current revision 232139](https://serlo-staging.dev/232139))
* [article 228147](https://serlo-staging.dev/228147) ([current revision 232156](https://serlo-staging.dev/232156))
* [text-exercise 231883](https://serlo-staging.dev/231883) ([current revision 232411](https://serlo-staging.dev/232411))
* [text-solution 231887](https://serlo-staging.dev/231887) ([current revision 232412](https://serlo-staging.dev/232412))
* [text-exercise 232452](https://serlo-staging.dev/232452) ([current revision 232454](https://serlo-staging.dev/232454))
* [text-solution 232453](https://serlo-staging.dev/232453) ([current revision 232455](https://serlo-staging.dev/232455))
* [text-solution 8443](https://serlo-staging.dev/8443) ([current revision 232503](https://serlo-staging.dev/232503))
* [text-solution 232569](https://serlo-staging.dev/232569) ([current revision 232582](https://serlo-staging.dev/232582))
* [text-solution 232571](https://serlo-staging.dev/232571) ([current revision 232584](https://serlo-staging.dev/232584))
* [text-solution 232573](https://serlo-staging.dev/232573) ([current revision 232586](https://serlo-staging.dev/232586))
* [text-solution 232575](https://serlo-staging.dev/232575) ([current revision 232588](https://serlo-staging.dev/232588))
* [text-solution 232577](https://serlo-staging.dev/232577) ([current revision 232590](https://serlo-staging.dev/232590))
* [text-solution 232579](https://serlo-staging.dev/232579) ([current revision 232592](https://serlo-staging.dev/232592))
* [article 1685](https://serlo-staging.dev/1685) ([current revision 232679](https://serlo-staging.dev/232679))
* [text-solution 231398](https://serlo-staging.dev/231398) ([current revision 232684](https://serlo-staging.dev/232684))
* [text-solution 231405](https://serlo-staging.dev/231405) ([current revision 232699](https://serlo-staging.dev/232699))
* [text-solution 231514](https://serlo-staging.dev/231514) ([current revision 232701](https://serlo-staging.dev/232701))
* [text-solution 231516](https://serlo-staging.dev/231516) ([current revision 232703](https://serlo-staging.dev/232703))
* [text-solution 231518](https://serlo-staging.dev/231518) ([current revision 232705](https://serlo-staging.dev/232705))
* [article 168879](https://serlo-staging.dev/168879) ([current revision 232717](https://serlo-staging.dev/232717))
* [article 160845](https://serlo-staging.dev/160845) ([current revision 232752](https://serlo-staging.dev/232752))
* [article 167678](https://serlo-staging.dev/167678) ([current revision 232899](https://serlo-staging.dev/232899))
* [article 1689](https://serlo-staging.dev/1689) ([current revision 232918](https://serlo-staging.dev/232918))
* [text-exercise 232709](https://serlo-staging.dev/232709) ([current revision 233028](https://serlo-staging.dev/233028))
* [text-solution 232720](https://serlo-staging.dev/232720) ([current revision 233119](https://serlo-staging.dev/233119))
* [text-solution 228741](https://serlo-staging.dev/228741) ([current revision 233195](https://serlo-staging.dev/233195))
* [text-solution 233250](https://serlo-staging.dev/233250) ([current revision 233261](https://serlo-staging.dev/233261))
* [text-solution 233252](https://serlo-staging.dev/233252) ([current revision 233263](https://serlo-staging.dev/233263))
* [text-solution 233254](https://serlo-staging.dev/233254) ([current revision 233265](https://serlo-staging.dev/233265))
* [text-solution 233256](https://serlo-staging.dev/233256) ([current revision 233267](https://serlo-staging.dev/233267))
* [text-solution 233258](https://serlo-staging.dev/233258) ([current revision 233269](https://serlo-staging.dev/233269))
* [text-solution 233273](https://serlo-staging.dev/233273) ([current revision 233284](https://serlo-staging.dev/233284))
* [text-solution 233275](https://serlo-staging.dev/233275) ([current revision 233286](https://serlo-staging.dev/233286))
* [text-solution 233277](https://serlo-staging.dev/233277) ([current revision 233288](https://serlo-staging.dev/233288))
* [text-solution 233279](https://serlo-staging.dev/233279) ([current revision 233290](https://serlo-staging.dev/233290))
* [text-solution 233281](https://serlo-staging.dev/233281) ([current revision 233292](https://serlo-staging.dev/233292))
* [text-solution 9651](https://serlo-staging.dev/9651) ([current revision 233645](https://serlo-staging.dev/233645))
* [article 167791](https://serlo-staging.dev/167791) ([current revision 234024](https://serlo-staging.dev/234024))
* [article 228149](https://serlo-staging.dev/228149) ([current revision 234111](https://serlo-staging.dev/234111))
* [article 228138](https://serlo-staging.dev/228138) ([current revision 234112](https://serlo-staging.dev/234112))
* [article 41295](https://serlo-staging.dev/41295) ([current revision 234126](https://serlo-staging.dev/234126))
* [course-page 137980](https://serlo-staging.dev/137980) ([current revision 234128](https://serlo-staging.dev/234128))
* [text-solution 234131](https://serlo-staging.dev/234131) ([current revision 234133](https://serlo-staging.dev/234133))
* [article 230335](https://serlo-staging.dev/230335) ([current revision 234165](https://serlo-staging.dev/234165))
* [course-page 233720](https://serlo-staging.dev/233720) ([current revision 234233](https://serlo-staging.dev/234233))
* [course-page 233719](https://serlo-staging.dev/233719) ([current revision 234283](https://serlo-staging.dev/234283))
* [course-page 233722](https://serlo-staging.dev/233722) ([current revision 234284](https://serlo-staging.dev/234284))
* [course-page 233724](https://serlo-staging.dev/233724) ([current revision 234286](https://serlo-staging.dev/234286))
* [course-page 129909](https://serlo-staging.dev/129909) ([current revision 234329](https://serlo-staging.dev/234329))
* [article 1751](https://serlo-staging.dev/1751) ([current revision 234604](https://serlo-staging.dev/234604))
* [article 170318](https://serlo-staging.dev/170318) ([current revision 234616](https://serlo-staging.dev/234616))
* [text-solution 230719](https://serlo-staging.dev/230719) ([current revision 234905](https://serlo-staging.dev/234905))
* [article 234395](https://serlo-staging.dev/234395) ([current revision 235404](https://serlo-staging.dev/235404))
* [article 235561](https://serlo-staging.dev/235561) ([current revision 235641](https://serlo-staging.dev/235641))
* [text-solution 235921](https://serlo-staging.dev/235921) ([current revision 236439](https://serlo-staging.dev/236439))
* [course-page 232638](https://serlo-staging.dev/232638) ([current revision 236615](https://serlo-staging.dev/236615))
* [course-page 232639](https://serlo-staging.dev/232639) ([current revision 236616](https://serlo-staging.dev/236616))
* [course-page 232640](https://serlo-staging.dev/232640) ([current revision 236617](https://serlo-staging.dev/236617))
* [course-page 232641](https://serlo-staging.dev/232641) ([current revision 236618](https://serlo-staging.dev/236618))
* [course-page 232642](https://serlo-staging.dev/232642) ([current revision 236619](https://serlo-staging.dev/236619))
* [course-page 232643](https://serlo-staging.dev/232643) ([current revision 236620](https://serlo-staging.dev/236620))
* [course-page 232644](https://serlo-staging.dev/232644) ([current revision 236621](https://serlo-staging.dev/236621))
* [course-page 232645](https://serlo-staging.dev/232645) ([current revision 236622](https://serlo-staging.dev/236622))
* [course-page 232646](https://serlo-staging.dev/232646) ([current revision 236623](https://serlo-staging.dev/236623))
* [course-page 232647](https://serlo-staging.dev/232647) ([current revision 236624](https://serlo-staging.dev/236624))
* [article 168159](https://serlo-staging.dev/168159) ([current revision 236725](https://serlo-staging.dev/236725))
* [article 236956](https://serlo-staging.dev/236956) ([current revision 237018](https://serlo-staging.dev/237018))
* [text-solution 225660](https://serlo-staging.dev/225660) ([current revision 237300](https://serlo-staging.dev/237300))
* [article 1703](https://serlo-staging.dev/1703) ([current revision 237387](https://serlo-staging.dev/237387))
* [course-page 237410](https://serlo-staging.dev/237410) ([current revision 237455](https://serlo-staging.dev/237455))
* [course-page 237469](https://serlo-staging.dev/237469) ([current revision 237473](https://serlo-staging.dev/237473))
* [article 141488](https://serlo-staging.dev/141488) ([current revision 237765](https://serlo-staging.dev/237765))
* [article 201374](https://serlo-staging.dev/201374) ([current revision 237769](https://serlo-staging.dev/237769))
* [article 201451](https://serlo-staging.dev/201451) ([current revision 237770](https://serlo-staging.dev/237770))
* [text-solution 235247](https://serlo-staging.dev/235247) ([current revision 237884](https://serlo-staging.dev/237884))
* [text-solution 237970](https://serlo-staging.dev/237970) ([current revision 237971](https://serlo-staging.dev/237971))
* [course-page 237487](https://serlo-staging.dev/237487) ([current revision 238022](https://serlo-staging.dev/238022))
* [article 202149](https://serlo-staging.dev/202149) ([current revision 238024](https://serlo-staging.dev/238024))
* [article 202308](https://serlo-staging.dev/202308) ([current revision 238030](https://serlo-staging.dev/238030))
* [course-page 238106](https://serlo-staging.dev/238106) ([current revision 238135](https://serlo-staging.dev/238135))
* [article 202501](https://serlo-staging.dev/202501) ([current revision 238331](https://serlo-staging.dev/238331))
* [article 202599](https://serlo-staging.dev/202599) ([current revision 238332](https://serlo-staging.dev/238332))
* [article 70650](https://serlo-staging.dev/70650) ([current revision 238404](https://serlo-staging.dev/238404))
* [article 185618](https://serlo-staging.dev/185618) ([current revision 238624](https://serlo-staging.dev/238624))
* [article 196626](https://serlo-staging.dev/196626) ([current revision 238627](https://serlo-staging.dev/238627))
* [article 212986](https://serlo-staging.dev/212986) ([current revision 238628](https://serlo-staging.dev/238628))
* [article 213826](https://serlo-staging.dev/213826) ([current revision 238629](https://serlo-staging.dev/238629))
* [article 215223](https://serlo-staging.dev/215223) ([current revision 238630](https://serlo-staging.dev/238630))
* [article 192279](https://serlo-staging.dev/192279) ([current revision 238632](https://serlo-staging.dev/238632))
* [text-solution 10615](https://serlo-staging.dev/10615) ([current revision 238742](https://serlo-staging.dev/238742))
* [article 134076](https://serlo-staging.dev/134076) ([current revision 238948](https://serlo-staging.dev/238948))
* [article 134118](https://serlo-staging.dev/134118) ([current revision 238964](https://serlo-staging.dev/238964))
* [text-solution 238659](https://serlo-staging.dev/238659) ([current revision 239028](https://serlo-staging.dev/239028))
* [article 239121](https://serlo-staging.dev/239121) ([current revision 239126](https://serlo-staging.dev/239126))
* [article 2215](https://serlo-staging.dev/2215) ([current revision 239201](https://serlo-staging.dev/239201))
* [article 197406](https://serlo-staging.dev/197406) ([current revision 239290](https://serlo-staging.dev/239290))
* [article 239214](https://serlo-staging.dev/239214) ([current revision 239316](https://serlo-staging.dev/239316))
* [course-page 223935](https://serlo-staging.dev/223935) ([current revision 239781](https://serlo-staging.dev/239781))
* [course-page 223941](https://serlo-staging.dev/223941) ([current revision 239793](https://serlo-staging.dev/239793))
* [course-page 223940](https://serlo-staging.dev/223940) ([current revision 239880](https://serlo-staging.dev/239880))
* [article 113289](https://serlo-staging.dev/113289) ([current revision 239977](https://serlo-staging.dev/239977))
* [text-solution 239982](https://serlo-staging.dev/239982) ([current revision 239986](https://serlo-staging.dev/239986))
* [article 240057](https://serlo-staging.dev/240057) ([current revision 240059](https://serlo-staging.dev/240059))
* [text-solution 235825](https://serlo-staging.dev/235825) ([current revision 240062](https://serlo-staging.dev/240062))
* [text-solution 11853](https://serlo-staging.dev/11853) ([current revision 240067](https://serlo-staging.dev/240067))
* [text-solution 235850](https://serlo-staging.dev/235850) ([current revision 240070](https://serlo-staging.dev/240070))
* [text-exercise 235876](https://serlo-staging.dev/235876) ([current revision 240083](https://serlo-staging.dev/240083))
* [text-exercise-group 11209](https://serlo-staging.dev/11209) ([current revision 240192](https://serlo-staging.dev/240192))
* [text-solution 29255](https://serlo-staging.dev/29255) ([current revision 240193](https://serlo-staging.dev/240193))
* [text-solution 240322](https://serlo-staging.dev/240322) ([current revision 240326](https://serlo-staging.dev/240326))
* [text-solution 240324](https://serlo-staging.dev/240324) ([current revision 240331](https://serlo-staging.dev/240331))
* [article 240300](https://serlo-staging.dev/240300) ([current revision 240349](https://serlo-staging.dev/240349))
* [article 1795](https://serlo-staging.dev/1795) ([current revision 240412](https://serlo-staging.dev/240412))
* [article 199799](https://serlo-staging.dev/199799) ([current revision 240436](https://serlo-staging.dev/240436))
* [article 240484](https://serlo-staging.dev/240484) ([current revision 240587](https://serlo-staging.dev/240587))
* [article 1683](https://serlo-staging.dev/1683) ([current revision 240597](https://serlo-staging.dev/240597))
* [article 1843](https://serlo-staging.dev/1843) ([current revision 240700](https://serlo-staging.dev/240700))
* [text-solution 240532](https://serlo-staging.dev/240532) ([current revision 240756](https://serlo-staging.dev/240756))
* [text-solution 240446](https://serlo-staging.dev/240446) ([current revision 240759](https://serlo-staging.dev/240759))
* [text-solution 240448](https://serlo-staging.dev/240448) ([current revision 240761](https://serlo-staging.dev/240761))
* [article 1861](https://serlo-staging.dev/1861) ([current revision 240765](https://serlo-staging.dev/240765))
* [text-solution 240469](https://serlo-staging.dev/240469) ([current revision 240773](https://serlo-staging.dev/240773))
* [text-exercise 240796](https://serlo-staging.dev/240796) ([current revision 240799](https://serlo-staging.dev/240799))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [text-solution 240530](https://serlo-staging.dev/240530) ([current revision 240882](https://serlo-staging.dev/240882))
* [article 2213](https://serlo-staging.dev/2213) ([current revision 240899](https://serlo-staging.dev/240899))
* [article 1963](https://serlo-staging.dev/1963) ([current revision 241026](https://serlo-staging.dev/241026))
* [article 1789](https://serlo-staging.dev/1789) ([current revision 241072](https://serlo-staging.dev/241072))
* [text-solution 240471](https://serlo-staging.dev/240471) ([current revision 241073](https://serlo-staging.dev/241073))
* [text-solution 240534](https://serlo-staging.dev/240534) ([current revision 241074](https://serlo-staging.dev/241074))
* [article 1893](https://serlo-staging.dev/1893) ([current revision 241093](https://serlo-staging.dev/241093))
* [article 1903](https://serlo-staging.dev/1903) ([current revision 241096](https://serlo-staging.dev/241096))
* [article 1897](https://serlo-staging.dev/1897) ([current revision 241098](https://serlo-staging.dev/241098))
* [article 1899](https://serlo-staging.dev/1899) ([current revision 241099](https://serlo-staging.dev/241099))
* [article 1905](https://serlo-staging.dev/1905) ([current revision 241105](https://serlo-staging.dev/241105))
* [article 1911](https://serlo-staging.dev/1911) ([current revision 241109](https://serlo-staging.dev/241109))
* [article 1613](https://serlo-staging.dev/1613) ([current revision 241139](https://serlo-staging.dev/241139))
* [text-solution 13263](https://serlo-staging.dev/13263) ([current revision 241170](https://serlo-staging.dev/241170))
* [text-solution 2769](https://serlo-staging.dev/2769) ([current revision 241188](https://serlo-staging.dev/241188))
* [article 241186](https://serlo-staging.dev/241186) ([current revision 241191](https://serlo-staging.dev/241191))
* [article 99597](https://serlo-staging.dev/99597) ([current revision 241439](https://serlo-staging.dev/241439))
* [article 2047](https://serlo-staging.dev/2047) ([current revision 241480](https://serlo-staging.dev/241480))
* [article 1571](https://serlo-staging.dev/1571) ([current revision 241555](https://serlo-staging.dev/241555))
* [article 2065](https://serlo-staging.dev/2065) ([current revision 241582](https://serlo-staging.dev/241582))
* [article 2063](https://serlo-staging.dev/2063) ([current revision 241583](https://serlo-staging.dev/241583))
* [article 2069](https://serlo-staging.dev/2069) ([current revision 241587](https://serlo-staging.dev/241587))
* [article 2041](https://serlo-staging.dev/2041) ([current revision 241681](https://serlo-staging.dev/241681))
* [article 97552](https://serlo-staging.dev/97552) ([current revision 241810](https://serlo-staging.dev/241810))
* [article 2083](https://serlo-staging.dev/2083) ([current revision 241815](https://serlo-staging.dev/241815))
* [article 2111](https://serlo-staging.dev/2111) ([current revision 241861](https://serlo-staging.dev/241861))
* [article 2121](https://serlo-staging.dev/2121) ([current revision 241866](https://serlo-staging.dev/241866))
* [article 2161](https://serlo-staging.dev/2161) ([current revision 241894](https://serlo-staging.dev/241894))
* [article 1867](https://serlo-staging.dev/1867) ([current revision 241909](https://serlo-staging.dev/241909))
* [article 170599](https://serlo-staging.dev/170599) ([current revision 241947](https://serlo-staging.dev/241947))
* [article 23953](https://serlo-staging.dev/23953) ([current revision 241979](https://serlo-staging.dev/241979))
* [article 25545](https://serlo-staging.dev/25545) ([current revision 242006](https://serlo-staging.dev/242006))
* [article 23996](https://serlo-staging.dev/23996) ([current revision 242009](https://serlo-staging.dev/242009))
* [article 24013](https://serlo-staging.dev/24013) ([current revision 242052](https://serlo-staging.dev/242052))
* [text-solution 235941](https://serlo-staging.dev/235941) ([current revision 242100](https://serlo-staging.dev/242100))
* [grouped-text-exercise 235986](https://serlo-staging.dev/235986) ([current revision 242105](https://serlo-staging.dev/242105))
* [text-solution 235987](https://serlo-staging.dev/235987) ([current revision 242106](https://serlo-staging.dev/242106))
* [article 27801](https://serlo-staging.dev/27801) ([current revision 242132](https://serlo-staging.dev/242132))
* [article 239345](https://serlo-staging.dev/239345) ([current revision 242157](https://serlo-staging.dev/242157))
* [article 2165](https://serlo-staging.dev/2165) ([current revision 242179](https://serlo-staging.dev/242179))
* [article 2091](https://serlo-staging.dev/2091) ([current revision 242182](https://serlo-staging.dev/242182))
* [article 2187](https://serlo-staging.dev/2187) ([current revision 242244](https://serlo-staging.dev/242244))
* [article 26953](https://serlo-staging.dev/26953) ([current revision 242280](https://serlo-staging.dev/242280))
* [article 2203](https://serlo-staging.dev/2203) ([current revision 242293](https://serlo-staging.dev/242293))
* [article 27365](https://serlo-staging.dev/27365) ([current revision 242336](https://serlo-staging.dev/242336))
* [article 27368](https://serlo-staging.dev/27368) ([current revision 242339](https://serlo-staging.dev/242339))
* [article 33888](https://serlo-staging.dev/33888) ([current revision 242350](https://serlo-staging.dev/242350))
* [article 242309](https://serlo-staging.dev/242309) ([current revision 242359](https://serlo-staging.dev/242359))
* [article 27276](https://serlo-staging.dev/27276) ([current revision 242384](https://serlo-staging.dev/242384))
* [article 24027](https://serlo-staging.dev/24027) ([current revision 242392](https://serlo-staging.dev/242392))
* [article 27443](https://serlo-staging.dev/27443) ([current revision 242419](https://serlo-staging.dev/242419))
* [article 27521](https://serlo-staging.dev/27521) ([current revision 242450](https://serlo-staging.dev/242450))
* [article 27536](https://serlo-staging.dev/27536) ([current revision 242451](https://serlo-staging.dev/242451))
* [grouped-text-exercise 236027](https://serlo-staging.dev/236027) ([current revision 242510](https://serlo-staging.dev/242510))
* [text-solution 236028](https://serlo-staging.dev/236028) ([current revision 242511](https://serlo-staging.dev/242511))
* [article 29291](https://serlo-staging.dev/29291) ([current revision 242536](https://serlo-staging.dev/242536))
* [article 29577](https://serlo-staging.dev/29577) ([current revision 242538](https://serlo-staging.dev/242538))
* [text-exercise 37826](https://serlo-staging.dev/37826) ([current revision 242592](https://serlo-staging.dev/242592))
* [article 33921](https://serlo-staging.dev/33921) ([current revision 242609](https://serlo-staging.dev/242609))
* [article 35621](https://serlo-staging.dev/35621) ([current revision 242691](https://serlo-staging.dev/242691))
* [article 35663](https://serlo-staging.dev/35663) ([current revision 242692](https://serlo-staging.dev/242692))
* [article 35686](https://serlo-staging.dev/35686) ([current revision 242693](https://serlo-staging.dev/242693))
* [text-exercise-group 29921](https://serlo-staging.dev/29921) ([current revision 242777](https://serlo-staging.dev/242777))
* [article 40346](https://serlo-staging.dev/40346) ([current revision 242883](https://serlo-staging.dev/242883))
* [article 241104](https://serlo-staging.dev/241104) ([current revision 242899](https://serlo-staging.dev/242899))
* [article 240050](https://serlo-staging.dev/240050) ([current revision 242910](https://serlo-staging.dev/242910))
* [article 46959](https://serlo-staging.dev/46959) ([current revision 242945](https://serlo-staging.dev/242945))
* [article 86766](https://serlo-staging.dev/86766) ([current revision 242954](https://serlo-staging.dev/242954))
* [article 242841](https://serlo-staging.dev/242841) ([current revision 242981](https://serlo-staging.dev/242981))
* [article 186937](https://serlo-staging.dev/186937) ([current revision 243041](https://serlo-staging.dev/243041))
* [article 240630](https://serlo-staging.dev/240630) ([current revision 243057](https://serlo-staging.dev/243057))
* [text-solution 211651](https://serlo-staging.dev/211651) ([current revision 243094](https://serlo-staging.dev/243094))
* [text-solution 243125](https://serlo-staging.dev/243125) ([current revision 243138](https://serlo-staging.dev/243138))
* [text-solution 243156](https://serlo-staging.dev/243156) ([current revision 243171](https://serlo-staging.dev/243171))
* [text-solution 242411](https://serlo-staging.dev/242411) ([current revision 243175](https://serlo-staging.dev/243175))
* [article 85858](https://serlo-staging.dev/85858) ([current revision 243234](https://serlo-staging.dev/243234))
* [article 85855](https://serlo-staging.dev/85855) ([current revision 243332](https://serlo-staging.dev/243332))
* [article 26477](https://serlo-staging.dev/26477) ([current revision 243336](https://serlo-staging.dev/243336))
* [article 36570](https://serlo-staging.dev/36570) ([current revision 243339](https://serlo-staging.dev/243339))
* [text-solution 243389](https://serlo-staging.dev/243389) ([current revision 243396](https://serlo-staging.dev/243396))
* [text-solution 243391](https://serlo-staging.dev/243391) ([current revision 243398](https://serlo-staging.dev/243398))
* [text-solution 243393](https://serlo-staging.dev/243393) ([current revision 243400](https://serlo-staging.dev/243400))
* [text-solution 243408](https://serlo-staging.dev/243408) ([current revision 243421](https://serlo-staging.dev/243421))
* [text-solution 243410](https://serlo-staging.dev/243410) ([current revision 243423](https://serlo-staging.dev/243423))
* [text-solution 243412](https://serlo-staging.dev/243412) ([current revision 243425](https://serlo-staging.dev/243425))
* [text-solution 243414](https://serlo-staging.dev/243414) ([current revision 243427](https://serlo-staging.dev/243427))
* [text-solution 243416](https://serlo-staging.dev/243416) ([current revision 243429](https://serlo-staging.dev/243429))
* [text-solution 243418](https://serlo-staging.dev/243418) ([current revision 243431](https://serlo-staging.dev/243431))
* [article 1565](https://serlo-staging.dev/1565) ([current revision 243496](https://serlo-staging.dev/243496))
* [article 36264](https://serlo-staging.dev/36264) ([current revision 243497](https://serlo-staging.dev/243497))
* [article 100589](https://serlo-staging.dev/100589) ([current revision 243586](https://serlo-staging.dev/243586))
* [article 243724](https://serlo-staging.dev/243724) ([current revision 243762](https://serlo-staging.dev/243762))
* [text-solution 240450](https://serlo-staging.dev/240450) ([current revision 243777](https://serlo-staging.dev/243777))
* [article 243786](https://serlo-staging.dev/243786) ([current revision 243849](https://serlo-staging.dev/243849))
* [article 1901](https://serlo-staging.dev/1901) ([current revision 243892](https://serlo-staging.dev/243892))
* [article 243740](https://serlo-staging.dev/243740) ([current revision 243956](https://serlo-staging.dev/243956))
* [article 83792](https://serlo-staging.dev/83792) ([current revision 243959](https://serlo-staging.dev/243959))
* [article 48889](https://serlo-staging.dev/48889) ([current revision 244054](https://serlo-staging.dev/244054))
* [article 241559](https://serlo-staging.dev/241559) ([current revision 244132](https://serlo-staging.dev/244132))
* [text-solution 2741](https://serlo-staging.dev/2741) ([current revision 244151](https://serlo-staging.dev/244151))
* [text-solution 2745](https://serlo-staging.dev/2745) ([current revision 244247](https://serlo-staging.dev/244247))
* [text-solution 2749](https://serlo-staging.dev/2749) ([current revision 244248](https://serlo-staging.dev/244248))
* [text-solution 2753](https://serlo-staging.dev/2753) ([current revision 244249](https://serlo-staging.dev/244249))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [article 212639](https://serlo-staging.dev/212639) ([current revision 244355](https://serlo-staging.dev/244355))
* [article 244359](https://serlo-staging.dev/244359) ([current revision 244377](https://serlo-staging.dev/244377))
* [article 28903](https://serlo-staging.dev/28903) ([current revision 244417](https://serlo-staging.dev/244417))
* [article 244374](https://serlo-staging.dev/244374) ([current revision 244419](https://serlo-staging.dev/244419))
* [article 48907](https://serlo-staging.dev/48907) ([current revision 244420](https://serlo-staging.dev/244420))
* [article 51385](https://serlo-staging.dev/51385) ([current revision 244424](https://serlo-staging.dev/244424))
* [article 51392](https://serlo-staging.dev/51392) ([current revision 244425](https://serlo-staging.dev/244425))
* [article 53420](https://serlo-staging.dev/53420) ([current revision 244427](https://serlo-staging.dev/244427))
* [article 32305](https://serlo-staging.dev/32305) ([current revision 244500](https://serlo-staging.dev/244500))
* [article 244471](https://serlo-staging.dev/244471) ([current revision 244505](https://serlo-staging.dev/244505))
* [article 231349](https://serlo-staging.dev/231349) ([current revision 244599](https://serlo-staging.dev/244599))
* [article 53514](https://serlo-staging.dev/53514) ([current revision 244712](https://serlo-staging.dev/244712))
* [article 53475](https://serlo-staging.dev/53475) ([current revision 244796](https://serlo-staging.dev/244796))
* [article 244568](https://serlo-staging.dev/244568) ([current revision 244834](https://serlo-staging.dev/244834))
* [article 171074](https://serlo-staging.dev/171074) ([current revision 244868](https://serlo-staging.dev/244868))
* [text-solution 11691](https://serlo-staging.dev/11691) ([current revision 244937](https://serlo-staging.dev/244937))
* [article 239164](https://serlo-staging.dev/239164) ([current revision 245031](https://serlo-staging.dev/245031))
* [article 28295](https://serlo-staging.dev/28295) ([current revision 245179](https://serlo-staging.dev/245179))
* [article 1845](https://serlo-staging.dev/1845) ([current revision 245200](https://serlo-staging.dev/245200))
* [article 244530](https://serlo-staging.dev/244530) ([current revision 245398](https://serlo-staging.dev/245398))
* [article 199370](https://serlo-staging.dev/199370) ([current revision 245588](https://serlo-staging.dev/245588))
* [text-solution 2757](https://serlo-staging.dev/2757) ([current revision 245653](https://serlo-staging.dev/245653))
* [article 26470](https://serlo-staging.dev/26470) ([current revision 245656](https://serlo-staging.dev/245656))
* [text-solution 2761](https://serlo-staging.dev/2761) ([current revision 245662](https://serlo-staging.dev/245662))
* [text-solution 2765](https://serlo-staging.dev/2765) ([current revision 245666](https://serlo-staging.dev/245666))
* [article 245448](https://serlo-staging.dev/245448) ([current revision 245756](https://serlo-staging.dev/245756))
* [article 127689](https://serlo-staging.dev/127689) ([current revision 245835](https://serlo-staging.dev/245835))
* [article 244510](https://serlo-staging.dev/244510) ([current revision 245958](https://serlo-staging.dev/245958))
* [article 1615](https://serlo-staging.dev/1615) ([current revision 245996](https://serlo-staging.dev/245996))
* [article 244449](https://serlo-staging.dev/244449) ([current revision 246157](https://serlo-staging.dev/246157))
* [article 244459](https://serlo-staging.dev/244459) ([current revision 246160](https://serlo-staging.dev/246160))
* [article 1677](https://serlo-staging.dev/1677) ([current revision 246499](https://serlo-staging.dev/246499))
* [article 53031](https://serlo-staging.dev/53031) ([current revision 246581](https://serlo-staging.dev/246581))
* [article 65237](https://serlo-staging.dev/65237) ([current revision 246586](https://serlo-staging.dev/246586))
* [article 216291](https://serlo-staging.dev/216291) ([current revision 246787](https://serlo-staging.dev/246787))
* [text-exercise 7013](https://serlo-staging.dev/7013) ([current revision 247634](https://serlo-staging.dev/247634))
* [text-solution 7015](https://serlo-staging.dev/7015) ([current revision 247635](https://serlo-staging.dev/247635))
* [text-solution 9237](https://serlo-staging.dev/9237) ([current revision 247667](https://serlo-staging.dev/247667))
* [text-exercise 9247](https://serlo-staging.dev/9247) ([current revision 247672](https://serlo-staging.dev/247672))
* [text-exercise 9319](https://serlo-staging.dev/9319) ([current revision 247686](https://serlo-staging.dev/247686))
* [text-solution 9321](https://serlo-staging.dev/9321) ([current revision 247687](https://serlo-staging.dev/247687))
* [text-solution 6923](https://serlo-staging.dev/6923) ([current revision 247917](https://serlo-staging.dev/247917))
* [article 200334](https://serlo-staging.dev/200334) ([current revision 247991](https://serlo-staging.dev/247991))
* [text-solution 9425](https://serlo-staging.dev/9425) ([current revision 248042](https://serlo-staging.dev/248042))
* [text-solution 9719](https://serlo-staging.dev/9719) ([current revision 248151](https://serlo-staging.dev/248151))
* [article 74575](https://serlo-staging.dev/74575) ([current revision 248281](https://serlo-staging.dev/248281))
* [article 227853](https://serlo-staging.dev/227853) ([current revision 248292](https://serlo-staging.dev/248292))
* [article 247328](https://serlo-staging.dev/247328) ([current revision 248727](https://serlo-staging.dev/248727))
* [article 77192](https://serlo-staging.dev/77192) ([current revision 248807](https://serlo-staging.dev/248807))
* [article 77223](https://serlo-staging.dev/77223) ([current revision 248808](https://serlo-staging.dev/248808))
* [article 77230](https://serlo-staging.dev/77230) ([current revision 248809](https://serlo-staging.dev/248809))
* [article 1555](https://serlo-staging.dev/1555) ([current revision 248817](https://serlo-staging.dev/248817))
* [article 1635](https://serlo-staging.dev/1635) ([current revision 248824](https://serlo-staging.dev/248824))
* [article 1681](https://serlo-staging.dev/1681) ([current revision 248825](https://serlo-staging.dev/248825))
* [article 1687](https://serlo-staging.dev/1687) ([current revision 248826](https://serlo-staging.dev/248826))
* [article 1717](https://serlo-staging.dev/1717) ([current revision 248827](https://serlo-staging.dev/248827))
* [article 1763](https://serlo-staging.dev/1763) ([current revision 248830](https://serlo-staging.dev/248830))
* [article 1787](https://serlo-staging.dev/1787) ([current revision 248831](https://serlo-staging.dev/248831))
* [article 1825](https://serlo-staging.dev/1825) ([current revision 248834](https://serlo-staging.dev/248834))
* [article 1857](https://serlo-staging.dev/1857) ([current revision 248838](https://serlo-staging.dev/248838))
* [article 1871](https://serlo-staging.dev/1871) ([current revision 248843](https://serlo-staging.dev/248843))
* [article 1923](https://serlo-staging.dev/1923) ([current revision 248845](https://serlo-staging.dev/248845))
* [article 1961](https://serlo-staging.dev/1961) ([current revision 248846](https://serlo-staging.dev/248846))
* [article 1965](https://serlo-staging.dev/1965) ([current revision 248847](https://serlo-staging.dev/248847))
* [article 2033](https://serlo-staging.dev/2033) ([current revision 248853](https://serlo-staging.dev/248853))
* [article 2205](https://serlo-staging.dev/2205) ([current revision 248866](https://serlo-staging.dev/248866))
* [article 56179](https://serlo-staging.dev/56179) ([current revision 248877](https://serlo-staging.dev/248877))
* [article 54557](https://serlo-staging.dev/54557) ([current revision 248880](https://serlo-staging.dev/248880))
* [article 28009](https://serlo-staging.dev/28009) ([current revision 248895](https://serlo-staging.dev/248895))
* [article 41805](https://serlo-staging.dev/41805) ([current revision 248902](https://serlo-staging.dev/248902))
* [article 210061](https://serlo-staging.dev/210061) ([current revision 249091](https://serlo-staging.dev/249091))
* [article 1579](https://serlo-staging.dev/1579) ([current revision 249098](https://serlo-staging.dev/249098))
* [article 209594](https://serlo-staging.dev/209594) ([current revision 249163](https://serlo-staging.dev/249163))
* [article 204851](https://serlo-staging.dev/204851) ([current revision 249170](https://serlo-staging.dev/249170))

In [6]:
printRevisionsWithPlugin("spoiler")

# Entites with plugin spoiler

* [text-exercise 141954](https://serlo-staging.dev/141954) ([current revision 143337](https://serlo-staging.dev/143337))
* [course-page 61126](https://serlo-staging.dev/61126) ([current revision 143420](https://serlo-staging.dev/143420))
* [course-page 144005](https://serlo-staging.dev/144005) ([current revision 144178](https://serlo-staging.dev/144178))
* [course-page 144021](https://serlo-staging.dev/144021) ([current revision 144490](https://serlo-staging.dev/144490))
* [course-page 143526](https://serlo-staging.dev/143526) ([current revision 144759](https://serlo-staging.dev/144759))
* [course-page 143780](https://serlo-staging.dev/143780) ([current revision 144760](https://serlo-staging.dev/144760))
* [course-page 143458](https://serlo-staging.dev/143458) ([current revision 145013](https://serlo-staging.dev/145013))
* [course-page 143760](https://serlo-staging.dev/143760) ([current revision 146518](https://serlo-staging.dev/146518))
* [course-page 144015](https://serlo-staging.dev/144015) ([current revision 146531](https://serlo-staging.dev/146531))
* [article 1589](https://serlo-staging.dev/1589) ([current revision 146700](https://serlo-staging.dev/146700))
* [course-page 145801](https://serlo-staging.dev/145801) ([current revision 146788](https://serlo-staging.dev/146788))
* [course-page 145522](https://serlo-staging.dev/145522) ([current revision 147083](https://serlo-staging.dev/147083))
* [course-page 145639](https://serlo-staging.dev/145639) ([current revision 147084](https://serlo-staging.dev/147084))
* [event 145455](https://serlo-staging.dev/145455) ([current revision 147732](https://serlo-staging.dev/147732))
* [event 146675](https://serlo-staging.dev/146675) ([current revision 147742](https://serlo-staging.dev/147742))
* [article 1781](https://serlo-staging.dev/1781) ([current revision 148027](https://serlo-staging.dev/148027))
* [article 148316](https://serlo-staging.dev/148316) ([current revision 148318](https://serlo-staging.dev/148318))
* [event 147532](https://serlo-staging.dev/147532) ([current revision 149743](https://serlo-staging.dev/149743))
* [event 147705](https://serlo-staging.dev/147705) ([current revision 149744](https://serlo-staging.dev/149744))
* [text-exercise 82564](https://serlo-staging.dev/82564) ([current revision 151368](https://serlo-staging.dev/151368))
* [grouped-text-exercise 114326](https://serlo-staging.dev/114326) ([current revision 151523](https://serlo-staging.dev/151523))
* [text-solution 116406](https://serlo-staging.dev/116406) ([current revision 151636](https://serlo-staging.dev/151636))
* [text-solution 116413](https://serlo-staging.dev/116413) ([current revision 151655](https://serlo-staging.dev/151655))
* [text-solution 116410](https://serlo-staging.dev/116410) ([current revision 151672](https://serlo-staging.dev/151672))
* [text-solution 3453](https://serlo-staging.dev/3453) ([current revision 152383](https://serlo-staging.dev/152383))
* [text-solution 6435](https://serlo-staging.dev/6435) ([current revision 152771](https://serlo-staging.dev/152771))
* [text-solution 6485](https://serlo-staging.dev/6485) ([current revision 152835](https://serlo-staging.dev/152835))
* [text-solution 7127](https://serlo-staging.dev/7127) ([current revision 153020](https://serlo-staging.dev/153020))
* [text-solution 7131](https://serlo-staging.dev/7131) ([current revision 153028](https://serlo-staging.dev/153028))
* [text-solution 7147](https://serlo-staging.dev/7147) ([current revision 153043](https://serlo-staging.dev/153043))
* [course-page 151925](https://serlo-staging.dev/151925) ([current revision 153228](https://serlo-staging.dev/153228))
* [course-page 151952](https://serlo-staging.dev/151952) ([current revision 153231](https://serlo-staging.dev/153231))
* [course-page 151954](https://serlo-staging.dev/151954) ([current revision 153251](https://serlo-staging.dev/153251))
* [article 153236](https://serlo-staging.dev/153236) ([current revision 153397](https://serlo-staging.dev/153397))
* [article 150401](https://serlo-staging.dev/150401) ([current revision 153531](https://serlo-staging.dev/153531))
* [event 149177](https://serlo-staging.dev/149177) ([current revision 153541](https://serlo-staging.dev/153541))
* [course-page 127669](https://serlo-staging.dev/127669) ([current revision 155000](https://serlo-staging.dev/155000))
* [course-page 146306](https://serlo-staging.dev/146306) ([current revision 155003](https://serlo-staging.dev/155003))
* [text-solution 136492](https://serlo-staging.dev/136492) ([current revision 155129](https://serlo-staging.dev/155129))
* [text-solution 111238](https://serlo-staging.dev/111238) ([current revision 155134](https://serlo-staging.dev/155134))
* [text-solution 69136](https://serlo-staging.dev/69136) ([current revision 155198](https://serlo-staging.dev/155198))
* [article 155226](https://serlo-staging.dev/155226) ([current revision 155227](https://serlo-staging.dev/155227))
* [text-exercise 25157](https://serlo-staging.dev/25157) ([current revision 155407](https://serlo-staging.dev/155407))
* [text-solution 25758](https://serlo-staging.dev/25758) ([current revision 155441](https://serlo-staging.dev/155441))
* [text-exercise 132272](https://serlo-staging.dev/132272) ([current revision 155468](https://serlo-staging.dev/155468))
* [text-solution 27018](https://serlo-staging.dev/27018) ([current revision 155595](https://serlo-staging.dev/155595))
* [article 143959](https://serlo-staging.dev/143959) ([current revision 155662](https://serlo-staging.dev/155662))
* [article 1697](https://serlo-staging.dev/1697) ([current revision 155857](https://serlo-staging.dev/155857))
* [course-page 63786](https://serlo-staging.dev/63786) ([current revision 155928](https://serlo-staging.dev/155928))
* [course-page 31058](https://serlo-staging.dev/31058) ([current revision 156024](https://serlo-staging.dev/156024))
* [course-page 73158](https://serlo-staging.dev/73158) ([current revision 156213](https://serlo-staging.dev/156213))
* [text-exercise 73184](https://serlo-staging.dev/73184) ([current revision 156226](https://serlo-staging.dev/156226))
* [text-exercise 73633](https://serlo-staging.dev/73633) ([current revision 156248](https://serlo-staging.dev/156248))
* [text-solution 156193](https://serlo-staging.dev/156193) ([current revision 156275](https://serlo-staging.dev/156275))
* [text-solution 155585](https://serlo-staging.dev/155585) ([current revision 156358](https://serlo-staging.dev/156358))
* [text-solution 137997](https://serlo-staging.dev/137997) ([current revision 156433](https://serlo-staging.dev/156433))
* [text-solution 154758](https://serlo-staging.dev/154758) ([current revision 156465](https://serlo-staging.dev/156465))
* [course-page 156511](https://serlo-staging.dev/156511) ([current revision 156864](https://serlo-staging.dev/156864))
* [article 148782](https://serlo-staging.dev/148782) ([current revision 157018](https://serlo-staging.dev/157018))
* [text-solution 144395](https://serlo-staging.dev/144395) ([current revision 157135](https://serlo-staging.dev/157135))
* [course-page 127666](https://serlo-staging.dev/127666) ([current revision 157162](https://serlo-staging.dev/157162))
* [text-solution 27159](https://serlo-staging.dev/27159) ([current revision 157454](https://serlo-staging.dev/157454))
* [article 152323](https://serlo-staging.dev/152323) ([current revision 157605](https://serlo-staging.dev/157605))
* [text-solution 155626](https://serlo-staging.dev/155626) ([current revision 157607](https://serlo-staging.dev/157607))
* [text-exercise 33945](https://serlo-staging.dev/33945) ([current revision 157757](https://serlo-staging.dev/157757))
* [text-solution 101304](https://serlo-staging.dev/101304) ([current revision 158098](https://serlo-staging.dev/158098))
* [text-solution 143287](https://serlo-staging.dev/143287) ([current revision 158101](https://serlo-staging.dev/158101))
* [text-solution 142975](https://serlo-staging.dev/142975) ([current revision 158102](https://serlo-staging.dev/158102))
* [text-exercise 69493](https://serlo-staging.dev/69493) ([current revision 158159](https://serlo-staging.dev/158159))
* [text-solution 106585](https://serlo-staging.dev/106585) ([current revision 158347](https://serlo-staging.dev/158347))
* [article 157270](https://serlo-staging.dev/157270) ([current revision 158427](https://serlo-staging.dev/158427))
* [article 158092](https://serlo-staging.dev/158092) ([current revision 158730](https://serlo-staging.dev/158730))
* [article 1955](https://serlo-staging.dev/1955) ([current revision 158869](https://serlo-staging.dev/158869))
* [article 99614](https://serlo-staging.dev/99614) ([current revision 158892](https://serlo-staging.dev/158892))
* [course-page 63596](https://serlo-staging.dev/63596) ([current revision 159214](https://serlo-staging.dev/159214))
* [course-page 63769](https://serlo-staging.dev/63769) ([current revision 159215](https://serlo-staging.dev/159215))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [article 35620](https://serlo-staging.dev/35620) ([current revision 159472](https://serlo-staging.dev/159472))
* [text-exercise-group 159580](https://serlo-staging.dev/159580) ([current revision 159607](https://serlo-staging.dev/159607))
* [text-solution 83205](https://serlo-staging.dev/83205) ([current revision 159755](https://serlo-staging.dev/159755))
* [text-solution 114897](https://serlo-staging.dev/114897) ([current revision 159778](https://serlo-staging.dev/159778))
* [article 158031](https://serlo-staging.dev/158031) ([current revision 160200](https://serlo-staging.dev/160200))
* [text-exercise 129359](https://serlo-staging.dev/129359) ([current revision 160311](https://serlo-staging.dev/160311))
* [text-solution 99238](https://serlo-staging.dev/99238) ([current revision 160319](https://serlo-staging.dev/160319))
* [course-page 160587](https://serlo-staging.dev/160587) ([current revision 160597](https://serlo-staging.dev/160597))
* [text-exercise 132214](https://serlo-staging.dev/132214) ([current revision 160739](https://serlo-staging.dev/160739))
* [text-exercise 131807](https://serlo-staging.dev/131807) ([current revision 160744](https://serlo-staging.dev/160744))
* [text-exercise 131945](https://serlo-staging.dev/131945) ([current revision 160746](https://serlo-staging.dev/160746))
* [text-solution 3263](https://serlo-staging.dev/3263) ([current revision 160780](https://serlo-staging.dev/160780))
* [article 153419](https://serlo-staging.dev/153419) ([current revision 160793](https://serlo-staging.dev/160793))
* [article 25361](https://serlo-staging.dev/25361) ([current revision 160794](https://serlo-staging.dev/160794))
* [article 154653](https://serlo-staging.dev/154653) ([current revision 160851](https://serlo-staging.dev/160851))
* [article 158511](https://serlo-staging.dev/158511) ([current revision 160852](https://serlo-staging.dev/160852))
* [course-page 160566](https://serlo-staging.dev/160566) ([current revision 161007](https://serlo-staging.dev/161007))
* [text-solution 7163](https://serlo-staging.dev/7163) ([current revision 161353](https://serlo-staging.dev/161353))
* [text-solution 22860](https://serlo-staging.dev/22860) ([current revision 161688](https://serlo-staging.dev/161688))
* [text-solution 22862](https://serlo-staging.dev/22862) ([current revision 161690](https://serlo-staging.dev/161690))
* [text-solution 22864](https://serlo-staging.dev/22864) ([current revision 161692](https://serlo-staging.dev/161692))
* [text-solution 22866](https://serlo-staging.dev/22866) ([current revision 161694](https://serlo-staging.dev/161694))
* [text-solution 22868](https://serlo-staging.dev/22868) ([current revision 161696](https://serlo-staging.dev/161696))
* [text-solution 22870](https://serlo-staging.dev/22870) ([current revision 161698](https://serlo-staging.dev/161698))
* [text-solution 22872](https://serlo-staging.dev/22872) ([current revision 161700](https://serlo-staging.dev/161700))
* [article 159967](https://serlo-staging.dev/159967) ([current revision 162001](https://serlo-staging.dev/162001))
* [article 152324](https://serlo-staging.dev/152324) ([current revision 162003](https://serlo-staging.dev/162003))
* [article 161873](https://serlo-staging.dev/161873) ([current revision 162037](https://serlo-staging.dev/162037))
* [article 160615](https://serlo-staging.dev/160615) ([current revision 162041](https://serlo-staging.dev/162041))
* [text-solution 11861](https://serlo-staging.dev/11861) ([current revision 162067](https://serlo-staging.dev/162067))
* [article 155299](https://serlo-staging.dev/155299) ([current revision 162092](https://serlo-staging.dev/162092))
* [article 160788](https://serlo-staging.dev/160788) ([current revision 162137](https://serlo-staging.dev/162137))
* [article 128519](https://serlo-staging.dev/128519) ([current revision 162143](https://serlo-staging.dev/162143))
* [course-page 162053](https://serlo-staging.dev/162053) ([current revision 162197](https://serlo-staging.dev/162197))
* [article 77322](https://serlo-staging.dev/77322) ([current revision 162198](https://serlo-staging.dev/162198))
* [article 161845](https://serlo-staging.dev/161845) ([current revision 162250](https://serlo-staging.dev/162250))
* [text-solution 22874](https://serlo-staging.dev/22874) ([current revision 162277](https://serlo-staging.dev/162277))
* [text-solution 145569](https://serlo-staging.dev/145569) ([current revision 162402](https://serlo-staging.dev/162402))
* [text-solution 142792](https://serlo-staging.dev/142792) ([current revision 162440](https://serlo-staging.dev/162440))
* [article 43778](https://serlo-staging.dev/43778) ([current revision 163662](https://serlo-staging.dev/163662))
* [text-solution 161910](https://serlo-staging.dev/161910) ([current revision 163747](https://serlo-staging.dev/163747))
* [course-page 45454](https://serlo-staging.dev/45454) ([current revision 163767](https://serlo-staging.dev/163767))
* [article 25812](https://serlo-staging.dev/25812) ([current revision 163795](https://serlo-staging.dev/163795))
* [text-solution 154796](https://serlo-staging.dev/154796) ([current revision 163825](https://serlo-staging.dev/163825))
* [course-page 44693](https://serlo-staging.dev/44693) ([current revision 164141](https://serlo-staging.dev/164141))
* [text-exercise 161294](https://serlo-staging.dev/161294) ([current revision 164220](https://serlo-staging.dev/164220))
* [article 54534](https://serlo-staging.dev/54534) ([current revision 164224](https://serlo-staging.dev/164224))
* [text-solution 150907](https://serlo-staging.dev/150907) ([current revision 164288](https://serlo-staging.dev/164288))
* [article 164294](https://serlo-staging.dev/164294) ([current revision 164314](https://serlo-staging.dev/164314))
* [text-solution 100590](https://serlo-staging.dev/100590) ([current revision 164650](https://serlo-staging.dev/164650))
* [text-solution 100611](https://serlo-staging.dev/100611) ([current revision 164652](https://serlo-staging.dev/164652))
* [text-solution 100669](https://serlo-staging.dev/100669) ([current revision 164654](https://serlo-staging.dev/164654))
* [text-solution 101214](https://serlo-staging.dev/101214) ([current revision 164724](https://serlo-staging.dev/164724))
* [text-solution 145601](https://serlo-staging.dev/145601) ([current revision 164784](https://serlo-staging.dev/164784))
* [article 156048](https://serlo-staging.dev/156048) ([current revision 164803](https://serlo-staging.dev/164803))
* [course-page 164985](https://serlo-staging.dev/164985) ([current revision 164989](https://serlo-staging.dev/164989))
* [text-exercise-group 133257](https://serlo-staging.dev/133257) ([current revision 165404](https://serlo-staging.dev/165404))
* [text-solution 133161](https://serlo-staging.dev/133161) ([current revision 165431](https://serlo-staging.dev/165431))
* [text-solution 133171](https://serlo-staging.dev/133171) ([current revision 165433](https://serlo-staging.dev/165433))
* [text-solution 133179](https://serlo-staging.dev/133179) ([current revision 165435](https://serlo-staging.dev/165435))
* [text-exercise-group 132941](https://serlo-staging.dev/132941) ([current revision 165476](https://serlo-staging.dev/165476))
* [grouped-text-exercise 132965](https://serlo-staging.dev/132965) ([current revision 165481](https://serlo-staging.dev/165481))
* [text-exercise-group 132872](https://serlo-staging.dev/132872) ([current revision 165492](https://serlo-staging.dev/165492))
* [grouped-text-exercise 132904](https://serlo-staging.dev/132904) ([current revision 165497](https://serlo-staging.dev/165497))
* [text-exercise-group 131989](https://serlo-staging.dev/131989) ([current revision 165598](https://serlo-staging.dev/165598))
* [text-solution 3295](https://serlo-staging.dev/3295) ([current revision 165653](https://serlo-staging.dev/165653))
* [article 75512](https://serlo-staging.dev/75512) ([current revision 165785](https://serlo-staging.dev/165785))
* [text-solution 3415](https://serlo-staging.dev/3415) ([current revision 165821](https://serlo-staging.dev/165821))
* [text-solution 3423](https://serlo-staging.dev/3423) ([current revision 165823](https://serlo-staging.dev/165823))
* [text-solution 3427](https://serlo-staging.dev/3427) ([current revision 165825](https://serlo-staging.dev/165825))
* [text-solution 3431](https://serlo-staging.dev/3431) ([current revision 165827](https://serlo-staging.dev/165827))
* [text-solution 3435](https://serlo-staging.dev/3435) ([current revision 165829](https://serlo-staging.dev/165829))
* [text-solution 3439](https://serlo-staging.dev/3439) ([current revision 165831](https://serlo-staging.dev/165831))
* [text-solution 7159](https://serlo-staging.dev/7159) ([current revision 166110](https://serlo-staging.dev/166110))
* [course-page 79427](https://serlo-staging.dev/79427) ([current revision 166199](https://serlo-staging.dev/166199))
* [text-exercise-group 66706](https://serlo-staging.dev/66706) ([current revision 166522](https://serlo-staging.dev/166522))
* [text-solution 122255](https://serlo-staging.dev/122255) ([current revision 166926](https://serlo-staging.dev/166926))
* [text-exercise 166963](https://serlo-staging.dev/166963) ([current revision 167018](https://serlo-staging.dev/167018))
* [text-solution 21808](https://serlo-staging.dev/21808) ([current revision 167040](https://serlo-staging.dev/167040))
* [article 2193](https://serlo-staging.dev/2193) ([current revision 167135](https://serlo-staging.dev/167135))
* [text-solution 68869](https://serlo-staging.dev/68869) ([current revision 167766](https://serlo-staging.dev/167766))
* [article 167635](https://serlo-staging.dev/167635) ([current revision 167790](https://serlo-staging.dev/167790))
* [article 2023](https://serlo-staging.dev/2023) ([current revision 167841](https://serlo-staging.dev/167841))
* [article 167847](https://serlo-staging.dev/167847) ([current revision 167922](https://serlo-staging.dev/167922))
* [text-exercise-group 114110](https://serlo-staging.dev/114110) ([current revision 167986](https://serlo-staging.dev/167986))
* [article 62559](https://serlo-staging.dev/62559) ([current revision 168430](https://serlo-staging.dev/168430))
* [article 167426](https://serlo-staging.dev/167426) ([current revision 168497](https://serlo-staging.dev/168497))
* [text-solution 114838](https://serlo-staging.dev/114838) ([current revision 168671](https://serlo-staging.dev/168671))
* [article 168315](https://serlo-staging.dev/168315) ([current revision 168804](https://serlo-staging.dev/168804))
* [article 168274](https://serlo-staging.dev/168274) ([current revision 168855](https://serlo-staging.dev/168855))
* [text-solution 161119](https://serlo-staging.dev/161119) ([current revision 168931](https://serlo-staging.dev/168931))
* [text-solution 161121](https://serlo-staging.dev/161121) ([current revision 168933](https://serlo-staging.dev/168933))
* [text-solution 161123](https://serlo-staging.dev/161123) ([current revision 168935](https://serlo-staging.dev/168935))
* [text-exercise-group 140471](https://serlo-staging.dev/140471) ([current revision 169006](https://serlo-staging.dev/169006))
* [text-solution 168376](https://serlo-staging.dev/168376) ([current revision 169039](https://serlo-staging.dev/169039))
* [article 168325](https://serlo-staging.dev/168325) ([current revision 169110](https://serlo-staging.dev/169110))
* [course-page 168441](https://serlo-staging.dev/168441) ([current revision 169170](https://serlo-staging.dev/169170))
* [text-exercise-group 131869](https://serlo-staging.dev/131869) ([current revision 169211](https://serlo-staging.dev/169211))
* [text-solution 91513](https://serlo-staging.dev/91513) ([current revision 169288](https://serlo-staging.dev/169288))
* [event 145464](https://serlo-staging.dev/145464) ([current revision 169335](https://serlo-staging.dev/169335))
* [event 145458](https://serlo-staging.dev/145458) ([current revision 169336](https://serlo-staging.dev/169336))
* [event 145459](https://serlo-staging.dev/145459) ([current revision 169337](https://serlo-staging.dev/169337))
* [event 147733](https://serlo-staging.dev/147733) ([current revision 169339](https://serlo-staging.dev/169339))
* [course-page 19554](https://serlo-staging.dev/19554) ([current revision 169342](https://serlo-staging.dev/169342))
* [course-page 19599](https://serlo-staging.dev/19599) ([current revision 169343](https://serlo-staging.dev/169343))
* [course-page 19943](https://serlo-staging.dev/19943) ([current revision 169344](https://serlo-staging.dev/169344))
* [course-page 19979](https://serlo-staging.dev/19979) ([current revision 169345](https://serlo-staging.dev/169345))
* [course-page 19965](https://serlo-staging.dev/19965) ([current revision 169346](https://serlo-staging.dev/169346))
* [text-exercise 88341](https://serlo-staging.dev/88341) ([current revision 169416](https://serlo-staging.dev/169416))
* [article 90810](https://serlo-staging.dev/90810) ([current revision 169495](https://serlo-staging.dev/169495))
* [article 50493](https://serlo-staging.dev/50493) ([current revision 169524](https://serlo-staging.dev/169524))
* [text-solution 14847](https://serlo-staging.dev/14847) ([current revision 169528](https://serlo-staging.dev/169528))
* [text-solution 89629](https://serlo-staging.dev/89629) ([current revision 169534](https://serlo-staging.dev/169534))
* [text-solution 151776](https://serlo-staging.dev/151776) ([current revision 169547](https://serlo-staging.dev/169547))
* [article 168166](https://serlo-staging.dev/168166) ([current revision 169647](https://serlo-staging.dev/169647))
* [article 169765](https://serlo-staging.dev/169765) ([current revision 169816](https://serlo-staging.dev/169816))
* [course-page 79466](https://serlo-staging.dev/79466) ([current revision 169926](https://serlo-staging.dev/169926))
* [course-page 80463](https://serlo-staging.dev/80463) ([current revision 169927](https://serlo-staging.dev/169927))
* [text-solution 169909](https://serlo-staging.dev/169909) ([current revision 169967](https://serlo-staging.dev/169967))
* [text-solution 3201](https://serlo-staging.dev/3201) ([current revision 170090](https://serlo-staging.dev/170090))
* [article 167746](https://serlo-staging.dev/167746) ([current revision 170096](https://serlo-staging.dev/170096))
* [article 167784](https://serlo-staging.dev/167784) ([current revision 170357](https://serlo-staging.dev/170357))
* [article 168248](https://serlo-staging.dev/168248) ([current revision 170414](https://serlo-staging.dev/170414))
* [text-solution 46854](https://serlo-staging.dev/46854) ([current revision 170453](https://serlo-staging.dev/170453))
* [article 167755](https://serlo-staging.dev/167755) ([current revision 170467](https://serlo-staging.dev/170467))
* [article 168329](https://serlo-staging.dev/168329) ([current revision 170522](https://serlo-staging.dev/170522))
* [text-solution 6447](https://serlo-staging.dev/6447) ([current revision 170576](https://serlo-staging.dev/170576))
* [text-solution 144362](https://serlo-staging.dev/144362) ([current revision 170607](https://serlo-staging.dev/170607))
* [article 168363](https://serlo-staging.dev/168363) ([current revision 170623](https://serlo-staging.dev/170623))
* [article 168371](https://serlo-staging.dev/168371) ([current revision 170624](https://serlo-staging.dev/170624))
* [article 168400](https://serlo-staging.dev/168400) ([current revision 170632](https://serlo-staging.dev/170632))
* [text-exercise 40930](https://serlo-staging.dev/40930) ([current revision 170699](https://serlo-staging.dev/170699))
* [text-solution 37925](https://serlo-staging.dev/37925) ([current revision 170706](https://serlo-staging.dev/170706))
* [article 48392](https://serlo-staging.dev/48392) ([current revision 170794](https://serlo-staging.dev/170794))
* [article 167910](https://serlo-staging.dev/167910) ([current revision 170815](https://serlo-staging.dev/170815))
* [article 168360](https://serlo-staging.dev/168360) ([current revision 170841](https://serlo-staging.dev/170841))
* [text-solution 7333](https://serlo-staging.dev/7333) ([current revision 170913](https://serlo-staging.dev/170913))
* [text-solution 170081](https://serlo-staging.dev/170081) ([current revision 171003](https://serlo-staging.dev/171003))
* [article 170388](https://serlo-staging.dev/170388) ([current revision 171102](https://serlo-staging.dev/171102))
* [article 166154](https://serlo-staging.dev/166154) ([current revision 171150](https://serlo-staging.dev/171150))
* [article 1989](https://serlo-staging.dev/1989) ([current revision 171348](https://serlo-staging.dev/171348))
* [article 171413](https://serlo-staging.dev/171413) ([current revision 171462](https://serlo-staging.dev/171462))
* [article 171415](https://serlo-staging.dev/171415) ([current revision 171473](https://serlo-staging.dev/171473))
* [text-solution 131698](https://serlo-staging.dev/131698) ([current revision 171587](https://serlo-staging.dev/171587))
* [article 171471](https://serlo-staging.dev/171471) ([current revision 171614](https://serlo-staging.dev/171614))
* [text-exercise 171231](https://serlo-staging.dev/171231) ([current revision 171676](https://serlo-staging.dev/171676))
* [article 54317](https://serlo-staging.dev/54317) ([current revision 171699](https://serlo-staging.dev/171699))
* [text-solution 83211](https://serlo-staging.dev/83211) ([current revision 171767](https://serlo-staging.dev/171767))
* [text-solution 171624](https://serlo-staging.dev/171624) ([current revision 171802](https://serlo-staging.dev/171802))
* [text-solution 38550](https://serlo-staging.dev/38550) ([current revision 172103](https://serlo-staging.dev/172103))
* [course-page 172196](https://serlo-staging.dev/172196) ([current revision 172217](https://serlo-staging.dev/172217))
* [course-page 172178](https://serlo-staging.dev/172178) ([current revision 172233](https://serlo-staging.dev/172233))
* [article 49677](https://serlo-staging.dev/49677) ([current revision 172467](https://serlo-staging.dev/172467))
* [text-solution 169876](https://serlo-staging.dev/169876) ([current revision 172615](https://serlo-staging.dev/172615))
* [article 171955](https://serlo-staging.dev/171955) ([current revision 172626](https://serlo-staging.dev/172626))
* [article 170884](https://serlo-staging.dev/170884) ([current revision 172629](https://serlo-staging.dev/172629))
* [text-solution 14709](https://serlo-staging.dev/14709) ([current revision 173012](https://serlo-staging.dev/173012))
* [text-solution 14713](https://serlo-staging.dev/14713) ([current revision 173013](https://serlo-staging.dev/173013))
* [text-solution 14717](https://serlo-staging.dev/14717) ([current revision 173014](https://serlo-staging.dev/173014))
* [text-solution 14721](https://serlo-staging.dev/14721) ([current revision 173015](https://serlo-staging.dev/173015))
* [text-solution 14725](https://serlo-staging.dev/14725) ([current revision 173016](https://serlo-staging.dev/173016))
* [text-solution 14729](https://serlo-staging.dev/14729) ([current revision 173017](https://serlo-staging.dev/173017))
* [article 168538](https://serlo-staging.dev/168538) ([current revision 173433](https://serlo-staging.dev/173433))
* [article 168163](https://serlo-staging.dev/168163) ([current revision 173486](https://serlo-staging.dev/173486))
* [article 2051](https://serlo-staging.dev/2051) ([current revision 173491](https://serlo-staging.dev/173491))
* [article 168076](https://serlo-staging.dev/168076) ([current revision 173754](https://serlo-staging.dev/173754))
* [article 55508](https://serlo-staging.dev/55508) ([current revision 173870](https://serlo-staging.dev/173870))
* [article 32200](https://serlo-staging.dev/32200) ([current revision 173878](https://serlo-staging.dev/173878))
* [article 34159](https://serlo-staging.dev/34159) ([current revision 174409](https://serlo-staging.dev/174409))
* [course-page 45558](https://serlo-staging.dev/45558) ([current revision 174634](https://serlo-staging.dev/174634))
* [course-page 173327](https://serlo-staging.dev/173327) ([current revision 174917](https://serlo-staging.dev/174917))
* [course-page 174901](https://serlo-staging.dev/174901) ([current revision 174918](https://serlo-staging.dev/174918))
* [course-page 173324](https://serlo-staging.dev/173324) ([current revision 174920](https://serlo-staging.dev/174920))
* [course-page 173325](https://serlo-staging.dev/173325) ([current revision 174921](https://serlo-staging.dev/174921))
* [course-page 173326](https://serlo-staging.dev/173326) ([current revision 174922](https://serlo-staging.dev/174922))
* [article 168256](https://serlo-staging.dev/168256) ([current revision 175193](https://serlo-staging.dev/175193))
* [article 164144](https://serlo-staging.dev/164144) ([current revision 175196](https://serlo-staging.dev/175196))
* [article 175266](https://serlo-staging.dev/175266) ([current revision 175277](https://serlo-staging.dev/175277))
* [text-solution 175148](https://serlo-staging.dev/175148) ([current revision 175444](https://serlo-staging.dev/175444))
* [text-solution 30070](https://serlo-staging.dev/30070) ([current revision 175833](https://serlo-staging.dev/175833))
* [text-exercise-group 67198](https://serlo-staging.dev/67198) ([current revision 175836](https://serlo-staging.dev/175836))
* [text-solution 30649](https://serlo-staging.dev/30649) ([current revision 176873](https://serlo-staging.dev/176873))
* [text-solution 30696](https://serlo-staging.dev/30696) ([current revision 176877](https://serlo-staging.dev/176877))
* [text-solution 30753](https://serlo-staging.dev/30753) ([current revision 176881](https://serlo-staging.dev/176881))
* [text-solution 30963](https://serlo-staging.dev/30963) ([current revision 176889](https://serlo-staging.dev/176889))
* [text-solution 30982](https://serlo-staging.dev/30982) ([current revision 176893](https://serlo-staging.dev/176893))
* [text-solution 30779](https://serlo-staging.dev/30779) ([current revision 176900](https://serlo-staging.dev/176900))
* [text-solution 30797](https://serlo-staging.dev/30797) ([current revision 176902](https://serlo-staging.dev/176902))
* [text-solution 30808](https://serlo-staging.dev/30808) ([current revision 176904](https://serlo-staging.dev/176904))
* [text-solution 31060](https://serlo-staging.dev/31060) ([current revision 176943](https://serlo-staging.dev/176943))
* [text-solution 31097](https://serlo-staging.dev/31097) ([current revision 176944](https://serlo-staging.dev/176944))
* [text-solution 31096](https://serlo-staging.dev/31096) ([current revision 176945](https://serlo-staging.dev/176945))
* [text-solution 15387](https://serlo-staging.dev/15387) ([current revision 176948](https://serlo-staging.dev/176948))
* [text-exercise-group 65763](https://serlo-staging.dev/65763) ([current revision 177212](https://serlo-staging.dev/177212))
* [text-solution 176362](https://serlo-staging.dev/176362) ([current revision 177378](https://serlo-staging.dev/177378))
* [text-exercise 38018](https://serlo-staging.dev/38018) ([current revision 177380](https://serlo-staging.dev/177380))
* [article 175766](https://serlo-staging.dev/175766) ([current revision 177473](https://serlo-staging.dev/177473))
* [text-solution 177056](https://serlo-staging.dev/177056) ([current revision 177592](https://serlo-staging.dev/177592))
* [article 1747](https://serlo-staging.dev/1747) ([current revision 177783](https://serlo-staging.dev/177783))
* [course-page 177804](https://serlo-staging.dev/177804) ([current revision 177811](https://serlo-staging.dev/177811))
* [text-solution 44394](https://serlo-staging.dev/44394) ([current revision 178079](https://serlo-staging.dev/178079))
* [text-solution 44470](https://serlo-staging.dev/44470) ([current revision 178082](https://serlo-staging.dev/178082))
* [article 178239](https://serlo-staging.dev/178239) ([current revision 178299](https://serlo-staging.dev/178299))
* [article 178252](https://serlo-staging.dev/178252) ([current revision 178394](https://serlo-staging.dev/178394))
* [text-solution 169572](https://serlo-staging.dev/169572) ([current revision 178484](https://serlo-staging.dev/178484))
* [text-exercise-group 66789](https://serlo-staging.dev/66789) ([current revision 178575](https://serlo-staging.dev/178575))
* [text-solution 66841](https://serlo-staging.dev/66841) ([current revision 178581](https://serlo-staging.dev/178581))
* [text-exercise-group 65920](https://serlo-staging.dev/65920) ([current revision 178593](https://serlo-staging.dev/178593))
* [grouped-text-exercise 62781](https://serlo-staging.dev/62781) ([current revision 178600](https://serlo-staging.dev/178600))
* [grouped-text-exercise 62800](https://serlo-staging.dev/62800) ([current revision 178603](https://serlo-staging.dev/178603))
* [grouped-text-exercise 62823](https://serlo-staging.dev/62823) ([current revision 178604](https://serlo-staging.dev/178604))
* [article 178269](https://serlo-staging.dev/178269) ([current revision 178722](https://serlo-staging.dev/178722))
* [article 120649](https://serlo-staging.dev/120649) ([current revision 178830](https://serlo-staging.dev/178830))
* [text-solution 96721](https://serlo-staging.dev/96721) ([current revision 179282](https://serlo-staging.dev/179282))
* [text-solution 106241](https://serlo-staging.dev/106241) ([current revision 179371](https://serlo-staging.dev/179371))
* [event 179657](https://serlo-staging.dev/179657) ([current revision 179698](https://serlo-staging.dev/179698))
* [article 178914](https://serlo-staging.dev/178914) ([current revision 179819](https://serlo-staging.dev/179819))
* [article 167497](https://serlo-staging.dev/167497) ([current revision 179907](https://serlo-staging.dev/179907))
* [article 178254](https://serlo-staging.dev/178254) ([current revision 180002](https://serlo-staging.dev/180002))
* [event 145590](https://serlo-staging.dev/145590) ([current revision 180193](https://serlo-staging.dev/180193))
* [course-page 179598](https://serlo-staging.dev/179598) ([current revision 180331](https://serlo-staging.dev/180331))
* [text-solution 9277](https://serlo-staging.dev/9277) ([current revision 180347](https://serlo-staging.dev/180347))
* [course-page 177812](https://serlo-staging.dev/177812) ([current revision 180479](https://serlo-staging.dev/180479))
* [article 178258](https://serlo-staging.dev/178258) ([current revision 180707](https://serlo-staging.dev/180707))
* [article 178261](https://serlo-staging.dev/178261) ([current revision 180773](https://serlo-staging.dev/180773))
* [article 178154](https://serlo-staging.dev/178154) ([current revision 180821](https://serlo-staging.dev/180821))
* [article 178732](https://serlo-staging.dev/178732) ([current revision 180836](https://serlo-staging.dev/180836))
* [article 179771](https://serlo-staging.dev/179771) ([current revision 180845](https://serlo-staging.dev/180845))
* [article 179780](https://serlo-staging.dev/179780) ([current revision 180850](https://serlo-staging.dev/180850))
* [article 90402](https://serlo-staging.dev/90402) ([current revision 181216](https://serlo-staging.dev/181216))
* [text-solution 108099](https://serlo-staging.dev/108099) ([current revision 181301](https://serlo-staging.dev/181301))
* [article 134827](https://serlo-staging.dev/134827) ([current revision 181491](https://serlo-staging.dev/181491))
* [article 181564](https://serlo-staging.dev/181564) ([current revision 181568](https://serlo-staging.dev/181568))
* [course-page 19495](https://serlo-staging.dev/19495) ([current revision 181887](https://serlo-staging.dev/181887))
* [article 27823](https://serlo-staging.dev/27823) ([current revision 181889](https://serlo-staging.dev/181889))
* [text-solution 7107](https://serlo-staging.dev/7107) ([current revision 181933](https://serlo-staging.dev/181933))
* [article 1651](https://serlo-staging.dev/1651) ([current revision 181977](https://serlo-staging.dev/181977))
* [text-solution 9429](https://serlo-staging.dev/9429) ([current revision 182201](https://serlo-staging.dev/182201))
* [text-solution 179336](https://serlo-staging.dev/179336) ([current revision 182209](https://serlo-staging.dev/182209))
* [article 2099](https://serlo-staging.dev/2099) ([current revision 182832](https://serlo-staging.dev/182832))
* [text-solution 4157](https://serlo-staging.dev/4157) ([current revision 183166](https://serlo-staging.dev/183166))
* [text-solution 5587](https://serlo-staging.dev/5587) ([current revision 183170](https://serlo-staging.dev/183170))
* [course-page 183260](https://serlo-staging.dev/183260) ([current revision 183277](https://serlo-staging.dev/183277))
* [text-exercise 181870](https://serlo-staging.dev/181870) ([current revision 183302](https://serlo-staging.dev/183302))
* [text-exercise 182017](https://serlo-staging.dev/182017) ([current revision 183303](https://serlo-staging.dev/183303))
* [article 181042](https://serlo-staging.dev/181042) ([current revision 183412](https://serlo-staging.dev/183412))
* [course-page 183179](https://serlo-staging.dev/183179) ([current revision 183444](https://serlo-staging.dev/183444))
* [text-solution 93470](https://serlo-staging.dev/93470) ([current revision 183446](https://serlo-staging.dev/183446))
* [event 169327](https://serlo-staging.dev/169327) ([current revision 183938](https://serlo-staging.dev/183938))
* [course-page 168342](https://serlo-staging.dev/168342) ([current revision 184009](https://serlo-staging.dev/184009))
* [course-page 167673](https://serlo-staging.dev/167673) ([current revision 184083](https://serlo-staging.dev/184083))
* [course-page 167674](https://serlo-staging.dev/167674) ([current revision 184084](https://serlo-staging.dev/184084))
* [course-page 168456](https://serlo-staging.dev/168456) ([current revision 184088](https://serlo-staging.dev/184088))
* [course-page 168475](https://serlo-staging.dev/168475) ([current revision 184089](https://serlo-staging.dev/184089))
* [course-page 168494](https://serlo-staging.dev/168494) ([current revision 184091](https://serlo-staging.dev/184091))
* [article 180173](https://serlo-staging.dev/180173) ([current revision 184390](https://serlo-staging.dev/184390))
* [article 170325](https://serlo-staging.dev/170325) ([current revision 184622](https://serlo-staging.dev/184622))
* [article 169545](https://serlo-staging.dev/169545) ([current revision 184628](https://serlo-staging.dev/184628))
* [article 170363](https://serlo-staging.dev/170363) ([current revision 184688](https://serlo-staging.dev/184688))
* [text-solution 111796](https://serlo-staging.dev/111796) ([current revision 184756](https://serlo-staging.dev/184756))
* [article 38906](https://serlo-staging.dev/38906) ([current revision 184807](https://serlo-staging.dev/184807))
* [article 90391](https://serlo-staging.dev/90391) ([current revision 184861](https://serlo-staging.dev/184861))
* [article 90395](https://serlo-staging.dev/90395) ([current revision 184862](https://serlo-staging.dev/184862))
* [text-solution 109477](https://serlo-staging.dev/109477) ([current revision 185118](https://serlo-staging.dev/185118))
* [article 184028](https://serlo-staging.dev/184028) ([current revision 185563](https://serlo-staging.dev/185563))
* [course-page 109327](https://serlo-staging.dev/109327) ([current revision 185597](https://serlo-staging.dev/185597))
* [article 181279](https://serlo-staging.dev/181279) ([current revision 185617](https://serlo-staging.dev/185617))
* [article 53038](https://serlo-staging.dev/53038) ([current revision 185644](https://serlo-staging.dev/185644))
* [text-solution 10957](https://serlo-staging.dev/10957) ([current revision 185726](https://serlo-staging.dev/185726))
* [course-page 182295](https://serlo-staging.dev/182295) ([current revision 185829](https://serlo-staging.dev/185829))
* [course-page 186021](https://serlo-staging.dev/186021) ([current revision 186027](https://serlo-staging.dev/186027))
* [course-page 186030](https://serlo-staging.dev/186030) ([current revision 186038](https://serlo-staging.dev/186038))
* [course-page 186029](https://serlo-staging.dev/186029) ([current revision 186041](https://serlo-staging.dev/186041))
* [course-page 186101](https://serlo-staging.dev/186101) ([current revision 186115](https://serlo-staging.dev/186115))
* [course-page 186102](https://serlo-staging.dev/186102) ([current revision 186116](https://serlo-staging.dev/186116))
* [course-page 186103](https://serlo-staging.dev/186103) ([current revision 186117](https://serlo-staging.dev/186117))
* [course-page 186104](https://serlo-staging.dev/186104) ([current revision 186118](https://serlo-staging.dev/186118))
* [course-page 186105](https://serlo-staging.dev/186105) ([current revision 186119](https://serlo-staging.dev/186119))
* [course-page 186320](https://serlo-staging.dev/186320) ([current revision 186457](https://serlo-staging.dev/186457))
* [text-solution 65505](https://serlo-staging.dev/65505) ([current revision 186496](https://serlo-staging.dev/186496))
* [text-exercise 104167](https://serlo-staging.dev/104167) ([current revision 186510](https://serlo-staging.dev/186510))
* [grouped-text-exercise 58323](https://serlo-staging.dev/58323) ([current revision 186520](https://serlo-staging.dev/186520))
* [grouped-text-exercise 58433](https://serlo-staging.dev/58433) ([current revision 186524](https://serlo-staging.dev/186524))
* [text-solution 98223](https://serlo-staging.dev/98223) ([current revision 186532](https://serlo-staging.dev/186532))
* [course-page 186563](https://serlo-staging.dev/186563) ([current revision 186573](https://serlo-staging.dev/186573))
* [article 186610](https://serlo-staging.dev/186610) ([current revision 186623](https://serlo-staging.dev/186623))
* [article 107863](https://serlo-staging.dev/107863) ([current revision 186662](https://serlo-staging.dev/186662))
* [text-solution 184813](https://serlo-staging.dev/184813) ([current revision 186672](https://serlo-staging.dev/186672))
* [text-solution 184505](https://serlo-staging.dev/184505) ([current revision 186673](https://serlo-staging.dev/186673))
* [article 54618](https://serlo-staging.dev/54618) ([current revision 186909](https://serlo-staging.dev/186909))
* [course-page 186270](https://serlo-staging.dev/186270) ([current revision 186979](https://serlo-staging.dev/186979))
* [course-page 186276](https://serlo-staging.dev/186276) ([current revision 186982](https://serlo-staging.dev/186982))
* [text-exercise 38634](https://serlo-staging.dev/38634) ([current revision 187099](https://serlo-staging.dev/187099))
* [course-page 187203](https://serlo-staging.dev/187203) ([current revision 187216](https://serlo-staging.dev/187216))
* [course-page 186280](https://serlo-staging.dev/186280) ([current revision 187244](https://serlo-staging.dev/187244))
* [course-page 187273](https://serlo-staging.dev/187273) ([current revision 187282](https://serlo-staging.dev/187282))
* [article 164239](https://serlo-staging.dev/164239) ([current revision 187403](https://serlo-staging.dev/187403))
* [course-page 18826](https://serlo-staging.dev/18826) ([current revision 187420](https://serlo-staging.dev/187420))
* [course-page 19046](https://serlo-staging.dev/19046) ([current revision 187421](https://serlo-staging.dev/187421))
* [text-solution 9233](https://serlo-staging.dev/9233) ([current revision 187622](https://serlo-staging.dev/187622))
* [article 185542](https://serlo-staging.dev/185542) ([current revision 187792](https://serlo-staging.dev/187792))
* [article 181412](https://serlo-staging.dev/181412) ([current revision 187880](https://serlo-staging.dev/187880))
* [text-solution 188245](https://serlo-staging.dev/188245) ([current revision 188247](https://serlo-staging.dev/188247))
* [text-solution 188255](https://serlo-staging.dev/188255) ([current revision 188257](https://serlo-staging.dev/188257))
* [article 1653](https://serlo-staging.dev/1653) ([current revision 188370](https://serlo-staging.dev/188370))
* [course-page 188326](https://serlo-staging.dev/188326) ([current revision 188385](https://serlo-staging.dev/188385))
* [course-page 186293](https://serlo-staging.dev/186293) ([current revision 188403](https://serlo-staging.dev/188403))
* [text-exercise 13363](https://serlo-staging.dev/13363) ([current revision 188426](https://serlo-staging.dev/188426))
* [text-solution 13365](https://serlo-staging.dev/13365) ([current revision 188427](https://serlo-staging.dev/188427))
* [article 35866](https://serlo-staging.dev/35866) ([current revision 188455](https://serlo-staging.dev/188455))
* [grouped-text-exercise 188579](https://serlo-staging.dev/188579) ([current revision 188612](https://serlo-staging.dev/188612))
* [article 2131](https://serlo-staging.dev/2131) ([current revision 188837](https://serlo-staging.dev/188837))
* [article 178256](https://serlo-staging.dev/178256) ([current revision 188868](https://serlo-staging.dev/188868))
* [article 1995](https://serlo-staging.dev/1995) ([current revision 189047](https://serlo-staging.dev/189047))
* [course-page 189067](https://serlo-staging.dev/189067) ([current revision 189075](https://serlo-staging.dev/189075))
* [text-solution 189084](https://serlo-staging.dev/189084) ([current revision 189086](https://serlo-staging.dev/189086))
* [course-page 189050](https://serlo-staging.dev/189050) ([current revision 189101](https://serlo-staging.dev/189101))
* [text-solution 31139](https://serlo-staging.dev/31139) ([current revision 189190](https://serlo-staging.dev/189190))
* [article 59178](https://serlo-staging.dev/59178) ([current revision 189471](https://serlo-staging.dev/189471))
* [article 1907](https://serlo-staging.dev/1907) ([current revision 189675](https://serlo-staging.dev/189675))
* [text-exercise-group 189476](https://serlo-staging.dev/189476) ([current revision 189703](https://serlo-staging.dev/189703))
* [text-solution 190044](https://serlo-staging.dev/190044) ([current revision 190186](https://serlo-staging.dev/190186))
* [course-page 19032](https://serlo-staging.dev/19032) ([current revision 190234](https://serlo-staging.dev/190234))
* [course-page 19037](https://serlo-staging.dev/19037) ([current revision 190235](https://serlo-staging.dev/190235))
* [course-page 19059](https://serlo-staging.dev/19059) ([current revision 190236](https://serlo-staging.dev/190236))
* [course-page 19079](https://serlo-staging.dev/19079) ([current revision 190237](https://serlo-staging.dev/190237))
* [course-page 19229](https://serlo-staging.dev/19229) ([current revision 190239](https://serlo-staging.dev/190239))
* [course-page 19270](https://serlo-staging.dev/19270) ([current revision 190240](https://serlo-staging.dev/190240))
* [course-page 19297](https://serlo-staging.dev/19297) ([current revision 190241](https://serlo-staging.dev/190241))
* [course-page 19334](https://serlo-staging.dev/19334) ([current revision 190243](https://serlo-staging.dev/190243))
* [article 187487](https://serlo-staging.dev/187487) ([current revision 190508](https://serlo-staging.dev/190508))
* [article 64627](https://serlo-staging.dev/64627) ([current revision 190514](https://serlo-staging.dev/190514))
* [course-page 190727](https://serlo-staging.dev/190727) ([current revision 190728](https://serlo-staging.dev/190728))
* [course-page 190760](https://serlo-staging.dev/190760) ([current revision 190764](https://serlo-staging.dev/190764))
* [text-solution 75931](https://serlo-staging.dev/75931) ([current revision 190967](https://serlo-staging.dev/190967))
* [course-page 84057](https://serlo-staging.dev/84057) ([current revision 190975](https://serlo-staging.dev/190975))
* [course-page 190971](https://serlo-staging.dev/190971) ([current revision 191135](https://serlo-staging.dev/191135))
* [article 155311](https://serlo-staging.dev/155311) ([current revision 191244](https://serlo-staging.dev/191244))
* [article 188985](https://serlo-staging.dev/188985) ([current revision 191254](https://serlo-staging.dev/191254))
* [article 43735](https://serlo-staging.dev/43735) ([current revision 191277](https://serlo-staging.dev/191277))
* [article 95773](https://serlo-staging.dev/95773) ([current revision 191281](https://serlo-staging.dev/191281))
* [article 173317](https://serlo-staging.dev/173317) ([current revision 191423](https://serlo-staging.dev/191423))
* [article 185678](https://serlo-staging.dev/185678) ([current revision 191534](https://serlo-staging.dev/191534))
* [article 188404](https://serlo-staging.dev/188404) ([current revision 191664](https://serlo-staging.dev/191664))
* [text-solution 143214](https://serlo-staging.dev/143214) ([current revision 191689](https://serlo-staging.dev/191689))
* [text-solution 138665](https://serlo-staging.dev/138665) ([current revision 191697](https://serlo-staging.dev/191697))
* [text-solution 137622](https://serlo-staging.dev/137622) ([current revision 191723](https://serlo-staging.dev/191723))
* [text-solution 134805](https://serlo-staging.dev/134805) ([current revision 191726](https://serlo-staging.dev/191726))
* [text-solution 5159](https://serlo-staging.dev/5159) ([current revision 191806](https://serlo-staging.dev/191806))
* [text-exercise 175512](https://serlo-staging.dev/175512) ([current revision 191867](https://serlo-staging.dev/191867))
* [text-solution 5467](https://serlo-staging.dev/5467) ([current revision 191947](https://serlo-staging.dev/191947))
* [text-solution 5471](https://serlo-staging.dev/5471) ([current revision 191963](https://serlo-staging.dev/191963))
* [article 191608](https://serlo-staging.dev/191608) ([current revision 191972](https://serlo-staging.dev/191972))
* [course-page 191987](https://serlo-staging.dev/191987) ([current revision 191998](https://serlo-staging.dev/191998))
* [article 2143](https://serlo-staging.dev/2143) ([current revision 192046](https://serlo-staging.dev/192046))
* [course-page 192036](https://serlo-staging.dev/192036) ([current revision 192194](https://serlo-staging.dev/192194))
* [course-page 192193](https://serlo-staging.dev/192193) ([current revision 192200](https://serlo-staging.dev/192200))
* [text-solution 113610](https://serlo-staging.dev/113610) ([current revision 192305](https://serlo-staging.dev/192305))
* [text-solution 99212](https://serlo-staging.dev/99212) ([current revision 192311](https://serlo-staging.dev/192311))
* [text-solution 100368](https://serlo-staging.dev/100368) ([current revision 192319](https://serlo-staging.dev/192319))
* [text-solution 94119](https://serlo-staging.dev/94119) ([current revision 192320](https://serlo-staging.dev/192320))
* [text-solution 81363](https://serlo-staging.dev/81363) ([current revision 192321](https://serlo-staging.dev/192321))
* [article 43629](https://serlo-staging.dev/43629) ([current revision 192347](https://serlo-staging.dev/192347))
* [article 35632](https://serlo-staging.dev/35632) ([current revision 192368](https://serlo-staging.dev/192368))
* [article 27802](https://serlo-staging.dev/27802) ([current revision 192373](https://serlo-staging.dev/192373))
* [course-page 56488](https://serlo-staging.dev/56488) ([current revision 192424](https://serlo-staging.dev/192424))
* [text-solution 139629](https://serlo-staging.dev/139629) ([current revision 192441](https://serlo-staging.dev/192441))
* [article 2009](https://serlo-staging.dev/2009) ([current revision 192444](https://serlo-staging.dev/192444))
* [article 1643](https://serlo-staging.dev/1643) ([current revision 192446](https://serlo-staging.dev/192446))
* [article 192450](https://serlo-staging.dev/192450) ([current revision 192477](https://serlo-staging.dev/192477))
* [course-page 192300](https://serlo-staging.dev/192300) ([current revision 192530](https://serlo-staging.dev/192530))
* [article 1875](https://serlo-staging.dev/1875) ([current revision 192709](https://serlo-staging.dev/192709))
* [article 51468](https://serlo-staging.dev/51468) ([current revision 192720](https://serlo-staging.dev/192720))
* [article 1607](https://serlo-staging.dev/1607) ([current revision 192797](https://serlo-staging.dev/192797))
* [article 1735](https://serlo-staging.dev/1735) ([current revision 192800](https://serlo-staging.dev/192800))
* [article 1803](https://serlo-staging.dev/1803) ([current revision 192807](https://serlo-staging.dev/192807))
* [article 1827](https://serlo-staging.dev/1827) ([current revision 192819](https://serlo-staging.dev/192819))
* [text-solution 12901](https://serlo-staging.dev/12901) ([current revision 192880](https://serlo-staging.dev/192880))
* [article 1629](https://serlo-staging.dev/1629) ([current revision 192942](https://serlo-staging.dev/192942))
* [article 184495](https://serlo-staging.dev/184495) ([current revision 192996](https://serlo-staging.dev/192996))
* [article 187106](https://serlo-staging.dev/187106) ([current revision 193010](https://serlo-staging.dev/193010))
* [course-page 36766](https://serlo-staging.dev/36766) ([current revision 193043](https://serlo-staging.dev/193043))
* [course-page 36800](https://serlo-staging.dev/36800) ([current revision 193049](https://serlo-staging.dev/193049))
* [article 62547](https://serlo-staging.dev/62547) ([current revision 193082](https://serlo-staging.dev/193082))
* [article 64134](https://serlo-staging.dev/64134) ([current revision 193128](https://serlo-staging.dev/193128))
* [course-page 185999](https://serlo-staging.dev/185999) ([current revision 193162](https://serlo-staging.dev/193162))
* [article 193131](https://serlo-staging.dev/193131) ([current revision 193199](https://serlo-staging.dev/193199))
* [text-solution 6459](https://serlo-staging.dev/6459) ([current revision 193271](https://serlo-staging.dev/193271))
* [course-page 125654](https://serlo-staging.dev/125654) ([current revision 193273](https://serlo-staging.dev/193273))
* [course-page 123982](https://serlo-staging.dev/123982) ([current revision 193278](https://serlo-staging.dev/193278))
* [article 85864](https://serlo-staging.dev/85864) ([current revision 193281](https://serlo-staging.dev/193281))
* [article 104026](https://serlo-staging.dev/104026) ([current revision 193293](https://serlo-staging.dev/193293))
* [course-page 46811](https://serlo-staging.dev/46811) ([current revision 193295](https://serlo-staging.dev/193295))
* [article 45468](https://serlo-staging.dev/45468) ([current revision 193302](https://serlo-staging.dev/193302))
* [article 193200](https://serlo-staging.dev/193200) ([current revision 193305](https://serlo-staging.dev/193305))
* [article 193413](https://serlo-staging.dev/193413) ([current revision 193433](https://serlo-staging.dev/193433))
* [article 90405](https://serlo-staging.dev/90405) ([current revision 193490](https://serlo-staging.dev/193490))
* [article 193457](https://serlo-staging.dev/193457) ([current revision 193540](https://serlo-staging.dev/193540))
* [text-solution 9417](https://serlo-staging.dev/9417) ([current revision 193575](https://serlo-staging.dev/193575))
* [text-solution 193588](https://serlo-staging.dev/193588) ([current revision 193593](https://serlo-staging.dev/193593))
* [text-solution 193590](https://serlo-staging.dev/193590) ([current revision 193595](https://serlo-staging.dev/193595))
* [article 1661](https://serlo-staging.dev/1661) ([current revision 193600](https://serlo-staging.dev/193600))
* [article 2155](https://serlo-staging.dev/2155) ([current revision 193618](https://serlo-staging.dev/193618))
* [article 43437](https://serlo-staging.dev/43437) ([current revision 193648](https://serlo-staging.dev/193648))
* [article 2145](https://serlo-staging.dev/2145) ([current revision 193664](https://serlo-staging.dev/193664))
* [course-page 43531](https://serlo-staging.dev/43531) ([current revision 193665](https://serlo-staging.dev/193665))
* [article 193578](https://serlo-staging.dev/193578) ([current revision 193669](https://serlo-staging.dev/193669))
* [course-page 124182](https://serlo-staging.dev/124182) ([current revision 193672](https://serlo-staging.dev/193672))
* [course-page 85122](https://serlo-staging.dev/85122) ([current revision 193685](https://serlo-staging.dev/193685))
* [course-page 84484](https://serlo-staging.dev/84484) ([current revision 193686](https://serlo-staging.dev/193686))
* [article 51369](https://serlo-staging.dev/51369) ([current revision 193749](https://serlo-staging.dev/193749))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [article 193486](https://serlo-staging.dev/193486) ([current revision 193849](https://serlo-staging.dev/193849))
* [article 193464](https://serlo-staging.dev/193464) ([current revision 193855](https://serlo-staging.dev/193855))
* [article 192935](https://serlo-staging.dev/192935) ([current revision 193971](https://serlo-staging.dev/193971))
* [text-solution 60472](https://serlo-staging.dev/60472) ([current revision 194153](https://serlo-staging.dev/194153))
* [text-solution 30746](https://serlo-staging.dev/30746) ([current revision 194167](https://serlo-staging.dev/194167))
* [article 1889](https://serlo-staging.dev/1889) ([current revision 194292](https://serlo-staging.dev/194292))
* [article 2019](https://serlo-staging.dev/2019) ([current revision 194315](https://serlo-staging.dev/194315))
* [article 181058](https://serlo-staging.dev/181058) ([current revision 194425](https://serlo-staging.dev/194425))
* [text-solution 28689](https://serlo-staging.dev/28689) ([current revision 194576](https://serlo-staging.dev/194576))
* [text-solution 28746](https://serlo-staging.dev/28746) ([current revision 194580](https://serlo-staging.dev/194580))
* [text-solution 29259](https://serlo-staging.dev/29259) ([current revision 194587](https://serlo-staging.dev/194587))
* [text-solution 29262](https://serlo-staging.dev/29262) ([current revision 194589](https://serlo-staging.dev/194589))
* [text-solution 29275](https://serlo-staging.dev/29275) ([current revision 194591](https://serlo-staging.dev/194591))
* [text-solution 29294](https://serlo-staging.dev/29294) ([current revision 194593](https://serlo-staging.dev/194593))
* [course-page 178836](https://serlo-staging.dev/178836) ([current revision 194655](https://serlo-staging.dev/194655))
* [text-solution 194804](https://serlo-staging.dev/194804) ([current revision 194811](https://serlo-staging.dev/194811))
* [article 77792](https://serlo-staging.dev/77792) ([current revision 194907](https://serlo-staging.dev/194907))
* [article 154847](https://serlo-staging.dev/154847) ([current revision 195011](https://serlo-staging.dev/195011))
* [article 42924](https://serlo-staging.dev/42924) ([current revision 195124](https://serlo-staging.dev/195124))
* [text-solution 195102](https://serlo-staging.dev/195102) ([current revision 195146](https://serlo-staging.dev/195146))
* [text-solution 117429](https://serlo-staging.dev/117429) ([current revision 195266](https://serlo-staging.dev/195266))
* [text-solution 117453](https://serlo-staging.dev/117453) ([current revision 195506](https://serlo-staging.dev/195506))
* [article 193455](https://serlo-staging.dev/193455) ([current revision 195667](https://serlo-staging.dev/195667))
* [article 195560](https://serlo-staging.dev/195560) ([current revision 195823](https://serlo-staging.dev/195823))
* [article 1887](https://serlo-staging.dev/1887) ([current revision 195835](https://serlo-staging.dev/195835))
* [text-solution 194374](https://serlo-staging.dev/194374) ([current revision 196038](https://serlo-staging.dev/196038))
* [text-solution 194380](https://serlo-staging.dev/194380) ([current revision 196039](https://serlo-staging.dev/196039))
* [text-solution 194381](https://serlo-staging.dev/194381) ([current revision 196040](https://serlo-staging.dev/196040))
* [course-page 196165](https://serlo-staging.dev/196165) ([current revision 196172](https://serlo-staging.dev/196172))
* [course-page 196541](https://serlo-staging.dev/196541) ([current revision 196599](https://serlo-staging.dev/196599))
* [course-page 196542](https://serlo-staging.dev/196542) ([current revision 196600](https://serlo-staging.dev/196600))
* [course-page 195590](https://serlo-staging.dev/195590) ([current revision 196612](https://serlo-staging.dev/196612))
* [course-page 196547](https://serlo-staging.dev/196547) ([current revision 196657](https://serlo-staging.dev/196657))
* [article 1583](https://serlo-staging.dev/1583) ([current revision 196686](https://serlo-staging.dev/196686))
* [text-solution 195459](https://serlo-staging.dev/195459) ([current revision 196811](https://serlo-staging.dev/196811))
* [article 56795](https://serlo-staging.dev/56795) ([current revision 196827](https://serlo-staging.dev/196827))
* [course-page 52448](https://serlo-staging.dev/52448) ([current revision 196828](https://serlo-staging.dev/196828))
* [course-page 52411](https://serlo-staging.dev/52411) ([current revision 196831](https://serlo-staging.dev/196831))
* [article 170218](https://serlo-staging.dev/170218) ([current revision 196962](https://serlo-staging.dev/196962))
* [text-solution 111305](https://serlo-staging.dev/111305) ([current revision 197045](https://serlo-staging.dev/197045))
* [text-solution 111281](https://serlo-staging.dev/111281) ([current revision 197049](https://serlo-staging.dev/197049))
* [text-solution 11951](https://serlo-staging.dev/11951) ([current revision 197087](https://serlo-staging.dev/197087))
* [text-solution 11955](https://serlo-staging.dev/11955) ([current revision 197089](https://serlo-staging.dev/197089))
* [text-solution 11959](https://serlo-staging.dev/11959) ([current revision 197091](https://serlo-staging.dev/197091))
* [text-solution 11965](https://serlo-staging.dev/11965) ([current revision 197093](https://serlo-staging.dev/197093))
* [text-solution 2401](https://serlo-staging.dev/2401) ([current revision 197116](https://serlo-staging.dev/197116))
* [article 1655](https://serlo-staging.dev/1655) ([current revision 197179](https://serlo-staging.dev/197179))
* [article 109527](https://serlo-staging.dev/109527) ([current revision 197183](https://serlo-staging.dev/197183))
* [article 1981](https://serlo-staging.dev/1981) ([current revision 197192](https://serlo-staging.dev/197192))
* [article 1775](https://serlo-staging.dev/1775) ([current revision 197265](https://serlo-staging.dev/197265))
* [article 42091](https://serlo-staging.dev/42091) ([current revision 197307](https://serlo-staging.dev/197307))
* [article 42090](https://serlo-staging.dev/42090) ([current revision 197312](https://serlo-staging.dev/197312))
* [article 158205](https://serlo-staging.dev/158205) ([current revision 197316](https://serlo-staging.dev/197316))
* [article 53073](https://serlo-staging.dev/53073) ([current revision 197340](https://serlo-staging.dev/197340))
* [course-page 197205](https://serlo-staging.dev/197205) ([current revision 197351](https://serlo-staging.dev/197351))
* [article 129696](https://serlo-staging.dev/129696) ([current revision 197437](https://serlo-staging.dev/197437))
* [text-solution 8941](https://serlo-staging.dev/8941) ([current revision 197480](https://serlo-staging.dev/197480))
* [article 1699](https://serlo-staging.dev/1699) ([current revision 197695](https://serlo-staging.dev/197695))
* [article 1865](https://serlo-staging.dev/1865) ([current revision 197830](https://serlo-staging.dev/197830))
* [article 2097](https://serlo-staging.dev/2097) ([current revision 197832](https://serlo-staging.dev/197832))
* [course-page 35392](https://serlo-staging.dev/35392) ([current revision 197848](https://serlo-staging.dev/197848))
* [article 90410](https://serlo-staging.dev/90410) ([current revision 197881](https://serlo-staging.dev/197881))
* [text-exercise-group 67055](https://serlo-staging.dev/67055) ([current revision 197964](https://serlo-staging.dev/197964))
* [article 2107](https://serlo-staging.dev/2107) ([current revision 198108](https://serlo-staging.dev/198108))
* [article 198035](https://serlo-staging.dev/198035) ([current revision 198445](https://serlo-staging.dev/198445))
* [text-solution 137658](https://serlo-staging.dev/137658) ([current revision 198536](https://serlo-staging.dev/198536))
* [course-page 65011](https://serlo-staging.dev/65011) ([current revision 198938](https://serlo-staging.dev/198938))
* [course-page 65013](https://serlo-staging.dev/65013) ([current revision 198939](https://serlo-staging.dev/198939))
* [course-page 65078](https://serlo-staging.dev/65078) ([current revision 198940](https://serlo-staging.dev/198940))
* [text-solution 115597](https://serlo-staging.dev/115597) ([current revision 199058](https://serlo-staging.dev/199058))
* [article 198294](https://serlo-staging.dev/198294) ([current revision 199636](https://serlo-staging.dev/199636))
* [article 199273](https://serlo-staging.dev/199273) ([current revision 199664](https://serlo-staging.dev/199664))
* [article 199079](https://serlo-staging.dev/199079) ([current revision 199667](https://serlo-staging.dev/199667))
* [article 199872](https://serlo-staging.dev/199872) ([current revision 199879](https://serlo-staging.dev/199879))
* [text-solution 5017](https://serlo-staging.dev/5017) ([current revision 199983](https://serlo-staging.dev/199983))
* [text-solution 5133](https://serlo-staging.dev/5133) ([current revision 200000](https://serlo-staging.dev/200000))
* [course-page 72006](https://serlo-staging.dev/72006) ([current revision 200184](https://serlo-staging.dev/200184))
* [text-solution 7279](https://serlo-staging.dev/7279) ([current revision 200374](https://serlo-staging.dev/200374))
* [article 198965](https://serlo-staging.dev/198965) ([current revision 200409](https://serlo-staging.dev/200409))
* [text-solution 137744](https://serlo-staging.dev/137744) ([current revision 200512](https://serlo-staging.dev/200512))
* [text-solution 137775](https://serlo-staging.dev/137775) ([current revision 200566](https://serlo-staging.dev/200566))
* [text-solution 137883](https://serlo-staging.dev/137883) ([current revision 200568](https://serlo-staging.dev/200568))
* [text-solution 138170](https://serlo-staging.dev/138170) ([current revision 200569](https://serlo-staging.dev/200569))
* [text-solution 75799](https://serlo-staging.dev/75799) ([current revision 200901](https://serlo-staging.dev/200901))
* [text-solution 38203](https://serlo-staging.dev/38203) ([current revision 200913](https://serlo-staging.dev/200913))
* [text-solution 87290](https://serlo-staging.dev/87290) ([current revision 200914](https://serlo-staging.dev/200914))
* [text-solution 58205](https://serlo-staging.dev/58205) ([current revision 200941](https://serlo-staging.dev/200941))
* [text-solution 58634](https://serlo-staging.dev/58634) ([current revision 200950](https://serlo-staging.dev/200950))
* [text-solution 198996](https://serlo-staging.dev/198996) ([current revision 201219](https://serlo-staging.dev/201219))
* [article 201403](https://serlo-staging.dev/201403) ([current revision 201469](https://serlo-staging.dev/201469))
* [article 127465](https://serlo-staging.dev/127465) ([current revision 201501](https://serlo-staging.dev/201501))
* [text-solution 46698](https://serlo-staging.dev/46698) ([current revision 201613](https://serlo-staging.dev/201613))
* [text-solution 59366](https://serlo-staging.dev/59366) ([current revision 201651](https://serlo-staging.dev/201651))
* [text-solution 60386](https://serlo-staging.dev/60386) ([current revision 201656](https://serlo-staging.dev/201656))
* [text-solution 76146](https://serlo-staging.dev/76146) ([current revision 201711](https://serlo-staging.dev/201711))
* [text-solution 107891](https://serlo-staging.dev/107891) ([current revision 201733](https://serlo-staging.dev/201733))
* [text-solution 138088](https://serlo-staging.dev/138088) ([current revision 201753](https://serlo-staging.dev/201753))
* [text-solution 83320](https://serlo-staging.dev/83320) ([current revision 201778](https://serlo-staging.dev/201778))
* [text-solution 121927](https://serlo-staging.dev/121927) ([current revision 202079](https://serlo-staging.dev/202079))
* [article 186051](https://serlo-staging.dev/186051) ([current revision 202081](https://serlo-staging.dev/202081))
* [article 202130](https://serlo-staging.dev/202130) ([current revision 202132](https://serlo-staging.dev/202132))
* [text-solution 123037](https://serlo-staging.dev/123037) ([current revision 202142](https://serlo-staging.dev/202142))
* [text-solution 107881](https://serlo-staging.dev/107881) ([current revision 202145](https://serlo-staging.dev/202145))
* [article 1663](https://serlo-staging.dev/1663) ([current revision 202244](https://serlo-staging.dev/202244))
* [article 178250](https://serlo-staging.dev/178250) ([current revision 202862](https://serlo-staging.dev/202862))
* [article 185658](https://serlo-staging.dev/185658) ([current revision 202908](https://serlo-staging.dev/202908))
* [article 186816](https://serlo-staging.dev/186816) ([current revision 202909](https://serlo-staging.dev/202909))
* [article 185660](https://serlo-staging.dev/185660) ([current revision 202910](https://serlo-staging.dev/202910))
* [article 185662](https://serlo-staging.dev/185662) ([current revision 202913](https://serlo-staging.dev/202913))
* [article 185665](https://serlo-staging.dev/185665) ([current revision 202919](https://serlo-staging.dev/202919))
* [article 186160](https://serlo-staging.dev/186160) ([current revision 202929](https://serlo-staging.dev/202929))
* [article 185676](https://serlo-staging.dev/185676) ([current revision 202930](https://serlo-staging.dev/202930))
* [article 185668](https://serlo-staging.dev/185668) ([current revision 202931](https://serlo-staging.dev/202931))
* [article 185689](https://serlo-staging.dev/185689) ([current revision 202932](https://serlo-staging.dev/202932))
* [article 185671](https://serlo-staging.dev/185671) ([current revision 202933](https://serlo-staging.dev/202933))
* [article 186187](https://serlo-staging.dev/186187) ([current revision 202934](https://serlo-staging.dev/202934))
* [article 185667](https://serlo-staging.dev/185667) ([current revision 202935](https://serlo-staging.dev/202935))
* [article 2079](https://serlo-staging.dev/2079) ([current revision 203031](https://serlo-staging.dev/203031))
* [text-solution 21003](https://serlo-staging.dev/21003) ([current revision 203069](https://serlo-staging.dev/203069))
* [text-solution 21019](https://serlo-staging.dev/21019) ([current revision 203071](https://serlo-staging.dev/203071))
* [text-solution 21222](https://serlo-staging.dev/21222) ([current revision 203079](https://serlo-staging.dev/203079))
* [article 203263](https://serlo-staging.dev/203263) ([current revision 203265](https://serlo-staging.dev/203265))
* [article 203274](https://serlo-staging.dev/203274) ([current revision 203327](https://serlo-staging.dev/203327))
* [article 203329](https://serlo-staging.dev/203329) ([current revision 203337](https://serlo-staging.dev/203337))
* [text-exercise 123362](https://serlo-staging.dev/123362) ([current revision 203391](https://serlo-staging.dev/203391))
* [text-solution 144851](https://serlo-staging.dev/144851) ([current revision 203398](https://serlo-staging.dev/203398))
* [article 203664](https://serlo-staging.dev/203664) ([current revision 203669](https://serlo-staging.dev/203669))
* [text-solution 28568](https://serlo-staging.dev/28568) ([current revision 203806](https://serlo-staging.dev/203806))
* [text-solution 28587](https://serlo-staging.dev/28587) ([current revision 203808](https://serlo-staging.dev/203808))
* [course-page 204042](https://serlo-staging.dev/204042) ([current revision 204052](https://serlo-staging.dev/204052))
* [course-page 204047](https://serlo-staging.dev/204047) ([current revision 204057](https://serlo-staging.dev/204057))
* [course-page 168442](https://serlo-staging.dev/168442) ([current revision 204061](https://serlo-staging.dev/204061))
* [course-page 199317](https://serlo-staging.dev/199317) ([current revision 204062](https://serlo-staging.dev/204062))
* [article 202720](https://serlo-staging.dev/202720) ([current revision 204126](https://serlo-staging.dev/204126))
* [article 192457](https://serlo-staging.dev/192457) ([current revision 204129](https://serlo-staging.dev/204129))
* [article 184341](https://serlo-staging.dev/184341) ([current revision 204156](https://serlo-staging.dev/204156))
* [article 150885](https://serlo-staging.dev/150885) ([current revision 204163](https://serlo-staging.dev/204163))
* [article 140479](https://serlo-staging.dev/140479) ([current revision 204194](https://serlo-staging.dev/204194))
* [course-page 199305](https://serlo-staging.dev/199305) ([current revision 204288](https://serlo-staging.dev/204288))
* [text-exercise 101383](https://serlo-staging.dev/101383) ([current revision 204315](https://serlo-staging.dev/204315))
* [course-page 204723](https://serlo-staging.dev/204723) ([current revision 204733](https://serlo-staging.dev/204733))
* [article 74686](https://serlo-staging.dev/74686) ([current revision 204838](https://serlo-staging.dev/204838))
* [course-page 205023](https://serlo-staging.dev/205023) ([current revision 205031](https://serlo-staging.dev/205031))
* [article 205033](https://serlo-staging.dev/205033) ([current revision 205036](https://serlo-staging.dev/205036))
* [article 205046](https://serlo-staging.dev/205046) ([current revision 205051](https://serlo-staging.dev/205051))
* [text-solution 62551](https://serlo-staging.dev/62551) ([current revision 205606](https://serlo-staging.dev/205606))
* [course-page 136302](https://serlo-staging.dev/136302) ([current revision 205704](https://serlo-staging.dev/205704))
* [article 203837](https://serlo-staging.dev/203837) ([current revision 205928](https://serlo-staging.dev/205928))
* [course-page 206221](https://serlo-staging.dev/206221) ([current revision 206224](https://serlo-staging.dev/206224))
* [course-page 206248](https://serlo-staging.dev/206248) ([current revision 206256](https://serlo-staging.dev/206256))
* [course-page 206249](https://serlo-staging.dev/206249) ([current revision 206257](https://serlo-staging.dev/206257))
* [course-page 206369](https://serlo-staging.dev/206369) ([current revision 206377](https://serlo-staging.dev/206377))
* [course-page 206371](https://serlo-staging.dev/206371) ([current revision 206379](https://serlo-staging.dev/206379))
* [text-solution 11029](https://serlo-staging.dev/11029) ([current revision 206891](https://serlo-staging.dev/206891))
* [text-solution 11545](https://serlo-staging.dev/11545) ([current revision 206892](https://serlo-staging.dev/206892))
* [course-page 204744](https://serlo-staging.dev/204744) ([current revision 206899](https://serlo-staging.dev/206899))
* [course-page 206406](https://serlo-staging.dev/206406) ([current revision 206903](https://serlo-staging.dev/206903))
* [text-solution 11171](https://serlo-staging.dev/11171) ([current revision 206958](https://serlo-staging.dev/206958))
* [text-solution 29237](https://serlo-staging.dev/29237) ([current revision 206991](https://serlo-staging.dev/206991))
* [course-page 207390](https://serlo-staging.dev/207390) ([current revision 207399](https://serlo-staging.dev/207399))
* [course-page 52710](https://serlo-staging.dev/52710) ([current revision 207420](https://serlo-staging.dev/207420))
* [text-solution 207574](https://serlo-staging.dev/207574) ([current revision 207579](https://serlo-staging.dev/207579))
* [text-exercise-group 207852](https://serlo-staging.dev/207852) ([current revision 207871](https://serlo-staging.dev/207871))
* [course-page 203725](https://serlo-staging.dev/203725) ([current revision 207981](https://serlo-staging.dev/207981))
* [course-page 203731](https://serlo-staging.dev/203731) ([current revision 207994](https://serlo-staging.dev/207994))
* [article 90416](https://serlo-staging.dev/90416) ([current revision 208006](https://serlo-staging.dev/208006))
* [text-solution 27179](https://serlo-staging.dev/27179) ([current revision 208055](https://serlo-staging.dev/208055))
* [course-page 207315](https://serlo-staging.dev/207315) ([current revision 208239](https://serlo-staging.dev/208239))
* [text-solution 207492](https://serlo-staging.dev/207492) ([current revision 208303](https://serlo-staging.dev/208303))
* [article 2005](https://serlo-staging.dev/2005) ([current revision 208323](https://serlo-staging.dev/208323))
* [course-page 208365](https://serlo-staging.dev/208365) ([current revision 208366](https://serlo-staging.dev/208366))
* [course-page 208367](https://serlo-staging.dev/208367) ([current revision 208368](https://serlo-staging.dev/208368))
* [course-page 208369](https://serlo-staging.dev/208369) ([current revision 208370](https://serlo-staging.dev/208370))
* [course-page 208371](https://serlo-staging.dev/208371) ([current revision 208372](https://serlo-staging.dev/208372))
* [course-page 208374](https://serlo-staging.dev/208374) ([current revision 208375](https://serlo-staging.dev/208375))
* [course-page 208376](https://serlo-staging.dev/208376) ([current revision 208378](https://serlo-staging.dev/208378))
* [course-page 208379](https://serlo-staging.dev/208379) ([current revision 208380](https://serlo-staging.dev/208380))
* [course-page 208381](https://serlo-staging.dev/208381) ([current revision 208382](https://serlo-staging.dev/208382))
* [course-page 208383](https://serlo-staging.dev/208383) ([current revision 208384](https://serlo-staging.dev/208384))
* [course-page 208385](https://serlo-staging.dev/208385) ([current revision 208386](https://serlo-staging.dev/208386))
* [course-page 208387](https://serlo-staging.dev/208387) ([current revision 208388](https://serlo-staging.dev/208388))
* [course-page 208389](https://serlo-staging.dev/208389) ([current revision 208390](https://serlo-staging.dev/208390))
* [course-page 208391](https://serlo-staging.dev/208391) ([current revision 208392](https://serlo-staging.dev/208392))
* [text-solution 111340](https://serlo-staging.dev/111340) ([current revision 208923](https://serlo-staging.dev/208923))
* [text-solution 32251](https://serlo-staging.dev/32251) ([current revision 208978](https://serlo-staging.dev/208978))
* [text-solution 32254](https://serlo-staging.dev/32254) ([current revision 208980](https://serlo-staging.dev/208980))
* [text-solution 32256](https://serlo-staging.dev/32256) ([current revision 208982](https://serlo-staging.dev/208982))
* [text-solution 209068](https://serlo-staging.dev/209068) ([current revision 209074](https://serlo-staging.dev/209074))
* [article 150648](https://serlo-staging.dev/150648) ([current revision 209138](https://serlo-staging.dev/209138))
* [course-page 207350](https://serlo-staging.dev/207350) ([current revision 209241](https://serlo-staging.dev/209241))
* [course-page 207351](https://serlo-staging.dev/207351) ([current revision 209248](https://serlo-staging.dev/209248))
* [course-page 207352](https://serlo-staging.dev/207352) ([current revision 209249](https://serlo-staging.dev/209249))
* [article 209220](https://serlo-staging.dev/209220) ([current revision 209510](https://serlo-staging.dev/209510))
* [text-solution 209075](https://serlo-staging.dev/209075) ([current revision 209555](https://serlo-staging.dev/209555))
* [course-page 206247](https://serlo-staging.dev/206247) ([current revision 209748](https://serlo-staging.dev/209748))
* [article 210007](https://serlo-staging.dev/210007) ([current revision 210013](https://serlo-staging.dev/210013))
* [course-page 210085](https://serlo-staging.dev/210085) ([current revision 210091](https://serlo-staging.dev/210091))
* [course-page 210266](https://serlo-staging.dev/210266) ([current revision 210276](https://serlo-staging.dev/210276))
* [course-page 210284](https://serlo-staging.dev/210284) ([current revision 210285](https://serlo-staging.dev/210285))
* [text-solution 210298](https://serlo-staging.dev/210298) ([current revision 210300](https://serlo-staging.dev/210300))
* [text-solution 210555](https://serlo-staging.dev/210555) ([current revision 210560](https://serlo-staging.dev/210560))
* [text-solution 210569](https://serlo-staging.dev/210569) ([current revision 210574](https://serlo-staging.dev/210574))
* [course-page 206250](https://serlo-staging.dev/206250) ([current revision 210612](https://serlo-staging.dev/210612))
* [course-page 210682](https://serlo-staging.dev/210682) ([current revision 210685](https://serlo-staging.dev/210685))
* [course-page 210688](https://serlo-staging.dev/210688) ([current revision 210691](https://serlo-staging.dev/210691))
* [course-page 210697](https://serlo-staging.dev/210697) ([current revision 210705](https://serlo-staging.dev/210705))
* [course-page 210702](https://serlo-staging.dev/210702) ([current revision 210710](https://serlo-staging.dev/210710))
* [course-page 210713](https://serlo-staging.dev/210713) ([current revision 210721](https://serlo-staging.dev/210721))
* [course-page 210718](https://serlo-staging.dev/210718) ([current revision 210726](https://serlo-staging.dev/210726))
* [course-page 210729](https://serlo-staging.dev/210729) ([current revision 210737](https://serlo-staging.dev/210737))
* [course-page 210734](https://serlo-staging.dev/210734) ([current revision 210742](https://serlo-staging.dev/210742))
* [course-page 210759](https://serlo-staging.dev/210759) ([current revision 210769](https://serlo-staging.dev/210769))
* [course-page 210760](https://serlo-staging.dev/210760) ([current revision 210770](https://serlo-staging.dev/210770))
* [course-page 210764](https://serlo-staging.dev/210764) ([current revision 210774](https://serlo-staging.dev/210774))
* [course-page 210782](https://serlo-staging.dev/210782) ([current revision 210792](https://serlo-staging.dev/210792))
* [course-page 210783](https://serlo-staging.dev/210783) ([current revision 210793](https://serlo-staging.dev/210793))
* [course-page 210784](https://serlo-staging.dev/210784) ([current revision 210794](https://serlo-staging.dev/210794))
* [course-page 210785](https://serlo-staging.dev/210785) ([current revision 210795](https://serlo-staging.dev/210795))
* [course-page 210786](https://serlo-staging.dev/210786) ([current revision 210796](https://serlo-staging.dev/210796))
* [course-page 210800](https://serlo-staging.dev/210800) ([current revision 210808](https://serlo-staging.dev/210808))
* [course-page 210801](https://serlo-staging.dev/210801) ([current revision 210809](https://serlo-staging.dev/210809))
* [course-page 210802](https://serlo-staging.dev/210802) ([current revision 210810](https://serlo-staging.dev/210810))
* [course-page 210803](https://serlo-staging.dev/210803) ([current revision 210811](https://serlo-staging.dev/210811))
* [course-page 210804](https://serlo-staging.dev/210804) ([current revision 210812](https://serlo-staging.dev/210812))
* [text-solution 210840](https://serlo-staging.dev/210840) ([current revision 210845](https://serlo-staging.dev/210845))
* [text-solution 210853](https://serlo-staging.dev/210853) ([current revision 210862](https://serlo-staging.dev/210862))
* [text-solution 210855](https://serlo-staging.dev/210855) ([current revision 210864](https://serlo-staging.dev/210864))
* [text-solution 210870](https://serlo-staging.dev/210870) ([current revision 210878](https://serlo-staging.dev/210878))
* [text-solution 210872](https://serlo-staging.dev/210872) ([current revision 210880](https://serlo-staging.dev/210880))
* [article 211023](https://serlo-staging.dev/211023) ([current revision 211039](https://serlo-staging.dev/211039))
* [course-page 210100](https://serlo-staging.dev/210100) ([current revision 211044](https://serlo-staging.dev/211044))
* [course-page 210101](https://serlo-staging.dev/210101) ([current revision 211045](https://serlo-staging.dev/211045))
* [course-page 210102](https://serlo-staging.dev/210102) ([current revision 211046](https://serlo-staging.dev/211046))
* [course-page 210103](https://serlo-staging.dev/210103) ([current revision 211047](https://serlo-staging.dev/211047))
* [text-exercise-group 61580](https://serlo-staging.dev/61580) ([current revision 211092](https://serlo-staging.dev/211092))
* [grouped-text-exercise 62033](https://serlo-staging.dev/62033) ([current revision 211096](https://serlo-staging.dev/211096))
* [text-solution 144606](https://serlo-staging.dev/144606) ([current revision 211290](https://serlo-staging.dev/211290))
* [text-solution 144653](https://serlo-staging.dev/144653) ([current revision 211291](https://serlo-staging.dev/211291))
* [text-solution 209845](https://serlo-staging.dev/209845) ([current revision 211370](https://serlo-staging.dev/211370))
* [text-solution 209857](https://serlo-staging.dev/209857) ([current revision 211384](https://serlo-staging.dev/211384))
* [text-solution 209908](https://serlo-staging.dev/209908) ([current revision 211386](https://serlo-staging.dev/211386))
* [text-solution 209917](https://serlo-staging.dev/209917) ([current revision 211438](https://serlo-staging.dev/211438))
* [text-solution 209924](https://serlo-staging.dev/209924) ([current revision 211450](https://serlo-staging.dev/211450))
* [text-solution 210041](https://serlo-staging.dev/210041) ([current revision 211512](https://serlo-staging.dev/211512))
* [text-solution 210046](https://serlo-staging.dev/210046) ([current revision 211540](https://serlo-staging.dev/211540))
* [text-solution 210051](https://serlo-staging.dev/210051) ([current revision 211627](https://serlo-staging.dev/211627))
* [text-solution 210164](https://serlo-staging.dev/210164) ([current revision 211637](https://serlo-staging.dev/211637))
* [text-solution 211873](https://serlo-staging.dev/211873) ([current revision 211875](https://serlo-staging.dev/211875))
* [text-solution 211896](https://serlo-staging.dev/211896) ([current revision 211898](https://serlo-staging.dev/211898))
* [article 31700](https://serlo-staging.dev/31700) ([current revision 211913](https://serlo-staging.dev/211913))
* [text-solution 62723](https://serlo-staging.dev/62723) ([current revision 212027](https://serlo-staging.dev/212027))
* [text-solution 31544](https://serlo-staging.dev/31544) ([current revision 212079](https://serlo-staging.dev/212079))
* [text-solution 48170](https://serlo-staging.dev/48170) ([current revision 212133](https://serlo-staging.dev/212133))
* [text-solution 48172](https://serlo-staging.dev/48172) ([current revision 212135](https://serlo-staging.dev/212135))
* [text-solution 48175](https://serlo-staging.dev/48175) ([current revision 212137](https://serlo-staging.dev/212137))
* [article 181051](https://serlo-staging.dev/181051) ([current revision 212149](https://serlo-staging.dev/212149))
* [article 209448](https://serlo-staging.dev/209448) ([current revision 212180](https://serlo-staging.dev/212180))
* [course-page 62618](https://serlo-staging.dev/62618) ([current revision 212185](https://serlo-staging.dev/212185))
* [article 1973](https://serlo-staging.dev/1973) ([current revision 212191](https://serlo-staging.dev/212191))
* [course-page 212168](https://serlo-staging.dev/212168) ([current revision 212210](https://serlo-staging.dev/212210))
* [course-page 212217](https://serlo-staging.dev/212217) ([current revision 212226](https://serlo-staging.dev/212226))
* [article 149480](https://serlo-staging.dev/149480) ([current revision 212417](https://serlo-staging.dev/212417))
* [article 1909](https://serlo-staging.dev/1909) ([current revision 212567](https://serlo-staging.dev/212567))
* [article 165120](https://serlo-staging.dev/165120) ([current revision 212598](https://serlo-staging.dev/212598))
* [text-solution 212186](https://serlo-staging.dev/212186) ([current revision 212621](https://serlo-staging.dev/212621))
* [article 212604](https://serlo-staging.dev/212604) ([current revision 212636](https://serlo-staging.dev/212636))
* [article 1601](https://serlo-staging.dev/1601) ([current revision 213084](https://serlo-staging.dev/213084))
* [text-solution 205768](https://serlo-staging.dev/205768) ([current revision 213143](https://serlo-staging.dev/213143))
* [text-solution 205770](https://serlo-staging.dev/205770) ([current revision 213145](https://serlo-staging.dev/213145))
* [text-solution 213507](https://serlo-staging.dev/213507) ([current revision 213512](https://serlo-staging.dev/213512))
* [text-solution 213518](https://serlo-staging.dev/213518) ([current revision 213523](https://serlo-staging.dev/213523))
* [article 210777](https://serlo-staging.dev/210777) ([current revision 213803](https://serlo-staging.dev/213803))
* [article 201607](https://serlo-staging.dev/201607) ([current revision 213809](https://serlo-staging.dev/213809))
* [article 206622](https://serlo-staging.dev/206622) ([current revision 213832](https://serlo-staging.dev/213832))
* [article 201974](https://serlo-staging.dev/201974) ([current revision 213833](https://serlo-staging.dev/213833))
* [article 206637](https://serlo-staging.dev/206637) ([current revision 213835](https://serlo-staging.dev/213835))
* [text-solution 214007](https://serlo-staging.dev/214007) ([current revision 214014](https://serlo-staging.dev/214014))
* [text-solution 214083](https://serlo-staging.dev/214083) ([current revision 214088](https://serlo-staging.dev/214088))
* [text-exercise-group 61396](https://serlo-staging.dev/61396) ([current revision 214089](https://serlo-staging.dev/214089))
* [article 48121](https://serlo-staging.dev/48121) ([current revision 214107](https://serlo-staging.dev/214107))
* [text-solution 214113](https://serlo-staging.dev/214113) ([current revision 214120](https://serlo-staging.dev/214120))
* [text-solution 206704](https://serlo-staging.dev/206704) ([current revision 214405](https://serlo-staging.dev/214405))
* [text-solution 206706](https://serlo-staging.dev/206706) ([current revision 214406](https://serlo-staging.dev/214406))
* [text-solution 206708](https://serlo-staging.dev/206708) ([current revision 214407](https://serlo-staging.dev/214407))
* [course-page 212169](https://serlo-staging.dev/212169) ([current revision 214566](https://serlo-staging.dev/214566))
* [text-solution 214593](https://serlo-staging.dev/214593) ([current revision 214600](https://serlo-staging.dev/214600))
* [course-page 97763](https://serlo-staging.dev/97763) ([current revision 214847](https://serlo-staging.dev/214847))
* [text-solution 214759](https://serlo-staging.dev/214759) ([current revision 214880](https://serlo-staging.dev/214880))
* [article 25369](https://serlo-staging.dev/25369) ([current revision 214983](https://serlo-staging.dev/214983))
* [text-solution 214794](https://serlo-staging.dev/214794) ([current revision 214988](https://serlo-staging.dev/214988))
* [course-page 215525](https://serlo-staging.dev/215525) ([current revision 215531](https://serlo-staging.dev/215531))
* [course-page 215526](https://serlo-staging.dev/215526) ([current revision 215532](https://serlo-staging.dev/215532))
* [course-page 215527](https://serlo-staging.dev/215527) ([current revision 215533](https://serlo-staging.dev/215533))
* [article 1755](https://serlo-staging.dev/1755) ([current revision 215574](https://serlo-staging.dev/215574))
* [text-solution 215103](https://serlo-staging.dev/215103) ([current revision 215630](https://serlo-staging.dev/215630))
* [article 2049](https://serlo-staging.dev/2049) ([current revision 215704](https://serlo-staging.dev/215704))
* [article 215748](https://serlo-staging.dev/215748) ([current revision 215756](https://serlo-staging.dev/215756))
* [course-page 215762](https://serlo-staging.dev/215762) ([current revision 215764](https://serlo-staging.dev/215764))
* [event 147713](https://serlo-staging.dev/147713) ([current revision 215780](https://serlo-staging.dev/215780))
* [course-page 215847](https://serlo-staging.dev/215847) ([current revision 215852](https://serlo-staging.dev/215852))
* [course-page 215850](https://serlo-staging.dev/215850) ([current revision 215853](https://serlo-staging.dev/215853))
* [article 1785](https://serlo-staging.dev/1785) ([current revision 215882](https://serlo-staging.dev/215882))
* [article 1797](https://serlo-staging.dev/1797) ([current revision 215934](https://serlo-staging.dev/215934))
* [article 214242](https://serlo-staging.dev/214242) ([current revision 216186](https://serlo-staging.dev/216186))
* [course-page 216198](https://serlo-staging.dev/216198) ([current revision 216222](https://serlo-staging.dev/216222))
* [course-page 216199](https://serlo-staging.dev/216199) ([current revision 216223](https://serlo-staging.dev/216223))
* [course-page 216201](https://serlo-staging.dev/216201) ([current revision 216296](https://serlo-staging.dev/216296))
* [text-exercise-group 210826](https://serlo-staging.dev/210826) ([current revision 216353](https://serlo-staging.dev/216353))
* [course-page 216398](https://serlo-staging.dev/216398) ([current revision 216405](https://serlo-staging.dev/216405))
* [course-page 216394](https://serlo-staging.dev/216394) ([current revision 216407](https://serlo-staging.dev/216407))
* [course-page 216396](https://serlo-staging.dev/216396) ([current revision 216409](https://serlo-staging.dev/216409))
* [course-page 216397](https://serlo-staging.dev/216397) ([current revision 216410](https://serlo-staging.dev/216410))
* [course-page 206125](https://serlo-staging.dev/206125) ([current revision 216451](https://serlo-staging.dev/216451))
* [course-page 206123](https://serlo-staging.dev/206123) ([current revision 216455](https://serlo-staging.dev/216455))
* [course-page 206216](https://serlo-staging.dev/206216) ([current revision 216481](https://serlo-staging.dev/216481))
* [article 215043](https://serlo-staging.dev/215043) ([current revision 216483](https://serlo-staging.dev/216483))
* [article 216520](https://serlo-staging.dev/216520) ([current revision 216523](https://serlo-staging.dev/216523))
* [article 216524](https://serlo-staging.dev/216524) ([current revision 216526](https://serlo-staging.dev/216526))
* [article 200275](https://serlo-staging.dev/200275) ([current revision 216549](https://serlo-staging.dev/216549))
* [article 216917](https://serlo-staging.dev/216917) ([current revision 216921](https://serlo-staging.dev/216921))
* [text-solution 214540](https://serlo-staging.dev/214540) ([current revision 216950](https://serlo-staging.dev/216950))
* [text-solution 217033](https://serlo-staging.dev/217033) ([current revision 217038](https://serlo-staging.dev/217038))
* [text-solution 216925](https://serlo-staging.dev/216925) ([current revision 217165](https://serlo-staging.dev/217165))
* [text-solution 217261](https://serlo-staging.dev/217261) ([current revision 217266](https://serlo-staging.dev/217266))
* [course-page 209964](https://serlo-staging.dev/209964) ([current revision 217517](https://serlo-staging.dev/217517))
* [text-solution 216822](https://serlo-staging.dev/216822) ([current revision 217860](https://serlo-staging.dev/217860))
* [article 178142](https://serlo-staging.dev/178142) ([current revision 218038](https://serlo-staging.dev/218038))
* [text-solution 218065](https://serlo-staging.dev/218065) ([current revision 218076](https://serlo-staging.dev/218076))
* [article 139990](https://serlo-staging.dev/139990) ([current revision 218468](https://serlo-staging.dev/218468))
* [text-solution 210136](https://serlo-staging.dev/210136) ([current revision 218521](https://serlo-staging.dev/218521))
* [text-solution 217647](https://serlo-staging.dev/217647) ([current revision 218673](https://serlo-staging.dev/218673))
* [text-solution 217649](https://serlo-staging.dev/217649) ([current revision 218674](https://serlo-staging.dev/218674))
* [text-solution 217674](https://serlo-staging.dev/217674) ([current revision 218684](https://serlo-staging.dev/218684))
* [text-solution 217690](https://serlo-staging.dev/217690) ([current revision 218686](https://serlo-staging.dev/218686))
* [text-solution 217909](https://serlo-staging.dev/217909) ([current revision 218707](https://serlo-staging.dev/218707))
* [text-solution 217952](https://serlo-staging.dev/217952) ([current revision 218724](https://serlo-staging.dev/218724))
* [text-solution 217965](https://serlo-staging.dev/217965) ([current revision 218746](https://serlo-staging.dev/218746))
* [text-solution 218061](https://serlo-staging.dev/218061) ([current revision 218762](https://serlo-staging.dev/218762))
* [text-solution 218083](https://serlo-staging.dev/218083) ([current revision 218767](https://serlo-staging.dev/218767))
* [text-solution 218089](https://serlo-staging.dev/218089) ([current revision 218771](https://serlo-staging.dev/218771))
* [text-solution 218399](https://serlo-staging.dev/218399) ([current revision 218941](https://serlo-staging.dev/218941))
* [text-solution 218728](https://serlo-staging.dev/218728) ([current revision 219053](https://serlo-staging.dev/219053))
* [text-solution 218732](https://serlo-staging.dev/218732) ([current revision 219055](https://serlo-staging.dev/219055))
* [text-solution 218734](https://serlo-staging.dev/218734) ([current revision 219056](https://serlo-staging.dev/219056))
* [text-solution 175168](https://serlo-staging.dev/175168) ([current revision 219116](https://serlo-staging.dev/219116))
* [course-page 215528](https://serlo-staging.dev/215528) ([current revision 219203](https://serlo-staging.dev/219203))
* [text-solution 219227](https://serlo-staging.dev/219227) ([current revision 219232](https://serlo-staging.dev/219232))
* [text-solution 219229](https://serlo-staging.dev/219229) ([current revision 219234](https://serlo-staging.dev/219234))
* [text-solution 219241](https://serlo-staging.dev/219241) ([current revision 219250](https://serlo-staging.dev/219250))
* [text-solution 219353](https://serlo-staging.dev/219353) ([current revision 219366](https://serlo-staging.dev/219366))
* [text-solution 219355](https://serlo-staging.dev/219355) ([current revision 219368](https://serlo-staging.dev/219368))
* [text-solution 219357](https://serlo-staging.dev/219357) ([current revision 219370](https://serlo-staging.dev/219370))
* [text-solution 219359](https://serlo-staging.dev/219359) ([current revision 219372](https://serlo-staging.dev/219372))
* [text-solution 219361](https://serlo-staging.dev/219361) ([current revision 219374](https://serlo-staging.dev/219374))
* [text-solution 219406](https://serlo-staging.dev/219406) ([current revision 219408](https://serlo-staging.dev/219408))
* [text-solution 219424](https://serlo-staging.dev/219424) ([current revision 219426](https://serlo-staging.dev/219426))
* [text-solution 219434](https://serlo-staging.dev/219434) ([current revision 219436](https://serlo-staging.dev/219436))
* [text-solution 218697](https://serlo-staging.dev/218697) ([current revision 219438](https://serlo-staging.dev/219438))
* [text-solution 219439](https://serlo-staging.dev/219439) ([current revision 219441](https://serlo-staging.dev/219441))
* [text-solution 219351](https://serlo-staging.dev/219351) ([current revision 219489](https://serlo-staging.dev/219489))
* [course-page 218989](https://serlo-staging.dev/218989) ([current revision 219591](https://serlo-staging.dev/219591))
* [text-solution 94370](https://serlo-staging.dev/94370) ([current revision 219640](https://serlo-staging.dev/219640))
* [text-solution 159554](https://serlo-staging.dev/159554) ([current revision 219642](https://serlo-staging.dev/219642))
* [text-solution 159557](https://serlo-staging.dev/159557) ([current revision 219644](https://serlo-staging.dev/219644))
* [text-solution 159560](https://serlo-staging.dev/159560) ([current revision 219647](https://serlo-staging.dev/219647))
* [text-solution 178725](https://serlo-staging.dev/178725) ([current revision 219650](https://serlo-staging.dev/219650))
* [text-solution 178757](https://serlo-staging.dev/178757) ([current revision 219653](https://serlo-staging.dev/219653))
* [text-solution 175170](https://serlo-staging.dev/175170) ([current revision 219671](https://serlo-staging.dev/219671))
* [text-solution 175176](https://serlo-staging.dev/175176) ([current revision 219674](https://serlo-staging.dev/219674))
* [text-solution 175179](https://serlo-staging.dev/175179) ([current revision 219677](https://serlo-staging.dev/219677))
* [text-solution 179054](https://serlo-staging.dev/179054) ([current revision 219679](https://serlo-staging.dev/219679))
* [text-solution 179065](https://serlo-staging.dev/179065) ([current revision 219681](https://serlo-staging.dev/219681))
* [text-solution 180563](https://serlo-staging.dev/180563) ([current revision 219688](https://serlo-staging.dev/219688))
* [text-solution 179157](https://serlo-staging.dev/179157) ([current revision 219692](https://serlo-staging.dev/219692))
* [text-solution 176842](https://serlo-staging.dev/176842) ([current revision 219743](https://serlo-staging.dev/219743))
* [text-solution 177111](https://serlo-staging.dev/177111) ([current revision 219745](https://serlo-staging.dev/219745))
* [text-solution 178678](https://serlo-staging.dev/178678) ([current revision 219747](https://serlo-staging.dev/219747))
* [text-solution 178680](https://serlo-staging.dev/178680) ([current revision 219749](https://serlo-staging.dev/219749))
* [text-solution 178710](https://serlo-staging.dev/178710) ([current revision 219752](https://serlo-staging.dev/219752))
* [text-solution 178715](https://serlo-staging.dev/178715) ([current revision 219755](https://serlo-staging.dev/219755))
* [text-solution 175556](https://serlo-staging.dev/175556) ([current revision 219758](https://serlo-staging.dev/219758))
* [text-solution 175218](https://serlo-staging.dev/175218) ([current revision 219760](https://serlo-staging.dev/219760))
* [text-solution 213368](https://serlo-staging.dev/213368) ([current revision 219763](https://serlo-staging.dev/219763))
* [text-solution 213372](https://serlo-staging.dev/213372) ([current revision 219766](https://serlo-staging.dev/219766))
* [text-solution 213376](https://serlo-staging.dev/213376) ([current revision 219777](https://serlo-staging.dev/219777))
* [text-solution 213380](https://serlo-staging.dev/213380) ([current revision 219780](https://serlo-staging.dev/219780))
* [text-solution 213383](https://serlo-staging.dev/213383) ([current revision 219782](https://serlo-staging.dev/219782))
* [article 54253](https://serlo-staging.dev/54253) ([current revision 219826](https://serlo-staging.dev/219826))
* [article 193439](https://serlo-staging.dev/193439) ([current revision 219926](https://serlo-staging.dev/219926))
* [text-solution 178763](https://serlo-staging.dev/178763) ([current revision 220079](https://serlo-staging.dev/220079))
* [text-solution 177763](https://serlo-staging.dev/177763) ([current revision 220081](https://serlo-staging.dev/220081))
* [text-solution 178929](https://serlo-staging.dev/178929) ([current revision 220082](https://serlo-staging.dev/220082))
* [text-solution 178783](https://serlo-staging.dev/178783) ([current revision 220083](https://serlo-staging.dev/220083))
* [text-solution 177366](https://serlo-staging.dev/177366) ([current revision 220085](https://serlo-staging.dev/220085))
* [article 220154](https://serlo-staging.dev/220154) ([current revision 220213](https://serlo-staging.dev/220213))
* [article 205995](https://serlo-staging.dev/205995) ([current revision 220253](https://serlo-staging.dev/220253))
* [article 116795](https://serlo-staging.dev/116795) ([current revision 220388](https://serlo-staging.dev/220388))
* [article 214672](https://serlo-staging.dev/214672) ([current revision 220389](https://serlo-staging.dev/220389))
* [article 169056](https://serlo-staging.dev/169056) ([current revision 220459](https://serlo-staging.dev/220459))
* [article 220146](https://serlo-staging.dev/220146) ([current revision 220571](https://serlo-staging.dev/220571))
* [text-solution 178959](https://serlo-staging.dev/178959) ([current revision 220678](https://serlo-staging.dev/220678))
* [article 86467](https://serlo-staging.dev/86467) ([current revision 220723](https://serlo-staging.dev/220723))
* [article 148759](https://serlo-staging.dev/148759) ([current revision 220724](https://serlo-staging.dev/220724))
* [article 146609](https://serlo-staging.dev/146609) ([current revision 220725](https://serlo-staging.dev/220725))
* [article 2201](https://serlo-staging.dev/2201) ([current revision 220771](https://serlo-staging.dev/220771))
* [text-solution 9731](https://serlo-staging.dev/9731) ([current revision 220795](https://serlo-staging.dev/220795))
* [article 146611](https://serlo-staging.dev/146611) ([current revision 220829](https://serlo-staging.dev/220829))
* [article 146610](https://serlo-staging.dev/146610) ([current revision 220955](https://serlo-staging.dev/220955))
* [article 220880](https://serlo-staging.dev/220880) ([current revision 220958](https://serlo-staging.dev/220958))
* [text-solution 214635](https://serlo-staging.dev/214635) ([current revision 221278](https://serlo-staging.dev/221278))
* [text-solution 214637](https://serlo-staging.dev/214637) ([current revision 221279](https://serlo-staging.dev/221279))
* [article 1821](https://serlo-staging.dev/1821) ([current revision 221534](https://serlo-staging.dev/221534))
* [article 155971](https://serlo-staging.dev/155971) ([current revision 221538](https://serlo-staging.dev/221538))
* [article 179161](https://serlo-staging.dev/179161) ([current revision 221546](https://serlo-staging.dev/221546))
* [article 178355](https://serlo-staging.dev/178355) ([current revision 221691](https://serlo-staging.dev/221691))
* [course-page 141339](https://serlo-staging.dev/141339) ([current revision 221993](https://serlo-staging.dev/221993))
* [course-page 133596](https://serlo-staging.dev/133596) ([current revision 222011](https://serlo-staging.dev/222011))
* [course-page 141347](https://serlo-staging.dev/141347) ([current revision 222141](https://serlo-staging.dev/222141))
* [course-page 141349](https://serlo-staging.dev/141349) ([current revision 222142](https://serlo-staging.dev/222142))
* [course-page 136399](https://serlo-staging.dev/136399) ([current revision 222176](https://serlo-staging.dev/222176))
* [course-page 132723](https://serlo-staging.dev/132723) ([current revision 222178](https://serlo-staging.dev/222178))
* [article 166757](https://serlo-staging.dev/166757) ([current revision 222326](https://serlo-staging.dev/222326))
* [article 146964](https://serlo-staging.dev/146964) ([current revision 222329](https://serlo-staging.dev/222329))
* [article 180846](https://serlo-staging.dev/180846) ([current revision 222331](https://serlo-staging.dev/222331))
* [article 157071](https://serlo-staging.dev/157071) ([current revision 222498](https://serlo-staging.dev/222498))
* [article 51490](https://serlo-staging.dev/51490) ([current revision 222668](https://serlo-staging.dev/222668))
* [course-page 136293](https://serlo-staging.dev/136293) ([current revision 222680](https://serlo-staging.dev/222680))
* [article 1835](https://serlo-staging.dev/1835) ([current revision 222710](https://serlo-staging.dev/222710))
* [article 222052](https://serlo-staging.dev/222052) ([current revision 222720](https://serlo-staging.dev/222720))
* [article 222470](https://serlo-staging.dev/222470) ([current revision 222734](https://serlo-staging.dev/222734))
* [article 222049](https://serlo-staging.dev/222049) ([current revision 222739](https://serlo-staging.dev/222739))
* [article 90414](https://serlo-staging.dev/90414) ([current revision 222745](https://serlo-staging.dev/222745))
* [article 220072](https://serlo-staging.dev/220072) ([current revision 222746](https://serlo-staging.dev/222746))
* [article 181038](https://serlo-staging.dev/181038) ([current revision 222779](https://serlo-staging.dev/222779))
* [article 28296](https://serlo-staging.dev/28296) ([current revision 222786](https://serlo-staging.dev/222786))
* [article 108286](https://serlo-staging.dev/108286) ([current revision 222881](https://serlo-staging.dev/222881))
* [article 1777](https://serlo-staging.dev/1777) ([current revision 222911](https://serlo-staging.dev/222911))
* [article 184763](https://serlo-staging.dev/184763) ([current revision 222912](https://serlo-staging.dev/222912))
* [article 33401](https://serlo-staging.dev/33401) ([current revision 222974](https://serlo-staging.dev/222974))
* [course-page 221665](https://serlo-staging.dev/221665) ([current revision 222996](https://serlo-staging.dev/222996))
* [course-page 221666](https://serlo-staging.dev/221666) ([current revision 223005](https://serlo-staging.dev/223005))
* [course-page 221662](https://serlo-staging.dev/221662) ([current revision 223013](https://serlo-staging.dev/223013))
* [article 90412](https://serlo-staging.dev/90412) ([current revision 223073](https://serlo-staging.dev/223073))
* [article 120628](https://serlo-staging.dev/120628) ([current revision 223235](https://serlo-staging.dev/223235))
* [article 1693](https://serlo-staging.dev/1693) ([current revision 223403](https://serlo-staging.dev/223403))
* [article 114026](https://serlo-staging.dev/114026) ([current revision 223647](https://serlo-staging.dev/223647))
* [article 146844](https://serlo-staging.dev/146844) ([current revision 223663](https://serlo-staging.dev/223663))
* [course-page 117682](https://serlo-staging.dev/117682) ([current revision 223673](https://serlo-staging.dev/223673))
* [course-page 132454](https://serlo-staging.dev/132454) ([current revision 223696](https://serlo-staging.dev/223696))
* [text-solution 223801](https://serlo-staging.dev/223801) ([current revision 224002](https://serlo-staging.dev/224002))
* [article 1497](https://serlo-staging.dev/1497) ([current revision 224107](https://serlo-staging.dev/224107))
* [article 1499](https://serlo-staging.dev/1499) ([current revision 224109](https://serlo-staging.dev/224109))
* [course-page 136878](https://serlo-staging.dev/136878) ([current revision 224177](https://serlo-staging.dev/224177))
* [course-page 136882](https://serlo-staging.dev/136882) ([current revision 224179](https://serlo-staging.dev/224179))
* [course-page 136885](https://serlo-staging.dev/136885) ([current revision 224180](https://serlo-staging.dev/224180))
* [course-page 136974](https://serlo-staging.dev/136974) ([current revision 224181](https://serlo-staging.dev/224181))
* [course-page 136984](https://serlo-staging.dev/136984) ([current revision 224183](https://serlo-staging.dev/224183))
* [article 1501](https://serlo-staging.dev/1501) ([current revision 224215](https://serlo-staging.dev/224215))
* [course-page 221394](https://serlo-staging.dev/221394) ([current revision 224225](https://serlo-staging.dev/224225))
* [course-page 221400](https://serlo-staging.dev/221400) ([current revision 224261](https://serlo-staging.dev/224261))
* [article 1509](https://serlo-staging.dev/1509) ([current revision 224275](https://serlo-staging.dev/224275))
* [article 224316](https://serlo-staging.dev/224316) ([current revision 224318](https://serlo-staging.dev/224318))
* [course-page 117684](https://serlo-staging.dev/117684) ([current revision 224364](https://serlo-staging.dev/224364))
* [course-page 118489](https://serlo-staging.dev/118489) ([current revision 224365](https://serlo-staging.dev/224365))
* [course-page 118503](https://serlo-staging.dev/118503) ([current revision 224368](https://serlo-staging.dev/224368))
* [course-page 118973](https://serlo-staging.dev/118973) ([current revision 224393](https://serlo-staging.dev/224393))
* [course-page 118989](https://serlo-staging.dev/118989) ([current revision 224396](https://serlo-staging.dev/224396))
* [article 34188](https://serlo-staging.dev/34188) ([current revision 224426](https://serlo-staging.dev/224426))
* [article 211912](https://serlo-staging.dev/211912) ([current revision 224515](https://serlo-staging.dev/224515))
* [article 214826](https://serlo-staging.dev/214826) ([current revision 224516](https://serlo-staging.dev/224516))
* [article 1527](https://serlo-staging.dev/1527) ([current revision 224591](https://serlo-staging.dev/224591))
* [course-page 222210](https://serlo-staging.dev/222210) ([current revision 224603](https://serlo-staging.dev/224603))
* [course-page 129919](https://serlo-staging.dev/129919) ([current revision 224640](https://serlo-staging.dev/224640))
* [article 133367](https://serlo-staging.dev/133367) ([current revision 224658](https://serlo-staging.dev/224658))
* [article 202374](https://serlo-staging.dev/202374) ([current revision 224666](https://serlo-staging.dev/224666))
* [article 1563](https://serlo-staging.dev/1563) ([current revision 224875](https://serlo-staging.dev/224875))
* [text-solution 224970](https://serlo-staging.dev/224970) ([current revision 224972](https://serlo-staging.dev/224972))
* [text-solution 225018](https://serlo-staging.dev/225018) ([current revision 225020](https://serlo-staging.dev/225020))
* [article 225024](https://serlo-staging.dev/225024) ([current revision 225051](https://serlo-staging.dev/225051))
* [article 216333](https://serlo-staging.dev/216333) ([current revision 225082](https://serlo-staging.dev/225082))
* [course-page 222218](https://serlo-staging.dev/222218) ([current revision 225090](https://serlo-staging.dev/225090))
* [text-exercise 136414](https://serlo-staging.dev/136414) ([current revision 225157](https://serlo-staging.dev/225157))
* [grouped-text-exercise 225234](https://serlo-staging.dev/225234) ([current revision 225236](https://serlo-staging.dev/225236))
* [article 225026](https://serlo-staging.dev/225026) ([current revision 225244](https://serlo-staging.dev/225244))
* [course-page 56066](https://serlo-staging.dev/56066) ([current revision 225264](https://serlo-staging.dev/225264))
* [text-solution 225417](https://serlo-staging.dev/225417) ([current revision 225461](https://serlo-staging.dev/225461))
* [course-page 222271](https://serlo-staging.dev/222271) ([current revision 225552](https://serlo-staging.dev/225552))
* [course-page 225764](https://serlo-staging.dev/225764) ([current revision 225770](https://serlo-staging.dev/225770))
* [course-page 225765](https://serlo-staging.dev/225765) ([current revision 225805](https://serlo-staging.dev/225805))
* [course-page 225803](https://serlo-staging.dev/225803) ([current revision 225807](https://serlo-staging.dev/225807))
* [article 222619](https://serlo-staging.dev/222619) ([current revision 225854](https://serlo-staging.dev/225854))
* [course-page 225918](https://serlo-staging.dev/225918) ([current revision 225928](https://serlo-staging.dev/225928))
* [article 1593](https://serlo-staging.dev/1593) ([current revision 225972](https://serlo-staging.dev/225972))
* [course-page 225976](https://serlo-staging.dev/225976) ([current revision 225984](https://serlo-staging.dev/225984))
* [course-page 225979](https://serlo-staging.dev/225979) ([current revision 225987](https://serlo-staging.dev/225987))
* [course-page 225973](https://serlo-staging.dev/225973) ([current revision 225996](https://serlo-staging.dev/225996))
* [course-page 225992](https://serlo-staging.dev/225992) ([current revision 226003](https://serlo-staging.dev/226003))
* [text-solution 225358](https://serlo-staging.dev/225358) ([current revision 226101](https://serlo-staging.dev/226101))
* [text-solution 226100](https://serlo-staging.dev/226100) ([current revision 226103](https://serlo-staging.dev/226103))
* [article 1603](https://serlo-staging.dev/1603) ([current revision 226180](https://serlo-staging.dev/226180))
* [course-page 222212](https://serlo-staging.dev/222212) ([current revision 226196](https://serlo-staging.dev/226196))
* [course-page 226332](https://serlo-staging.dev/226332) ([current revision 226341](https://serlo-staging.dev/226341))
* [course-page 226333](https://serlo-staging.dev/226333) ([current revision 226342](https://serlo-staging.dev/226342))
* [course-page 226329](https://serlo-staging.dev/226329) ([current revision 226348](https://serlo-staging.dev/226348))
* [course-page 226331](https://serlo-staging.dev/226331) ([current revision 226350](https://serlo-staging.dev/226350))
* [article 169063](https://serlo-staging.dev/169063) ([current revision 226407](https://serlo-staging.dev/226407))
* [article 1585](https://serlo-staging.dev/1585) ([current revision 226416](https://serlo-staging.dev/226416))
* [text-solution 226461](https://serlo-staging.dev/226461) ([current revision 226463](https://serlo-staging.dev/226463))
* [article 133149](https://serlo-staging.dev/133149) ([current revision 226476](https://serlo-staging.dev/226476))
* [article 1599](https://serlo-staging.dev/1599) ([current revision 226482](https://serlo-staging.dev/226482))
* [course-page 216219](https://serlo-staging.dev/216219) ([current revision 226486](https://serlo-staging.dev/226486))
* [article 226495](https://serlo-staging.dev/226495) ([current revision 226500](https://serlo-staging.dev/226500))
* [article 215581](https://serlo-staging.dev/215581) ([current revision 226808](https://serlo-staging.dev/226808))
* [article 193313](https://serlo-staging.dev/193313) ([current revision 226810](https://serlo-staging.dev/226810))
* [article 134818](https://serlo-staging.dev/134818) ([current revision 226887](https://serlo-staging.dev/226887))
* [article 134867](https://serlo-staging.dev/134867) ([current revision 226927](https://serlo-staging.dev/226927))
* [course-page 117949](https://serlo-staging.dev/117949) ([current revision 227068](https://serlo-staging.dev/227068))
* [course-page 120833](https://serlo-staging.dev/120833) ([current revision 227077](https://serlo-staging.dev/227077))
* [article 134087](https://serlo-staging.dev/134087) ([current revision 227295](https://serlo-staging.dev/227295))
* [text-solution 5475](https://serlo-staging.dev/5475) ([current revision 227316](https://serlo-staging.dev/227316))
* [article 1617](https://serlo-staging.dev/1617) ([current revision 227337](https://serlo-staging.dev/227337))
* [text-solution 226561](https://serlo-staging.dev/226561) ([current revision 227348](https://serlo-staging.dev/227348))
* [course-page 72152](https://serlo-staging.dev/72152) ([current revision 227409](https://serlo-staging.dev/227409))
* [course-page 72194](https://serlo-staging.dev/72194) ([current revision 227412](https://serlo-staging.dev/227412))
* [course-page 227414](https://serlo-staging.dev/227414) ([current revision 227423](https://serlo-staging.dev/227423))
* [course-page 222954](https://serlo-staging.dev/222954) ([current revision 227436](https://serlo-staging.dev/227436))
* [course-page 227429](https://serlo-staging.dev/227429) ([current revision 227438](https://serlo-staging.dev/227438))
* [course-page 227453](https://serlo-staging.dev/227453) ([current revision 227461](https://serlo-staging.dev/227461))
* [course-page 227454](https://serlo-staging.dev/227454) ([current revision 227462](https://serlo-staging.dev/227462))
* [article 143547](https://serlo-staging.dev/143547) ([current revision 227509](https://serlo-staging.dev/227509))
* [course-page 118047](https://serlo-staging.dev/118047) ([current revision 227609](https://serlo-staging.dev/227609))
* [article 181031](https://serlo-staging.dev/181031) ([current revision 227664](https://serlo-staging.dev/227664))
* [course-page 106915](https://serlo-staging.dev/106915) ([current revision 227740](https://serlo-staging.dev/227740))
* [article 2109](https://serlo-staging.dev/2109) ([current revision 227741](https://serlo-staging.dev/227741))
* [text-solution 226563](https://serlo-staging.dev/226563) ([current revision 227775](https://serlo-staging.dev/227775))
* [article 2195](https://serlo-staging.dev/2195) ([current revision 227796](https://serlo-staging.dev/227796))
* [course-page 20326](https://serlo-staging.dev/20326) ([current revision 227835](https://serlo-staging.dev/227835))
* [course-page 21725](https://serlo-staging.dev/21725) ([current revision 227836](https://serlo-staging.dev/227836))
* [course-page 21729](https://serlo-staging.dev/21729) ([current revision 227837](https://serlo-staging.dev/227837))
* [course-page 20329](https://serlo-staging.dev/20329) ([current revision 227839](https://serlo-staging.dev/227839))
* [article 202923](https://serlo-staging.dev/202923) ([current revision 227855](https://serlo-staging.dev/227855))
* [course-page 227874](https://serlo-staging.dev/227874) ([current revision 227886](https://serlo-staging.dev/227886))
* [course-page 227875](https://serlo-staging.dev/227875) ([current revision 227887](https://serlo-staging.dev/227887))
* [course-page 227876](https://serlo-staging.dev/227876) ([current revision 227888](https://serlo-staging.dev/227888))
* [course-page 227877](https://serlo-staging.dev/227877) ([current revision 227889](https://serlo-staging.dev/227889))
* [article 32996](https://serlo-staging.dev/32996) ([current revision 227897](https://serlo-staging.dev/227897))
* [article 1627](https://serlo-staging.dev/1627) ([current revision 228033](https://serlo-staging.dev/228033))
* [text-solution 227767](https://serlo-staging.dev/227767) ([current revision 228217](https://serlo-staging.dev/228217))
* [article 226208](https://serlo-staging.dev/226208) ([current revision 228252](https://serlo-staging.dev/228252))
* [text-solution 228397](https://serlo-staging.dev/228397) ([current revision 228413](https://serlo-staging.dev/228413))
* [text-solution 228176](https://serlo-staging.dev/228176) ([current revision 228445](https://serlo-staging.dev/228445))
* [article 144838](https://serlo-staging.dev/144838) ([current revision 228465](https://serlo-staging.dev/228465))
* [text-solution 228010](https://serlo-staging.dev/228010) ([current revision 228468](https://serlo-staging.dev/228468))
* [text-solution 228549](https://serlo-staging.dev/228549) ([current revision 228551](https://serlo-staging.dev/228551))
* [text-solution 227643](https://serlo-staging.dev/227643) ([current revision 228553](https://serlo-staging.dev/228553))
* [text-solution 228563](https://serlo-staging.dev/228563) ([current revision 228572](https://serlo-staging.dev/228572))
* [text-solution 226762](https://serlo-staging.dev/226762) ([current revision 228581](https://serlo-staging.dev/228581))
* [text-solution 226829](https://serlo-staging.dev/226829) ([current revision 228582](https://serlo-staging.dev/228582))
* [text-solution 228588](https://serlo-staging.dev/228588) ([current revision 228593](https://serlo-staging.dev/228593))
* [text-solution 228590](https://serlo-staging.dev/228590) ([current revision 228595](https://serlo-staging.dev/228595))
* [text-solution 228561](https://serlo-staging.dev/228561) ([current revision 228687](https://serlo-staging.dev/228687))
* [text-solution 228691](https://serlo-staging.dev/228691) ([current revision 228720](https://serlo-staging.dev/228720))
* [text-solution 228293](https://serlo-staging.dev/228293) ([current revision 228780](https://serlo-staging.dev/228780))
* [article 227544](https://serlo-staging.dev/227544) ([current revision 228792](https://serlo-staging.dev/228792))
* [event 142218](https://serlo-staging.dev/142218) ([current revision 228807](https://serlo-staging.dev/228807))
* [text-solution 228712](https://serlo-staging.dev/228712) ([current revision 228813](https://serlo-staging.dev/228813))
* [text-solution 226850](https://serlo-staging.dev/226850) ([current revision 228819](https://serlo-staging.dev/228819))
* [article 1647](https://serlo-staging.dev/1647) ([current revision 228830](https://serlo-staging.dev/228830))
* [text-solution 13175](https://serlo-staging.dev/13175) ([current revision 228858](https://serlo-staging.dev/228858))
* [article 1659](https://serlo-staging.dev/1659) ([current revision 229056](https://serlo-staging.dev/229056))
* [article 131323](https://serlo-staging.dev/131323) ([current revision 229154](https://serlo-staging.dev/229154))
* [article 227033](https://serlo-staging.dev/227033) ([current revision 229158](https://serlo-staging.dev/229158))
* [text-solution 229238](https://serlo-staging.dev/229238) ([current revision 229240](https://serlo-staging.dev/229240))
* [article 226215](https://serlo-staging.dev/226215) ([current revision 229279](https://serlo-staging.dev/229279))
* [article 226224](https://serlo-staging.dev/226224) ([current revision 229280](https://serlo-staging.dev/229280))
* [article 2067](https://serlo-staging.dev/2067) ([current revision 229326](https://serlo-staging.dev/229326))
* [text-solution 228318](https://serlo-staging.dev/228318) ([current revision 229343](https://serlo-staging.dev/229343))
* [text-solution 229264](https://serlo-staging.dev/229264) ([current revision 229408](https://serlo-staging.dev/229408))
* [article 227342](https://serlo-staging.dev/227342) ([current revision 229427](https://serlo-staging.dev/229427))
* [text-solution 229097](https://serlo-staging.dev/229097) ([current revision 229668](https://serlo-staging.dev/229668))
* [text-solution 229271](https://serlo-staging.dev/229271) ([current revision 229876](https://serlo-staging.dev/229876))
* [article 228799](https://serlo-staging.dev/228799) ([current revision 229881](https://serlo-staging.dev/229881))
* [article 229014](https://serlo-staging.dev/229014) ([current revision 229882](https://serlo-staging.dev/229882))
* [text-solution 228698](https://serlo-staging.dev/228698) ([current revision 229895](https://serlo-staging.dev/229895))
* [course-page 229604](https://serlo-staging.dev/229604) ([current revision 229998](https://serlo-staging.dev/229998))
* [article 1765](https://serlo-staging.dev/1765) ([current revision 230047](https://serlo-staging.dev/230047))
* [text-solution 230075](https://serlo-staging.dev/230075) ([current revision 230077](https://serlo-staging.dev/230077))
* [course-page 119279](https://serlo-staging.dev/119279) ([current revision 230093](https://serlo-staging.dev/230093))
* [course-page 119277](https://serlo-staging.dev/119277) ([current revision 230094](https://serlo-staging.dev/230094))
* [course-page 132449](https://serlo-staging.dev/132449) ([current revision 230098](https://serlo-staging.dev/230098))
* [course-page 132458](https://serlo-staging.dev/132458) ([current revision 230099](https://serlo-staging.dev/230099))
* [course-page 134740](https://serlo-staging.dev/134740) ([current revision 230109](https://serlo-staging.dev/230109))
* [course-page 145632](https://serlo-staging.dev/145632) ([current revision 230114](https://serlo-staging.dev/230114))
* [course-page 120643](https://serlo-staging.dev/120643) ([current revision 230120](https://serlo-staging.dev/230120))
* [text-solution 230183](https://serlo-staging.dev/230183) ([current revision 230185](https://serlo-staging.dev/230185))
* [article 166759](https://serlo-staging.dev/166759) ([current revision 230215](https://serlo-staging.dev/230215))
* [article 215482](https://serlo-staging.dev/215482) ([current revision 230216](https://serlo-staging.dev/230216))
* [course-page 134971](https://serlo-staging.dev/134971) ([current revision 230290](https://serlo-staging.dev/230290))
* [course-page 134960](https://serlo-staging.dev/134960) ([current revision 230291](https://serlo-staging.dev/230291))
* [course-page 134838](https://serlo-staging.dev/134838) ([current revision 230294](https://serlo-staging.dev/230294))
* [article 134410](https://serlo-staging.dev/134410) ([current revision 230295](https://serlo-staging.dev/230295))
* [course-page 134789](https://serlo-staging.dev/134789) ([current revision 230296](https://serlo-staging.dev/230296))
* [text-solution 229777](https://serlo-staging.dev/229777) ([current revision 230307](https://serlo-staging.dev/230307))
* [text-solution 229779](https://serlo-staging.dev/229779) ([current revision 230308](https://serlo-staging.dev/230308))
* [text-solution 229781](https://serlo-staging.dev/229781) ([current revision 230309](https://serlo-staging.dev/230309))
* [text-solution 229783](https://serlo-staging.dev/229783) ([current revision 230310](https://serlo-staging.dev/230310))
* [article 227575](https://serlo-staging.dev/227575) ([current revision 230325](https://serlo-staging.dev/230325))
* [text-solution 229709](https://serlo-staging.dev/229709) ([current revision 230982](https://serlo-staging.dev/230982))
* [text-solution 229711](https://serlo-staging.dev/229711) ([current revision 230983](https://serlo-staging.dev/230983))
* [text-solution 229713](https://serlo-staging.dev/229713) ([current revision 230984](https://serlo-staging.dev/230984))
* [text-solution 229715](https://serlo-staging.dev/229715) ([current revision 230985](https://serlo-staging.dev/230985))
* [text-solution 179942](https://serlo-staging.dev/179942) ([current revision 231008](https://serlo-staging.dev/231008))
* [article 164911](https://serlo-staging.dev/164911) ([current revision 231108](https://serlo-staging.dev/231108))
* [article 1623](https://serlo-staging.dev/1623) ([current revision 231212](https://serlo-staging.dev/231212))
* [article 231195](https://serlo-staging.dev/231195) ([current revision 231224](https://serlo-staging.dev/231224))
* [article 198121](https://serlo-staging.dev/198121) ([current revision 231330](https://serlo-staging.dev/231330))
* [article 199259](https://serlo-staging.dev/199259) ([current revision 231337](https://serlo-staging.dev/231337))
* [article 129150](https://serlo-staging.dev/129150) ([current revision 231339](https://serlo-staging.dev/231339))
* [article 199564](https://serlo-staging.dev/199564) ([current revision 231341](https://serlo-staging.dev/231341))
* [article 1701](https://serlo-staging.dev/1701) ([current revision 231468](https://serlo-staging.dev/231468))
* [text-solution 116304](https://serlo-staging.dev/116304) ([current revision 231652](https://serlo-staging.dev/231652))
* [article 192262](https://serlo-staging.dev/192262) ([current revision 231746](https://serlo-staging.dev/231746))
* [article 231763](https://serlo-staging.dev/231763) ([current revision 231787](https://serlo-staging.dev/231787))
* [article 230494](https://serlo-staging.dev/230494) ([current revision 231812](https://serlo-staging.dev/231812))
* [text-solution 230618](https://serlo-staging.dev/230618) ([current revision 231925](https://serlo-staging.dev/231925))
* [text-solution 231990](https://serlo-staging.dev/231990) ([current revision 232047](https://serlo-staging.dev/232047))
* [course-page 90383](https://serlo-staging.dev/90383) ([current revision 232087](https://serlo-staging.dev/232087))
* [article 228140](https://serlo-staging.dev/228140) ([current revision 232137](https://serlo-staging.dev/232137))
* [text-solution 14517](https://serlo-staging.dev/14517) ([current revision 232524](https://serlo-staging.dev/232524))
* [article 1685](https://serlo-staging.dev/1685) ([current revision 232679](https://serlo-staging.dev/232679))
* [article 168549](https://serlo-staging.dev/168549) ([current revision 232715](https://serlo-staging.dev/232715))
* [article 160845](https://serlo-staging.dev/160845) ([current revision 232752](https://serlo-staging.dev/232752))
* [article 2171](https://serlo-staging.dev/2171) ([current revision 232861](https://serlo-staging.dev/232861))
* [article 167678](https://serlo-staging.dev/167678) ([current revision 232899](https://serlo-staging.dev/232899))
* [article 1847](https://serlo-staging.dev/1847) ([current revision 232907](https://serlo-staging.dev/232907))
* [article 1689](https://serlo-staging.dev/1689) ([current revision 232918](https://serlo-staging.dev/232918))
* [article 1619](https://serlo-staging.dev/1619) ([current revision 232929](https://serlo-staging.dev/232929))
* [text-solution 232720](https://serlo-staging.dev/232720) ([current revision 233119](https://serlo-staging.dev/233119))
* [text-solution 228741](https://serlo-staging.dev/228741) ([current revision 233195](https://serlo-staging.dev/233195))
* [text-solution 229234](https://serlo-staging.dev/229234) ([current revision 233672](https://serlo-staging.dev/233672))
* [course-page 202804](https://serlo-staging.dev/202804) ([current revision 233764](https://serlo-staging.dev/233764))
* [course-page 233767](https://serlo-staging.dev/233767) ([current revision 233771](https://serlo-staging.dev/233771))
* [article 204749](https://serlo-staging.dev/204749) ([current revision 233784](https://serlo-staging.dev/233784))
* [article 167791](https://serlo-staging.dev/167791) ([current revision 234024](https://serlo-staging.dev/234024))
* [article 1817](https://serlo-staging.dev/1817) ([current revision 234108](https://serlo-staging.dev/234108))
* [article 228149](https://serlo-staging.dev/228149) ([current revision 234111](https://serlo-staging.dev/234111))
* [article 228138](https://serlo-staging.dev/228138) ([current revision 234112](https://serlo-staging.dev/234112))
* [text-solution 229230](https://serlo-staging.dev/229230) ([current revision 234120](https://serlo-staging.dev/234120))
* [article 230333](https://serlo-staging.dev/230333) ([current revision 234162](https://serlo-staging.dev/234162))
* [article 230335](https://serlo-staging.dev/230335) ([current revision 234165](https://serlo-staging.dev/234165))
* [article 230337](https://serlo-staging.dev/230337) ([current revision 234167](https://serlo-staging.dev/234167))
* [course-page 233720](https://serlo-staging.dev/233720) ([current revision 234233](https://serlo-staging.dev/234233))
* [course-page 233719](https://serlo-staging.dev/233719) ([current revision 234283](https://serlo-staging.dev/234283))
* [course-page 233722](https://serlo-staging.dev/233722) ([current revision 234284](https://serlo-staging.dev/234284))
* [course-page 233723](https://serlo-staging.dev/233723) ([current revision 234285](https://serlo-staging.dev/234285))
* [course-page 233724](https://serlo-staging.dev/233724) ([current revision 234286](https://serlo-staging.dev/234286))
* [course-page 222284](https://serlo-staging.dev/222284) ([current revision 234328](https://serlo-staging.dev/234328))
* [article 1581](https://serlo-staging.dev/1581) ([current revision 234504](https://serlo-staging.dev/234504))
* [text-solution 234423](https://serlo-staging.dev/234423) ([current revision 234571](https://serlo-staging.dev/234571))
* [article 230343](https://serlo-staging.dev/230343) ([current revision 234621](https://serlo-staging.dev/234621))
* [article 230345](https://serlo-staging.dev/230345) ([current revision 234622](https://serlo-staging.dev/234622))
* [article 230347](https://serlo-staging.dev/230347) ([current revision 234624](https://serlo-staging.dev/234624))
* [article 230349](https://serlo-staging.dev/230349) ([current revision 234627](https://serlo-staging.dev/234627))
* [article 234551](https://serlo-staging.dev/234551) ([current revision 234679](https://serlo-staging.dev/234679))
* [text-solution 137572](https://serlo-staging.dev/137572) ([current revision 234715](https://serlo-staging.dev/234715))
* [text-solution 234586](https://serlo-staging.dev/234586) ([current revision 234749](https://serlo-staging.dev/234749))
* [course-page 106913](https://serlo-staging.dev/106913) ([current revision 234818](https://serlo-staging.dev/234818))
* [article 1631](https://serlo-staging.dev/1631) ([current revision 235093](https://serlo-staging.dev/235093))
* [article 54352](https://serlo-staging.dev/54352) ([current revision 235196](https://serlo-staging.dev/235196))
* [article 235304](https://serlo-staging.dev/235304) ([current revision 235376](https://serlo-staging.dev/235376))
* [article 117326](https://serlo-staging.dev/117326) ([current revision 235405](https://serlo-staging.dev/235405))
* [course-page 235390](https://serlo-staging.dev/235390) ([current revision 235525](https://serlo-staging.dev/235525))
* [course-page 235391](https://serlo-staging.dev/235391) ([current revision 235526](https://serlo-staging.dev/235526))
* [text-solution 231417](https://serlo-staging.dev/231417) ([current revision 235635](https://serlo-staging.dev/235635))
* [article 235561](https://serlo-staging.dev/235561) ([current revision 235641](https://serlo-staging.dev/235641))
* [course-page 235633](https://serlo-staging.dev/235633) ([current revision 235649](https://serlo-staging.dev/235649))
* [article 230366](https://serlo-staging.dev/230366) ([current revision 235912](https://serlo-staging.dev/235912))
* [article 210143](https://serlo-staging.dev/210143) ([current revision 235962](https://serlo-staging.dev/235962))
* [article 235653](https://serlo-staging.dev/235653) ([current revision 235968](https://serlo-staging.dev/235968))
* [article 230368](https://serlo-staging.dev/230368) ([current revision 236053](https://serlo-staging.dev/236053))
* [article 1727](https://serlo-staging.dev/1727) ([current revision 236057](https://serlo-staging.dev/236057))
* [course-page 235389](https://serlo-staging.dev/235389) ([current revision 236110](https://serlo-staging.dev/236110))
* [text-solution 236429](https://serlo-staging.dev/236429) ([current revision 236431](https://serlo-staging.dev/236431))
* [article 160405](https://serlo-staging.dev/160405) ([current revision 236608](https://serlo-staging.dev/236608))
* [article 230452](https://serlo-staging.dev/230452) ([current revision 236772](https://serlo-staging.dev/236772))
* [article 230454](https://serlo-staging.dev/230454) ([current revision 236780](https://serlo-staging.dev/236780))
* [article 235379](https://serlo-staging.dev/235379) ([current revision 236800](https://serlo-staging.dev/236800))
* [article 230456](https://serlo-staging.dev/230456) ([current revision 236812](https://serlo-staging.dev/236812))
* [article 1873](https://serlo-staging.dev/1873) ([current revision 236834](https://serlo-staging.dev/236834))
* [course-page 236212](https://serlo-staging.dev/236212) ([current revision 236888](https://serlo-staging.dev/236888))
* [article 230372](https://serlo-staging.dev/230372) ([current revision 236921](https://serlo-staging.dev/236921))
* [text-solution 11943](https://serlo-staging.dev/11943) ([current revision 237116](https://serlo-staging.dev/237116))
* [text-solution 11947](https://serlo-staging.dev/11947) ([current revision 237117](https://serlo-staging.dev/237117))
* [text-solution 231032](https://serlo-staging.dev/231032) ([current revision 237120](https://serlo-staging.dev/237120))
* [text-solution 231034](https://serlo-staging.dev/231034) ([current revision 237122](https://serlo-staging.dev/237122))
* [text-solution 231036](https://serlo-staging.dev/231036) ([current revision 237124](https://serlo-staging.dev/237124))
* [text-solution 231038](https://serlo-staging.dev/231038) ([current revision 237126](https://serlo-staging.dev/237126))
* [text-solution 231040](https://serlo-staging.dev/231040) ([current revision 237128](https://serlo-staging.dev/237128))
* [text-solution 231042](https://serlo-staging.dev/231042) ([current revision 237130](https://serlo-staging.dev/237130))
* [text-solution 237090](https://serlo-staging.dev/237090) ([current revision 237224](https://serlo-staging.dev/237224))
* [article 236208](https://serlo-staging.dev/236208) ([current revision 237314](https://serlo-staging.dev/237314))
* [course-page 237339](https://serlo-staging.dev/237339) ([current revision 237453](https://serlo-staging.dev/237453))
* [course-page 237340](https://serlo-staging.dev/237340) ([current revision 237454](https://serlo-staging.dev/237454))
* [course-page 237468](https://serlo-staging.dev/237468) ([current revision 237472](https://serlo-staging.dev/237472))
* [course-page 237469](https://serlo-staging.dev/237469) ([current revision 237473](https://serlo-staging.dev/237473))
* [article 1731](https://serlo-staging.dev/1731) ([current revision 237766](https://serlo-staging.dev/237766))
* [article 201259](https://serlo-staging.dev/201259) ([current revision 237767](https://serlo-staging.dev/237767))
* [article 1733](https://serlo-staging.dev/1733) ([current revision 237772](https://serlo-staging.dev/237772))
* [article 200358](https://serlo-staging.dev/200358) ([current revision 237773](https://serlo-staging.dev/237773))
* [course-page 236211](https://serlo-staging.dev/236211) ([current revision 237857](https://serlo-staging.dev/237857))
* [text-solution 237879](https://serlo-staging.dev/237879) ([current revision 237880](https://serlo-staging.dev/237880))
* [text-solution 235247](https://serlo-staging.dev/235247) ([current revision 237884](https://serlo-staging.dev/237884))
* [article 232901](https://serlo-staging.dev/232901) ([current revision 237935](https://serlo-staging.dev/237935))
* [article 201470](https://serlo-staging.dev/201470) ([current revision 238016](https://serlo-staging.dev/238016))
* [article 1737](https://serlo-staging.dev/1737) ([current revision 238017](https://serlo-staging.dev/238017))
* [article 202005](https://serlo-staging.dev/202005) ([current revision 238018](https://serlo-staging.dev/238018))
* [course-page 237487](https://serlo-staging.dev/237487) ([current revision 238022](https://serlo-staging.dev/238022))
* [article 202308](https://serlo-staging.dev/202308) ([current revision 238030](https://serlo-staging.dev/238030))
* [article 202484](https://serlo-staging.dev/202484) ([current revision 238032](https://serlo-staging.dev/238032))
* [article 1741](https://serlo-staging.dev/1741) ([current revision 238044](https://serlo-staging.dev/238044))
* [course-page 238091](https://serlo-staging.dev/238091) ([current revision 238097](https://serlo-staging.dev/238097))
* [course-page 238092](https://serlo-staging.dev/238092) ([current revision 238132](https://serlo-staging.dev/238132))
* [course-page 238094](https://serlo-staging.dev/238094) ([current revision 238134](https://serlo-staging.dev/238134))
* [article 202602](https://serlo-staging.dev/202602) ([current revision 238334](https://serlo-staging.dev/238334))
* [article 224642](https://serlo-staging.dev/224642) ([current revision 238348](https://serlo-staging.dev/238348))
* [article 70650](https://serlo-staging.dev/70650) ([current revision 238404](https://serlo-staging.dev/238404))
* [article 235384](https://serlo-staging.dev/235384) ([current revision 238454](https://serlo-staging.dev/238454))
* [article 2021](https://serlo-staging.dev/2021) ([current revision 238612](https://serlo-staging.dev/238612))
* [article 2137](https://serlo-staging.dev/2137) ([current revision 238614](https://serlo-staging.dev/238614))
* [article 170361](https://serlo-staging.dev/170361) ([current revision 238618](https://serlo-staging.dev/238618))
* [article 169197](https://serlo-staging.dev/169197) ([current revision 238620](https://serlo-staging.dev/238620))
* [article 169481](https://serlo-staging.dev/169481) ([current revision 238622](https://serlo-staging.dev/238622))
* [article 185618](https://serlo-staging.dev/185618) ([current revision 238624](https://serlo-staging.dev/238624))
* [article 212986](https://serlo-staging.dev/212986) ([current revision 238628](https://serlo-staging.dev/238628))
* [article 215223](https://serlo-staging.dev/215223) ([current revision 238630](https://serlo-staging.dev/238630))
* [article 192279](https://serlo-staging.dev/192279) ([current revision 238632](https://serlo-staging.dev/238632))
* [text-solution 28640](https://serlo-staging.dev/28640) ([current revision 238840](https://serlo-staging.dev/238840))
* [article 202957](https://serlo-staging.dev/202957) ([current revision 238887](https://serlo-staging.dev/238887))
* [article 1743](https://serlo-staging.dev/1743) ([current revision 238888](https://serlo-staging.dev/238888))
* [article 203204](https://serlo-staging.dev/203204) ([current revision 238890](https://serlo-staging.dev/238890))
* [article 203940](https://serlo-staging.dev/203940) ([current revision 238892](https://serlo-staging.dev/238892))
* [text-solution 235541](https://serlo-staging.dev/235541) ([current revision 238942](https://serlo-staging.dev/238942))
* [article 1759](https://serlo-staging.dev/1759) ([current revision 239005](https://serlo-staging.dev/239005))
* [article 231621](https://serlo-staging.dev/231621) ([current revision 239036](https://serlo-staging.dev/239036))
* [text-solution 21137](https://serlo-staging.dev/21137) ([current revision 239087](https://serlo-staging.dev/239087))
* [article 197411](https://serlo-staging.dev/197411) ([current revision 239289](https://serlo-staging.dev/239289))
* [article 197406](https://serlo-staging.dev/197406) ([current revision 239290](https://serlo-staging.dev/239290))
* [article 117722](https://serlo-staging.dev/117722) ([current revision 239294](https://serlo-staging.dev/239294))
* [course-page 62620](https://serlo-staging.dev/62620) ([current revision 239451](https://serlo-staging.dev/239451))
* [article 236738](https://serlo-staging.dev/236738) ([current revision 239620](https://serlo-staging.dev/239620))
* [text-solution 214542](https://serlo-staging.dev/214542) ([current revision 239651](https://serlo-staging.dev/239651))
* [text-solution 216946](https://serlo-staging.dev/216946) ([current revision 239653](https://serlo-staging.dev/239653))
* [text-solution 216948](https://serlo-staging.dev/216948) ([current revision 239655](https://serlo-staging.dev/239655))
* [course-page 58154](https://serlo-staging.dev/58154) ([current revision 239656](https://serlo-staging.dev/239656))
* [article 1773](https://serlo-staging.dev/1773) ([current revision 239670](https://serlo-staging.dev/239670))
* [course-page 90385](https://serlo-staging.dev/90385) ([current revision 239685](https://serlo-staging.dev/239685))
* [article 136597](https://serlo-staging.dev/136597) ([current revision 239687](https://serlo-staging.dev/239687))
* [article 132516](https://serlo-staging.dev/132516) ([current revision 239745](https://serlo-staging.dev/239745))
* [article 239649](https://serlo-staging.dev/239649) ([current revision 239802](https://serlo-staging.dev/239802))
* [article 239715](https://serlo-staging.dev/239715) ([current revision 239803](https://serlo-staging.dev/239803))
* [article 239638](https://serlo-staging.dev/239638) ([current revision 239885](https://serlo-staging.dev/239885))
* [article 220139](https://serlo-staging.dev/220139) ([current revision 239895](https://serlo-staging.dev/239895))
* [text-solution 235854](https://serlo-staging.dev/235854) ([current revision 240074](https://serlo-staging.dev/240074))
* [text-solution 235884](https://serlo-staging.dev/235884) ([current revision 240086](https://serlo-staging.dev/240086))
* [article 240300](https://serlo-staging.dev/240300) ([current revision 240349](https://serlo-staging.dev/240349))
* [article 204824](https://serlo-staging.dev/204824) ([current revision 240379](https://serlo-staging.dev/240379))
* [article 1801](https://serlo-staging.dev/1801) ([current revision 240409](https://serlo-staging.dev/240409))
* [article 1795](https://serlo-staging.dev/1795) ([current revision 240412](https://serlo-staging.dev/240412))
* [article 199799](https://serlo-staging.dev/199799) ([current revision 240436](https://serlo-staging.dev/240436))
* [article 1709](https://serlo-staging.dev/1709) ([current revision 240602](https://serlo-staging.dev/240602))
* [article 1819](https://serlo-staging.dev/1819) ([current revision 240612](https://serlo-staging.dev/240612))
* [article 1807](https://serlo-staging.dev/1807) ([current revision 240648](https://serlo-staging.dev/240648))
* [article 1833](https://serlo-staging.dev/1833) ([current revision 240665](https://serlo-staging.dev/240665))
* [article 1839](https://serlo-staging.dev/1839) ([current revision 240669](https://serlo-staging.dev/240669))
* [article 108667](https://serlo-staging.dev/108667) ([current revision 240673](https://serlo-staging.dev/240673))
* [article 108660](https://serlo-staging.dev/108660) ([current revision 240680](https://serlo-staging.dev/240680))
* [article 1881](https://serlo-staging.dev/1881) ([current revision 240695](https://serlo-staging.dev/240695))
* [article 1851](https://serlo-staging.dev/1851) ([current revision 240710](https://serlo-staging.dev/240710))
* [article 1853](https://serlo-staging.dev/1853) ([current revision 240711](https://serlo-staging.dev/240711))
* [article 1783](https://serlo-staging.dev/1783) ([current revision 240793](https://serlo-staging.dev/240793))
* [article 224320](https://serlo-staging.dev/224320) ([current revision 240858](https://serlo-staging.dev/240858))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [article 2213](https://serlo-staging.dev/2213) ([current revision 240899](https://serlo-staging.dev/240899))
* [article 239525](https://serlo-staging.dev/239525) ([current revision 240910](https://serlo-staging.dev/240910))
* [course-page 62609](https://serlo-staging.dev/62609) ([current revision 240915](https://serlo-staging.dev/240915))
* [article 234735](https://serlo-staging.dev/234735) ([current revision 240958](https://serlo-staging.dev/240958))
* [article 108246](https://serlo-staging.dev/108246) ([current revision 241006](https://serlo-staging.dev/241006))
* [article 105986](https://serlo-staging.dev/105986) ([current revision 241007](https://serlo-staging.dev/241007))
* [article 105900](https://serlo-staging.dev/105900) ([current revision 241015](https://serlo-staging.dev/241015))
* [article 105640](https://serlo-staging.dev/105640) ([current revision 241018](https://serlo-staging.dev/241018))
* [article 104206](https://serlo-staging.dev/104206) ([current revision 241022](https://serlo-staging.dev/241022))
* [article 239718](https://serlo-staging.dev/239718) ([current revision 241070](https://serlo-staging.dev/241070))
* [article 239698](https://serlo-staging.dev/239698) ([current revision 241071](https://serlo-staging.dev/241071))
* [article 1789](https://serlo-staging.dev/1789) ([current revision 241072](https://serlo-staging.dev/241072))
* [article 100082](https://serlo-staging.dev/100082) ([current revision 241121](https://serlo-staging.dev/241121))
* [article 99914](https://serlo-staging.dev/99914) ([current revision 241125](https://serlo-staging.dev/241125))
* [article 240545](https://serlo-staging.dev/240545) ([current revision 241204](https://serlo-staging.dev/241204))
* [article 2017](https://serlo-staging.dev/2017) ([current revision 241320](https://serlo-staging.dev/241320))
* [article 90400](https://serlo-staging.dev/90400) ([current revision 241353](https://serlo-staging.dev/241353))
* [article 140933](https://serlo-staging.dev/140933) ([current revision 241355](https://serlo-staging.dev/241355))
* [article 159135](https://serlo-staging.dev/159135) ([current revision 241356](https://serlo-staging.dev/241356))
* [article 165214](https://serlo-staging.dev/165214) ([current revision 241425](https://serlo-staging.dev/241425))
* [article 99597](https://serlo-staging.dev/99597) ([current revision 241439](https://serlo-staging.dev/241439))
* [article 1571](https://serlo-staging.dev/1571) ([current revision 241555](https://serlo-staging.dev/241555))
* [article 2063](https://serlo-staging.dev/2063) ([current revision 241583](https://serlo-staging.dev/241583))
* [article 1929](https://serlo-staging.dev/1929) ([current revision 241616](https://serlo-staging.dev/241616))
* [text-solution 241749](https://serlo-staging.dev/241749) ([current revision 241754](https://serlo-staging.dev/241754))
* [text-solution 241751](https://serlo-staging.dev/241751) ([current revision 241756](https://serlo-staging.dev/241756))
* [text-solution 241571](https://serlo-staging.dev/241571) ([current revision 241762](https://serlo-staging.dev/241762))
* [text-solution 241690](https://serlo-staging.dev/241690) ([current revision 241763](https://serlo-staging.dev/241763))
* [article 2095](https://serlo-staging.dev/2095) ([current revision 241851](https://serlo-staging.dev/241851))
* [article 1867](https://serlo-staging.dev/1867) ([current revision 241909](https://serlo-staging.dev/241909))
* [course-page 62611](https://serlo-staging.dev/62611) ([current revision 241946](https://serlo-staging.dev/241946))
* [article 170599](https://serlo-staging.dev/170599) ([current revision 241947](https://serlo-staging.dev/241947))
* [course-page 240914](https://serlo-staging.dev/240914) ([current revision 242050](https://serlo-staging.dev/242050))
* [article 26060](https://serlo-staging.dev/26060) ([current revision 242051](https://serlo-staging.dev/242051))
* [article 91050](https://serlo-staging.dev/91050) ([current revision 242092](https://serlo-staging.dev/242092))
* [article 104201](https://serlo-staging.dev/104201) ([current revision 242097](https://serlo-staging.dev/242097))
* [article 27801](https://serlo-staging.dev/27801) ([current revision 242132](https://serlo-staging.dev/242132))
* [article 158541](https://serlo-staging.dev/158541) ([current revision 242160](https://serlo-staging.dev/242160))
* [article 99517](https://serlo-staging.dev/99517) ([current revision 242181](https://serlo-staging.dev/242181))
* [text-solution 53550](https://serlo-staging.dev/53550) ([current revision 242213](https://serlo-staging.dev/242213))
* [article 26529](https://serlo-staging.dev/26529) ([current revision 242241](https://serlo-staging.dev/242241))
* [article 90418](https://serlo-staging.dev/90418) ([current revision 242270](https://serlo-staging.dev/242270))
* [article 26953](https://serlo-staging.dev/26953) ([current revision 242280](https://serlo-staging.dev/242280))
* [article 239676](https://serlo-staging.dev/239676) ([current revision 242284](https://serlo-staging.dev/242284))
* [course-page 93606](https://serlo-staging.dev/93606) ([current revision 242313](https://serlo-staging.dev/242313))
* [article 27276](https://serlo-staging.dev/27276) ([current revision 242384](https://serlo-staging.dev/242384))
* [article 24027](https://serlo-staging.dev/24027) ([current revision 242392](https://serlo-staging.dev/242392))
* [article 214865](https://serlo-staging.dev/214865) ([current revision 242422](https://serlo-staging.dev/242422))
* [article 241758](https://serlo-staging.dev/241758) ([current revision 242423](https://serlo-staging.dev/242423))
* [text-solution 236009](https://serlo-staging.dev/236009) ([current revision 242502](https://serlo-staging.dev/242502))
* [article 29291](https://serlo-staging.dev/29291) ([current revision 242536](https://serlo-staging.dev/242536))
* [article 29577](https://serlo-staging.dev/29577) ([current revision 242538](https://serlo-staging.dev/242538))
* [article 32567](https://serlo-staging.dev/32567) ([current revision 242562](https://serlo-staging.dev/242562))
* [article 33896](https://serlo-staging.dev/33896) ([current revision 242578](https://serlo-staging.dev/242578))
* [text-solution 37834](https://serlo-staging.dev/37834) ([current revision 242611](https://serlo-staging.dev/242611))
* [article 35621](https://serlo-staging.dev/35621) ([current revision 242691](https://serlo-staging.dev/242691))
* [article 35686](https://serlo-staging.dev/35686) ([current revision 242693](https://serlo-staging.dev/242693))
* [article 28557](https://serlo-staging.dev/28557) ([current revision 242806](https://serlo-staging.dev/242806))
* [article 36160](https://serlo-staging.dev/36160) ([current revision 242813](https://serlo-staging.dev/242813))
* [article 1767](https://serlo-staging.dev/1767) ([current revision 242870](https://serlo-staging.dev/242870))
* [article 86805](https://serlo-staging.dev/86805) ([current revision 242938](https://serlo-staging.dev/242938))
* [article 202973](https://serlo-staging.dev/202973) ([current revision 242966](https://serlo-staging.dev/242966))
* [article 242841](https://serlo-staging.dev/242841) ([current revision 242981](https://serlo-staging.dev/242981))
* [article 186937](https://serlo-staging.dev/186937) ([current revision 243041](https://serlo-staging.dev/243041))
* [article 240630](https://serlo-staging.dev/240630) ([current revision 243057](https://serlo-staging.dev/243057))
* [article 243066](https://serlo-staging.dev/243066) ([current revision 243111](https://serlo-staging.dev/243111))
* [course-page 65009](https://serlo-staging.dev/65009) ([current revision 243183](https://serlo-staging.dev/243183))
* [article 85858](https://serlo-staging.dev/85858) ([current revision 243234](https://serlo-staging.dev/243234))
* [article 36213](https://serlo-staging.dev/36213) ([current revision 243333](https://serlo-staging.dev/243333))
* [text-solution 13575](https://serlo-staging.dev/13575) ([current revision 243438](https://serlo-staging.dev/243438))
* [article 1565](https://serlo-staging.dev/1565) ([current revision 243496](https://serlo-staging.dev/243496))
* [article 243538](https://serlo-staging.dev/243538) ([current revision 243548](https://serlo-staging.dev/243548))
* [article 100589](https://serlo-staging.dev/100589) ([current revision 243586](https://serlo-staging.dev/243586))
* [article 100130](https://serlo-staging.dev/100130) ([current revision 243587](https://serlo-staging.dev/243587))
* [article 36520](https://serlo-staging.dev/36520) ([current revision 243599](https://serlo-staging.dev/243599))
* [article 46917](https://serlo-staging.dev/46917) ([current revision 243765](https://serlo-staging.dev/243765))
* [article 216527](https://serlo-staging.dev/216527) ([current revision 243788](https://serlo-staging.dev/243788))
* [article 25882](https://serlo-staging.dev/25882) ([current revision 243891](https://serlo-staging.dev/243891))
* [article 243665](https://serlo-staging.dev/243665) ([current revision 243901](https://serlo-staging.dev/243901))
* [article 239713](https://serlo-staging.dev/239713) ([current revision 243936](https://serlo-staging.dev/243936))
* [article 213172](https://serlo-staging.dev/213172) ([current revision 243943](https://serlo-staging.dev/243943))
* [article 93374](https://serlo-staging.dev/93374) ([current revision 243944](https://serlo-staging.dev/243944))
* [article 50583](https://serlo-staging.dev/50583) ([current revision 243958](https://serlo-staging.dev/243958))
* [text-solution 156614](https://serlo-staging.dev/156614) ([current revision 243973](https://serlo-staging.dev/243973))
* [article 243577](https://serlo-staging.dev/243577) ([current revision 243977](https://serlo-staging.dev/243977))
* [article 243174](https://serlo-staging.dev/243174) ([current revision 243980](https://serlo-staging.dev/243980))
* [article 36268](https://serlo-staging.dev/36268) ([current revision 243988](https://serlo-staging.dev/243988))
* [article 242919](https://serlo-staging.dev/242919) ([current revision 244058](https://serlo-staging.dev/244058))
* [article 243203](https://serlo-staging.dev/243203) ([current revision 244059](https://serlo-staging.dev/244059))
* [article 1945](https://serlo-staging.dev/1945) ([current revision 244060](https://serlo-staging.dev/244060))
* [text-solution 244090](https://serlo-staging.dev/244090) ([current revision 244095](https://serlo-staging.dev/244095))
* [text-solution 244092](https://serlo-staging.dev/244092) ([current revision 244097](https://serlo-staging.dev/244097))
* [course-page 244101](https://serlo-staging.dev/244101) ([current revision 244175](https://serlo-staging.dev/244175))
* [article 243913](https://serlo-staging.dev/243913) ([current revision 244227](https://serlo-staging.dev/244227))
* [article 244258](https://serlo-staging.dev/244258) ([current revision 244273](https://serlo-staging.dev/244273))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [article 27826](https://serlo-staging.dev/27826) ([current revision 244341](https://serlo-staging.dev/244341))
* [article 244359](https://serlo-staging.dev/244359) ([current revision 244377](https://serlo-staging.dev/244377))
* [article 48907](https://serlo-staging.dev/48907) ([current revision 244420](https://serlo-staging.dev/244420))
* [course-page 244310](https://serlo-staging.dev/244310) ([current revision 244491](https://serlo-staging.dev/244491))
* [article 244552](https://serlo-staging.dev/244552) ([current revision 244555](https://serlo-staging.dev/244555))
* [article 244544](https://serlo-staging.dev/244544) ([current revision 244557](https://serlo-staging.dev/244557))
* [article 231349](https://serlo-staging.dev/231349) ([current revision 244599](https://serlo-staging.dev/244599))
* [article 232392](https://serlo-staging.dev/232392) ([current revision 244600](https://serlo-staging.dev/244600))
* [article 231956](https://serlo-staging.dev/231956) ([current revision 244601](https://serlo-staging.dev/244601))
* [article 55761](https://serlo-staging.dev/55761) ([current revision 244663](https://serlo-staging.dev/244663))
* [article 131332](https://serlo-staging.dev/131332) ([current revision 244708](https://serlo-staging.dev/244708))
* [article 143554](https://serlo-staging.dev/143554) ([current revision 244709](https://serlo-staging.dev/244709))
* [article 244614](https://serlo-staging.dev/244614) ([current revision 244715](https://serlo-staging.dev/244715))
* [article 91567](https://serlo-staging.dev/91567) ([current revision 244831](https://serlo-staging.dev/244831))
* [article 171074](https://serlo-staging.dev/171074) ([current revision 244868](https://serlo-staging.dev/244868))
* [course-page 244890](https://serlo-staging.dev/244890) ([current revision 244935](https://serlo-staging.dev/244935))
* [course-page 244719](https://serlo-staging.dev/244719) ([current revision 244993](https://serlo-staging.dev/244993))
* [course-page 244972](https://serlo-staging.dev/244972) ([current revision 244994](https://serlo-staging.dev/244994))
* [course-page 244973](https://serlo-staging.dev/244973) ([current revision 244995](https://serlo-staging.dev/244995))
* [article 1849](https://serlo-staging.dev/1849) ([current revision 245008](https://serlo-staging.dev/245008))
* [article 239164](https://serlo-staging.dev/239164) ([current revision 245031](https://serlo-staging.dev/245031))
* [article 35149](https://serlo-staging.dev/35149) ([current revision 245032](https://serlo-staging.dev/245032))
* [text-solution 244879](https://serlo-staging.dev/244879) ([current revision 245068](https://serlo-staging.dev/245068))
* [text-solution 244407](https://serlo-staging.dev/244407) ([current revision 245122](https://serlo-staging.dev/245122))
* [text-solution 244409](https://serlo-staging.dev/244409) ([current revision 245123](https://serlo-staging.dev/245123))
* [article 154656](https://serlo-staging.dev/154656) ([current revision 245146](https://serlo-staging.dev/245146))
* [article 28295](https://serlo-staging.dev/28295) ([current revision 245179](https://serlo-staging.dev/245179))
* [article 242220](https://serlo-staging.dev/242220) ([current revision 245191](https://serlo-staging.dev/245191))
* [article 1845](https://serlo-staging.dev/1845) ([current revision 245200](https://serlo-staging.dev/245200))
* [text-solution 243474](https://serlo-staging.dev/243474) ([current revision 245202](https://serlo-staging.dev/245202))
* [course-page 243927](https://serlo-staging.dev/243927) ([current revision 245267](https://serlo-staging.dev/245267))
* [text-solution 78152](https://serlo-staging.dev/78152) ([current revision 245271](https://serlo-staging.dev/245271))
* [text-solution 76639](https://serlo-staging.dev/76639) ([current revision 245272](https://serlo-staging.dev/245272))
* [course-page 243926](https://serlo-staging.dev/243926) ([current revision 245274](https://serlo-staging.dev/245274))
* [text-solution 245294](https://serlo-staging.dev/245294) ([current revision 245299](https://serlo-staging.dev/245299))
* [article 243669](https://serlo-staging.dev/243669) ([current revision 245302](https://serlo-staging.dev/245302))
* [article 2025](https://serlo-staging.dev/2025) ([current revision 245323](https://serlo-staging.dev/245323))
* [article 30287](https://serlo-staging.dev/30287) ([current revision 245336](https://serlo-staging.dev/245336))
* [article 30393](https://serlo-staging.dev/30393) ([current revision 245338](https://serlo-staging.dev/245338))
* [article 34548](https://serlo-staging.dev/34548) ([current revision 245343](https://serlo-staging.dev/245343))
* [article 34896](https://serlo-staging.dev/34896) ([current revision 245346](https://serlo-staging.dev/245346))
* [article 43048](https://serlo-staging.dev/43048) ([current revision 245352](https://serlo-staging.dev/245352))
* [text-solution 245381](https://serlo-staging.dev/245381) ([current revision 245386](https://serlo-staging.dev/245386))
* [article 45643](https://serlo-staging.dev/45643) ([current revision 245400](https://serlo-staging.dev/245400))
* [text-exercise-group 245308](https://serlo-staging.dev/245308) ([current revision 245513](https://serlo-staging.dev/245513))
* [article 192834](https://serlo-staging.dev/192834) ([current revision 245587](https://serlo-staging.dev/245587))
* [article 47770](https://serlo-staging.dev/47770) ([current revision 245618](https://serlo-staging.dev/245618))
* [article 2199](https://serlo-staging.dev/2199) ([current revision 245642](https://serlo-staging.dev/245642))
* [article 216254](https://serlo-staging.dev/216254) ([current revision 245700](https://serlo-staging.dev/245700))
* [article 245448](https://serlo-staging.dev/245448) ([current revision 245756](https://serlo-staging.dev/245756))
* [article 243567](https://serlo-staging.dev/243567) ([current revision 245761](https://serlo-staging.dev/245761))
* [article 2027](https://serlo-staging.dev/2027) ([current revision 245765](https://serlo-staging.dev/245765))
* [text-solution 245292](https://serlo-staging.dev/245292) ([current revision 245791](https://serlo-staging.dev/245791))
* [text-solution 245383](https://serlo-staging.dev/245383) ([current revision 245799](https://serlo-staging.dev/245799))
* [article 49720](https://serlo-staging.dev/49720) ([current revision 245828](https://serlo-staging.dev/245828))
* [article 127689](https://serlo-staging.dev/127689) ([current revision 245835](https://serlo-staging.dev/245835))
* [article 226222](https://serlo-staging.dev/226222) ([current revision 245912](https://serlo-staging.dev/245912))
* [course-page 20322](https://serlo-staging.dev/20322) ([current revision 245917](https://serlo-staging.dev/245917))
* [event 196426](https://serlo-staging.dev/196426) ([current revision 246085](https://serlo-staging.dev/246085))
* [event 246046](https://serlo-staging.dev/246046) ([current revision 246113](https://serlo-staging.dev/246113))
* [event 246087](https://serlo-staging.dev/246087) ([current revision 246134](https://serlo-staging.dev/246134))
* [article 50515](https://serlo-staging.dev/50515) ([current revision 246155](https://serlo-staging.dev/246155))
* [course-page 246211](https://serlo-staging.dev/246211) ([current revision 246226](https://serlo-staging.dev/246226))
* [article 53246](https://serlo-staging.dev/53246) ([current revision 246308](https://serlo-staging.dev/246308))
* [article 52973](https://serlo-staging.dev/52973) ([current revision 246312](https://serlo-staging.dev/246312))
* [article 129708](https://serlo-staging.dev/129708) ([current revision 246323](https://serlo-staging.dev/246323))
* [article 52463](https://serlo-staging.dev/52463) ([current revision 246326](https://serlo-staging.dev/246326))
* [course-page 246203](https://serlo-staging.dev/246203) ([current revision 246401](https://serlo-staging.dev/246401))
* [course-page 246412](https://serlo-staging.dev/246412) ([current revision 246419](https://serlo-staging.dev/246419))
* [course-page 246447](https://serlo-staging.dev/246447) ([current revision 246460](https://serlo-staging.dev/246460))
* [course-page 20317](https://serlo-staging.dev/20317) ([current revision 246488](https://serlo-staging.dev/246488))
* [article 1677](https://serlo-staging.dev/1677) ([current revision 246499](https://serlo-staging.dev/246499))
* [article 176564](https://serlo-staging.dev/176564) ([current revision 246525](https://serlo-staging.dev/246525))
* [article 65231](https://serlo-staging.dev/65231) ([current revision 246583](https://serlo-staging.dev/246583))
* [article 174877](https://serlo-staging.dev/174877) ([current revision 246584](https://serlo-staging.dev/246584))
* [article 57029](https://serlo-staging.dev/57029) ([current revision 246588](https://serlo-staging.dev/246588))
* [course-page 246621](https://serlo-staging.dev/246621) ([current revision 246664](https://serlo-staging.dev/246664))
* [course-page 246622](https://serlo-staging.dev/246622) ([current revision 246665](https://serlo-staging.dev/246665))
* [course-page 246623](https://serlo-staging.dev/246623) ([current revision 246666](https://serlo-staging.dev/246666))
* [course-page 246624](https://serlo-staging.dev/246624) ([current revision 246667](https://serlo-staging.dev/246667))
* [course-page 246626](https://serlo-staging.dev/246626) ([current revision 246669](https://serlo-staging.dev/246669))
* [course-page 246245](https://serlo-staging.dev/246245) ([current revision 246671](https://serlo-staging.dev/246671))
* [article 224619](https://serlo-staging.dev/224619) ([current revision 246684](https://serlo-staging.dev/246684))
* [course-page 246748](https://serlo-staging.dev/246748) ([current revision 246759](https://serlo-staging.dev/246759))
* [course-page 246745](https://serlo-staging.dev/246745) ([current revision 246770](https://serlo-staging.dev/246770))
* [course-page 246619](https://serlo-staging.dev/246619) ([current revision 246782](https://serlo-staging.dev/246782))
* [course-page 216218](https://serlo-staging.dev/216218) ([current revision 246788](https://serlo-staging.dev/246788))
* [text-solution 5145](https://serlo-staging.dev/5145) ([current revision 247148](https://serlo-staging.dev/247148))
* [article 190844](https://serlo-staging.dev/190844) ([current revision 247171](https://serlo-staging.dev/247171))
* [article 57636](https://serlo-staging.dev/57636) ([current revision 247172](https://serlo-staging.dev/247172))
* [event 146500](https://serlo-staging.dev/146500) ([current revision 247191](https://serlo-staging.dev/247191))
* [article 112377](https://serlo-staging.dev/112377) ([current revision 247223](https://serlo-staging.dev/247223))
* [article 246693](https://serlo-staging.dev/246693) ([current revision 247225](https://serlo-staging.dev/247225))
* [text-solution 246845](https://serlo-staging.dev/246845) ([current revision 247236](https://serlo-staging.dev/247236))
* [article 224376](https://serlo-staging.dev/224376) ([current revision 247595](https://serlo-staging.dev/247595))
* [course-page 244718](https://serlo-staging.dev/244718) ([current revision 247626](https://serlo-staging.dev/247626))
* [article 247604](https://serlo-staging.dev/247604) ([current revision 247681](https://serlo-staging.dev/247681))
* [article 247139](https://serlo-staging.dev/247139) ([current revision 247798](https://serlo-staging.dev/247798))
* [article 247251](https://serlo-staging.dev/247251) ([current revision 248029](https://serlo-staging.dev/248029))
* [text-solution 9425](https://serlo-staging.dev/9425) ([current revision 248042](https://serlo-staging.dev/248042))
* [text-solution 9445](https://serlo-staging.dev/9445) ([current revision 248048](https://serlo-staging.dev/248048))
* [text-solution 9449](https://serlo-staging.dev/9449) ([current revision 248050](https://serlo-staging.dev/248050))
* [article 244678](https://serlo-staging.dev/244678) ([current revision 248070](https://serlo-staging.dev/248070))
* [article 1503](https://serlo-staging.dev/1503) ([current revision 248078](https://serlo-staging.dev/248078))
* [article 1511](https://serlo-staging.dev/1511) ([current revision 248085](https://serlo-staging.dev/248085))
* [article 62546](https://serlo-staging.dev/62546) ([current revision 248094](https://serlo-staging.dev/248094))
* [article 1513](https://serlo-staging.dev/1513) ([current revision 248103](https://serlo-staging.dev/248103))
* [article 1515](https://serlo-staging.dev/1515) ([current revision 248116](https://serlo-staging.dev/248116))
* [article 1715](https://serlo-staging.dev/1715) ([current revision 248173](https://serlo-staging.dev/248173))
* [article 1525](https://serlo-staging.dev/1525) ([current revision 248211](https://serlo-staging.dev/248211))
* [article 1535](https://serlo-staging.dev/1535) ([current revision 248219](https://serlo-staging.dev/248219))
* [article 71317](https://serlo-staging.dev/71317) ([current revision 248223](https://serlo-staging.dev/248223))
* [article 1537](https://serlo-staging.dev/1537) ([current revision 248228](https://serlo-staging.dev/248228))
* [article 71611](https://serlo-staging.dev/71611) ([current revision 248246](https://serlo-staging.dev/248246))
* [article 71867](https://serlo-staging.dev/71867) ([current revision 248253](https://serlo-staging.dev/248253))
* [article 71892](https://serlo-staging.dev/71892) ([current revision 248279](https://serlo-staging.dev/248279))
* [article 73649](https://serlo-staging.dev/73649) ([current revision 248280](https://serlo-staging.dev/248280))
* [article 74639](https://serlo-staging.dev/74639) ([current revision 248282](https://serlo-staging.dev/248282))
* [article 74829](https://serlo-staging.dev/74829) ([current revision 248284](https://serlo-staging.dev/248284))
* [article 78129](https://serlo-staging.dev/78129) ([current revision 248285](https://serlo-staging.dev/248285))
* [text-solution 248382](https://serlo-staging.dev/248382) ([current revision 248453](https://serlo-staging.dev/248453))
* [article 1539](https://serlo-staging.dev/1539) ([current revision 248632](https://serlo-staging.dev/248632))
* [article 74888](https://serlo-staging.dev/74888) ([current revision 248633](https://serlo-staging.dev/248633))
* [article 75079](https://serlo-staging.dev/75079) ([current revision 248635](https://serlo-staging.dev/248635))
* [article 1549](https://serlo-staging.dev/1549) ([current revision 248653](https://serlo-staging.dev/248653))
* [text-solution 248699](https://serlo-staging.dev/248699) ([current revision 248707](https://serlo-staging.dev/248707))
* [article 1591](https://serlo-staging.dev/1591) ([current revision 248726](https://serlo-staging.dev/248726))
* [text-solution 248697](https://serlo-staging.dev/248697) ([current revision 248742](https://serlo-staging.dev/248742))
* [event 146498](https://serlo-staging.dev/146498) ([current revision 248786](https://serlo-staging.dev/248786))
* [event 146512](https://serlo-staging.dev/146512) ([current revision 248787](https://serlo-staging.dev/248787))
* [event 150501](https://serlo-staging.dev/150501) ([current revision 248788](https://serlo-staging.dev/248788))
* [article 77192](https://serlo-staging.dev/77192) ([current revision 248807](https://serlo-staging.dev/248807))
* [article 77230](https://serlo-staging.dev/77230) ([current revision 248809](https://serlo-staging.dev/248809))
* [article 77333](https://serlo-staging.dev/77333) ([current revision 248810](https://serlo-staging.dev/248810))
* [article 77353](https://serlo-staging.dev/77353) ([current revision 248811](https://serlo-staging.dev/248811))
* [article 1559](https://serlo-staging.dev/1559) ([current revision 248816](https://serlo-staging.dev/248816))
* [article 1555](https://serlo-staging.dev/1555) ([current revision 248817](https://serlo-staging.dev/248817))
* [article 1561](https://serlo-staging.dev/1561) ([current revision 248818](https://serlo-staging.dev/248818))
* [article 1605](https://serlo-staging.dev/1605) ([current revision 248819](https://serlo-staging.dev/248819))
* [article 1717](https://serlo-staging.dev/1717) ([current revision 248827](https://serlo-staging.dev/248827))
* [article 1745](https://serlo-staging.dev/1745) ([current revision 248828](https://serlo-staging.dev/248828))
* [article 1753](https://serlo-staging.dev/1753) ([current revision 248829](https://serlo-staging.dev/248829))
* [article 1763](https://serlo-staging.dev/1763) ([current revision 248830](https://serlo-staging.dev/248830))
* [article 1787](https://serlo-staging.dev/1787) ([current revision 248831](https://serlo-staging.dev/248831))
* [article 1825](https://serlo-staging.dev/1825) ([current revision 248834](https://serlo-staging.dev/248834))
* [article 1837](https://serlo-staging.dev/1837) ([current revision 248836](https://serlo-staging.dev/248836))
* [article 1841](https://serlo-staging.dev/1841) ([current revision 248837](https://serlo-staging.dev/248837))
* [article 1869](https://serlo-staging.dev/1869) ([current revision 248842](https://serlo-staging.dev/248842))
* [article 1965](https://serlo-staging.dev/1965) ([current revision 248847](https://serlo-staging.dev/248847))
* [article 2003](https://serlo-staging.dev/2003) ([current revision 248850](https://serlo-staging.dev/248850))
* [article 2007](https://serlo-staging.dev/2007) ([current revision 248851](https://serlo-staging.dev/248851))
* [article 2059](https://serlo-staging.dev/2059) ([current revision 248855](https://serlo-staging.dev/248855))
* [article 2081](https://serlo-staging.dev/2081) ([current revision 248856](https://serlo-staging.dev/248856))
* [article 2115](https://serlo-staging.dev/2115) ([current revision 248858](https://serlo-staging.dev/248858))
* [article 2147](https://serlo-staging.dev/2147) ([current revision 248862](https://serlo-staging.dev/248862))
* [article 86745](https://serlo-staging.dev/86745) ([current revision 248868](https://serlo-staging.dev/248868))
* [article 78256](https://serlo-staging.dev/78256) ([current revision 248869](https://serlo-staging.dev/248869))
* [article 77078](https://serlo-staging.dev/77078) ([current revision 248871](https://serlo-staging.dev/248871))
* [article 69504](https://serlo-staging.dev/69504) ([current revision 248874](https://serlo-staging.dev/248874))
* [article 60970](https://serlo-staging.dev/60970) ([current revision 248875](https://serlo-staging.dev/248875))
* [article 55637](https://serlo-staging.dev/55637) ([current revision 248878](https://serlo-staging.dev/248878))
* [article 49388](https://serlo-staging.dev/49388) ([current revision 248881](https://serlo-staging.dev/248881))
* [article 49133](https://serlo-staging.dev/49133) ([current revision 248883](https://serlo-staging.dev/248883))
* [article 48767](https://serlo-staging.dev/48767) ([current revision 248884](https://serlo-staging.dev/248884))
* [article 28009](https://serlo-staging.dev/28009) ([current revision 248895](https://serlo-staging.dev/248895))
* [article 36070](https://serlo-staging.dev/36070) ([current revision 248897](https://serlo-staging.dev/248897))
* [article 41763](https://serlo-staging.dev/41763) ([current revision 248901](https://serlo-staging.dev/248901))
* [article 43118](https://serlo-staging.dev/43118) ([current revision 248903](https://serlo-staging.dev/248903))
* [article 43192](https://serlo-staging.dev/43192) ([current revision 248904](https://serlo-staging.dev/248904))
* [text-solution 248487](https://serlo-staging.dev/248487) ([current revision 248905](https://serlo-staging.dev/248905))
* [text-solution 248755](https://serlo-staging.dev/248755) ([current revision 248919](https://serlo-staging.dev/248919))
* [text-solution 248933](https://serlo-staging.dev/248933) ([current revision 248956](https://serlo-staging.dev/248956))
* [article 1551](https://serlo-staging.dev/1551) ([current revision 248977](https://serlo-staging.dev/248977))
* [article 1573](https://serlo-staging.dev/1573) ([current revision 248985](https://serlo-staging.dev/248985))
* [article 214839](https://serlo-staging.dev/214839) ([current revision 248986](https://serlo-staging.dev/248986))
* [article 214825](https://serlo-staging.dev/214825) ([current revision 248992](https://serlo-staging.dev/248992))
* [article 214823](https://serlo-staging.dev/214823) ([current revision 248993](https://serlo-staging.dev/248993))
* [article 214562](https://serlo-staging.dev/214562) ([current revision 248995](https://serlo-staging.dev/248995))
* [article 177769](https://serlo-staging.dev/177769) ([current revision 249017](https://serlo-staging.dev/249017))
* [article 1749](https://serlo-staging.dev/1749) ([current revision 249050](https://serlo-staging.dev/249050))
* [article 213139](https://serlo-staging.dev/213139) ([current revision 249072](https://serlo-staging.dev/249072))
* [article 212976](https://serlo-staging.dev/212976) ([current revision 249073](https://serlo-staging.dev/249073))
* [article 1577](https://serlo-staging.dev/1577) ([current revision 249079](https://serlo-staging.dev/249079))
* [article 212396](https://serlo-staging.dev/212396) ([current revision 249080](https://serlo-staging.dev/249080))
* [article 211060](https://serlo-staging.dev/211060) ([current revision 249082](https://serlo-staging.dev/249082))
* [article 199861](https://serlo-staging.dev/199861) ([current revision 249093](https://serlo-staging.dev/249093))
* [article 200685](https://serlo-staging.dev/200685) ([current revision 249095](https://serlo-staging.dev/249095))
* [article 209594](https://serlo-staging.dev/209594) ([current revision 249163](https://serlo-staging.dev/249163))
* [article 209254](https://serlo-staging.dev/209254) ([current revision 249164](https://serlo-staging.dev/249164))
* [article 205957](https://serlo-staging.dev/205957) ([current revision 249168](https://serlo-staging.dev/249168))
* [article 204851](https://serlo-staging.dev/204851) ([current revision 249170](https://serlo-staging.dev/249170))
* [article 203244](https://serlo-staging.dev/203244) ([current revision 249183](https://serlo-staging.dev/249183))
* [article 203177](https://serlo-staging.dev/203177) ([current revision 249187](https://serlo-staging.dev/249187))
* [article 202125](https://serlo-staging.dev/202125) ([current revision 249200](https://serlo-staging.dev/249200))
* [article 201038](https://serlo-staging.dev/201038) ([current revision 249205](https://serlo-staging.dev/249205))
* [article 199600](https://serlo-staging.dev/199600) ([current revision 249226](https://serlo-staging.dev/249226))
* [article 244476](https://serlo-staging.dev/244476) ([current revision 249304](https://serlo-staging.dev/249304))